In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('database').getOrCreate()

In [5]:
from pyspark.sql.functions import (col,expr,count,countDistinct,datediff,to_date,date_add,year,month,lag,lead,rank,max,min,round,
        sum,when,lit,desc,coalesce,abs,greatest,least,array,array_sort,substring, explode,collect_list,array_intersect,
        unix_timestamp,rank,dense_rank,least,greatest,row_number,array_join,expr,trim,lower,array,sort_array,
        array_distinct,size,initcap,length,date_format,to_timestamp,concat,regexp_extract,length,regexp_replace,
        sum as spark_sum,sqrt,length,last,abs,avg,flatten,to_timestamp,array_contains,array_intersect,array_union,
        array_distinct,element_at,array_remove,row_number,ceil,floor,dayofweek,monotonically_increasing_id,last,first,
        asc_nulls_last,hours,hour,aggregate,weekofyear,dayofmonth,date_sub,array_distinct,percentile_approx,from_unixtime,
        lpad,rpad,array_intersect,array_except,dayofmonth,dayofyear,weekofyear,month,quarter,year,last_day
                                  )
from pyspark.sql.types import (StructField,StructType,
                    IntegerType,StringType,DateType,TimestampType )
from pyspark.sql import Window
from pyspark.sql import Row
#rlike,contains,array_join,collect_list,substring,array_size,cast,stack(to unpivot),LATERAL VIEW 
#Window.orderBy("id").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
#Window.partitionBy("category").orderBy("id").rangeBetween(Window.currentRow, 1)
#row_number(), rank, dense_rank(), sum(), count(), max(), min()

#### 1 2 Second Highest Salary M

In [6]:
employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/1m_employee.csv'))

In [56]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [59]:
window_spec = Window.orderBy(col('salary').desc())

employee_df.withColumn('rnk',rank().over(window_spec))\
.withColumn('rnksalary',when((col('rnk') == 2),col('salary'))).select(max(col('rnksalary')).alias('second_high_salary')).show()

+------------------+
|second_high_salary|
+------------------+
|              null|
+------------------+



24/09/01 20:47:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [121]:
employee_df.createOrReplaceTempView('employeedf')

spark.sql('''
            select max(rnksalary) as second_high_salary from 
           (select salary, case when rnk = 2 then salary
                               else null end  as rnksalary from
           (select salary,rank() over(order by salary desc) rnk from employeedf))

        ''').show()

+------------------+
|second_high_salary|
+------------------+
|               200|
+------------------+



24/09/01 21:26:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 2 3 Nth Highest Salary M

In [7]:
employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/2m_employee.csv'))

In [64]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [122]:
window_spec = Window.orderBy(col('salary').desc())
n = 0

employee_df.withColumn('rnk',rank().over(window_spec))\
.withColumn('rnksalary',when((col('rnk') == n),col('salary'))).select(max(col('rnksalary')).alias('second_high_salary')).show()\



+------------------+
|second_high_salary|
+------------------+
|              null|
+------------------+



24/09/01 21:27:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [128]:
employee_df.createOrReplaceTempView('employeedf')

n = 1

spark.sql(f'''
            select max(rnksalary) as second_high_salary from 
           (select salary, case when rnk = {n} then salary
                               else null end  as rnksalary from
           (select salary,rank() over(order by salary desc) rnk from employeedf))

        ''').show()

+------------------+
|second_high_salary|
+------------------+
|               300|
+------------------+



24/09/01 21:30:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 3 4 Rank Scores M

In [8]:
scores_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/3m_scores.csv'))

In [108]:
scores_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- score: double (nullable = true)



In [111]:
window_spec  = Window.orderBy(col('score').desc())

scores_df.withColumn('rnk',dense_rank().over(window_spec)).show()


+---+-----+---+
| id|score|rnk|
+---+-----+---+
|  3|  4.0|  1|
|  5|  4.0|  1|
|  4| 3.85|  2|
|  2| 3.65|  3|
|  6| 3.65|  3|
|  1|  3.5|  4|
+---+-----+---+



24/09/01 21:17:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [130]:
scores_df.createOrReplaceTempView('scoresdf')

spark.sql('''
          select score,dense_rank() over(order by score desc) as rnk from scoresdf
         ''').show()

+-----+---+
|score|rnk|
+-----+---+
|  4.0|  1|
|  4.0|  1|
| 3.85|  2|
| 3.65|  3|
| 3.65|  3|
|  3.5|  4|
+-----+---+



24/09/01 21:34:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 4 5 Consecutive Numbers M

In [9]:
logs_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/4m_logs.csv'))

In [194]:
logs_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- num: integer (nullable = true)



In [195]:
window_spec = Window.orderBy(col('id').asc())

with_group_id = logs_df.withColumn('prev_num',lag(col('num'),1,0).over(window_spec))\
.withColumn('next_num',lead(col('num'),1,0).over(window_spec))\
.filter(
    (col("num") == col("prev_num")) &
    (col("num") == col("next_num"))
).select('num').show()


+---+
|num|
+---+
|  1|
+---+



24/09/01 22:39:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [209]:
logs_df.createOrReplaceTempView('logsdf')

spark.sql(''' 
         select num from 
         (select num, lag(num,1,0) over(order by id asc) as pre_num, 
                   lead(num,1,0) over(order by id asc) as next_num from logsdf) 
                   where num == pre_num and num == next_num
        '''
         ).show()

+---+
|num|
+---+
|  1|
+---+



24/09/01 22:49:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 5 9 Department Highest Salary M

In [10]:
employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/5m_employee.csv'))

In [11]:
department_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/5m_department.csv'))

In [212]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- departmentId: integer (nullable = true)



In [213]:
department_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [229]:
window_spec = Window.partitionBy(department_df.name)

employee_df.join(department_df,employee_df.departmentId == department_df.id,'inner')\
.withColumn('maxsalary',max(employee_df.salary).over(window_spec) )\
.filter(col('salary') == col('maxsalary'))\
.select( (department_df.name).alias('department'),(employee_df.name).alias('employee'),'salary' ).show()


+----------+--------+------+
|department|employee|salary|
+----------+--------+------+
|     Sales|   Henry| 80000|
|        IT|     Jim| 90000|
|        IT|     Max| 90000|
+----------+--------+------+



In [244]:
employee_df.createOrReplaceTempView('employeedf')
department_df.createOrReplaceTempView('departmentdf')

spark.sql('''
  with maxl as (select max(salary) over(partition by id) as salary from
                      (select employeedf.salary as salary,departmentdf.id as id  from employeedf
                      join departmentdf on employeedf.departmentId = departmentdf.id))
            select departmentdf.name as department,employeedf.name,employeedf.salary as salary from employeedf
                      join departmentdf on employeedf.departmentId = departmentdf.id
                      where salary in (select * from maxl)
         ''').show()

+----------+-----+------+
|department| name|salary|
+----------+-----+------+
|        IT|  Jim| 90000|
|     Sales|Henry| 80000|
|        IT|  Max| 90000|
+----------+-----+------+



#### 6 16 Game Play Analysis III M

In [12]:
activity_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/6m_activity.csv'))

In [5]:
activity_df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- event_date: string (nullable = true)
 |-- games_played: integer (nullable = true)



In [17]:
window_spec = Window.partitionBy('player_id').orderBy(col('event_date').asc())

activity_df.withColumn('games_played_so_far',sum('games_played').over(window_spec))\
.select('player_id','event_date','games_played_so_far').show()

+---------+----------+-------------------+
|player_id|event_date|games_played_so_far|
+---------+----------+-------------------+
|        1|2016-03-01|                  5|
|        1|2016-05-02|                 11|
|        1|2017-06-25|                 12|
|        3|2016-03-02|                  0|
|        3|2018-07-03|                  5|
+---------+----------+-------------------+



In [24]:
activity_df.createOrReplaceTempView('activitydf')


spark.sql('''

         select player_id,event_date,sum(games_played) over(partition By player_id order By event_date) as games_played_so_far
              from activitydf
         ''').show()


+---------+----------+-------------------+
|player_id|event_date|games_played_so_far|
+---------+----------+-------------------+
|        1|2016-03-01|                  5|
|        1|2016-05-02|                 11|
|        1|2017-06-25|                 12|
|        3|2016-03-02|                  0|
|        3|2018-07-03|                  5|
+---------+----------+-------------------+



#### 7 17 Game Play Analysis IV M

In [13]:
activity_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/7m_activity.csv'))

In [27]:
activity_df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- event_date: string (nullable = true)
 |-- games_played: integer (nullable = true)



In [113]:
window_spec = Window.partitionBy('player_id').orderBy(col('event_date').asc())

num_of_players = activity_df.select(countDistinct('player_id').alias('num_of_players')).collect()[0][0]

activity_df.withColumn('lead',lead('event_date',1).over(window_spec))\
.withColumn('datediff',datediff('lead','event_date'))\
.withColumn('fraction',when((col('datediff') == 1) ,round((col('datediff').cast('double')/ num_of_players),2)))\
.select(col('fraction')).dropna().show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+



In [187]:
activity_df.createOrReplaceTempView('activitydf')


spark.sql('''
 with main as (select player_id,event_date,lead(event_date) over(partition by player_id order by event_date asc) as lead_date
                    from activitydf),
       one as (select count(distinct player_id) as cnt, '1' as id from main),
       two as (select  datediff(lead_date,event_date) as date_dff, '1' as id from main)
               select round(date_dff/cnt,2)  as fraction from one join two on one.id = two.id where date_dff = 1
         ''').show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+



#### 8 19 Managers with at Least 5 Direct Reports M

In [14]:
employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/8m_employee.csv'))

In [189]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- managerId: integer (nullable = true)



In [204]:
window_spec = Window.partitionBy('managerId')
mrg_count_df = employee_df.filter(~col('managerId').isNull()).withColumn('mgr_count',count('managerId').over(window_spec))\
.select('managerId','mgr_count')

emp_df = employee_df.filter(col('managerId').isNull()).select('id','name')

emp_df.join(mrg_count_df,mrg_count_df.managerId == emp_df.id,'inner').filter(col('mgr_count') >= 5).select('name').distinct().show()

+----+
|name|
+----+
|John|
+----+



In [211]:
employee_df.createOrReplaceTempView('employeedf')

spark.sql('''
  with emp_id as (select managerId as mgr_id from employeedf where managerId is not null),
          mgr as (select name, id from employeedf where managerId is null)
                  select name from mgr join emp_id on mgr.id = emp_id.mgr_id group by name having count(name) >= 5 
        ''').show()


+----+
|name|
+----+
|John|
+----+



#### 9 25 Winning Candidate M

In [15]:
candidate_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/9m_candidate.csv'))

In [16]:
vote_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/9m_vote.csv'))

In [214]:
candidate_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [215]:
vote_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- candidateId: integer (nullable = true)



In [247]:
window_spec = Window.partitionBy('name')
result_df = candidate_df.join(vote_df,candidate_df.id == vote_df.candidateId,'inner')\
.withColumn('candiate_cnt',count('name').over(window_spec)).orderBy(col('candiate_cnt').desc())

result_df.first()[1][0]


'B'

In [256]:
candidate_df.createOrReplaceTempView('candidatedf')
vote_df.createOrReplaceTempView('votedf')

spark.sql('''
   with main as (select name,count(name) as cnt from candidatedf join votedf on candidatedf.id = votedf.candidateId
                  group by name),
         one as (select max(cnt) mx from main)
                 select name from main where cnt in (select mx from one)
         ''').show()

+----+
|name|
+----+
|   B|
+----+



#### 10 27 et Highest Answer Rate Question M

In [17]:
surveylog_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/10m_surveylog.csv'))

In [294]:
surveylog_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- action: string (nullable = true)
 |-- question_id: integer (nullable = true)
 |-- answer_id: integer (nullable = true)
 |-- q_num: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [287]:
window_spec = Window.partitionBy('qid')

answer_df = surveylog_df.filter(col('action') == lit('answer')).select(col('question_id').alias('qid'))

answer_df.join(surveylog_df,surveylog_df.question_id == answer_df.qid,'inner').select(col('qid')).distinct()\
.withColumn('cnt',count('qid').over(window_spec)).orderBy(col('cnt').desc(),col('qid').asc())\
.select(col('qid').alias('survey_log')).show()

+----------+
|survey_log|
+----------+
|       285|
+----------+



In [311]:
surveylog_df.createOrReplaceTempView('surveylogdf')

spark.sql('''
          select question_id from
          (select question_id,count(question_id) over(partition by question_id, action) as surlog from surveylogdf
              where action = "answer" order by surlog desc,question_id asc)
       ''').show()

+-----------+
|question_id|
+-----------+
|        285|
+-----------+



#### 11 29 Count Student Number in Departments M

In [18]:
student_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/11m_student.csv'))

In [19]:
department_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/11m_department.csv'))

In [5]:
student_df.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- student_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dept_id: integer (nullable = true)



In [6]:
department_df.printSchema()

root
 |-- dept_id: integer (nullable = true)
 |-- dept_name: string (nullable = true)



In [14]:
student_df.join(department_df,student_df.dept_id == department_df.dept_id,'right')\
.groupBy('dept_name').agg(count(student_df.dept_id).alias('student_number'))\
.orderBy(col('student_number').desc(),col('dept_name').asc()).show()


+-----------+--------------+
|  dept_name|student_number|
+-----------+--------------+
|Engineering|             2|
|    Science|             1|
|        Law|             0|
+-----------+--------------+



In [23]:
department_df.createOrReplaceTempView('departmentdf')
student_df.createOrReplaceTempView('studentdf')

spark.sql('''
         select departmentdf.dept_name,count(studentdf.dept_id) as student_number from studentdf
              right join departmentdf on studentdf.dept_id = departmentdf.dept_id 
              group by departmentdf.dept_name order by student_number desc,dept_name asc
          ''').show()

+-----------+--------------+
|  dept_name|student_number|
+-----------+--------------+
|Engineering|             2|
|    Science|             1|
|        Law|             0|
+-----------+--------------+



#### 12 31 Investments in 2016 M

In [20]:
insurance_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/12m_insurance.csv'))

In [25]:
insurance_df.printSchema()

root
 |-- pid: integer (nullable = true)
 |-- tiv_2015: integer (nullable = true)
 |-- tiv_2016: integer (nullable = true)
 |-- lat: integer (nullable = true)
 |-- lon: integer (nullable = true)



In [47]:
window_spec = Window.partitionBy('tiv_2015')
window_lon= Window.partitionBy('lon')

insurance_df.withColumn('gp2015',count('tiv_2015').over(window_spec))\
.withColumn('gplat',count('lat').over(window_lat))\
.withColumn('gplon',count('lon').over(window_lon)).filter( (col('gp2015') > 1) & (col('gplat') == 1) )\
.select(round(sum(col('tiv_2016')).cast('float'),2).alias('tiv_2016')).show()

+--------+
|tiv_2016|
+--------+
|    45.0|
+--------+



In [55]:
insurance_df.createOrReplaceTempView('insurancedf')

spark.sql('''
          
          select round(cast(sum(tiv_2016) as float),2) as tiv_2016 from 
          (select tiv_2016,count(tiv_2015) over(partition by tiv_2015) as gp2015, count(lat) over(partition by lat) as gplat
               from insurancedf) where gp2015 > 1 and gplat = 1
          ''').show()

+--------+
|tiv_2016|
+--------+
|    45.0|
+--------+



#### 13 37 Friend Requests II: Who Has the Most Friends M

In [21]:
requestaccepted_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/13m_requestaccepted.csv'))

In [98]:
requestaccepted_df.printSchema()

root
 |-- requester_id: integer (nullable = true)
 |-- accepter_id: integer (nullable = true)
 |-- accept_date: string (nullable = true)



In [99]:
window_spec = Window.partitionBy('accepter_id')

req_df = requestaccepted_df.withColumn('gp',count('accepter_id').over(window_spec)).filter(col('gp') == 1)\
.select( lit('0').alias('requester_id'),col('requester_id').alias('accepter_id'))

acc_df = requestaccepted_df.withColumn('gp',count('accepter_id').over(window_spec)).filter(col('gp') > 1)\
.select(lit('0').alias('requester_id'),col('accepter_id'))

req_df.union(acc_df).select('accepter_id').groupby('accepter_id').agg(count(col('accepter_id')).alias('count'))\
.filter(col('count') > 1).show()

+-----------+-----+
|accepter_id|count|
+-----------+-----+
|          3|    3|
+-----------+-----+



In [105]:
requestaccepted_df.show()

+------------+-----------+-----------+
|requester_id|accepter_id|accept_date|
+------------+-----------+-----------+
|           1|          2| 2016/06/03|
|           1|          3| 2016/06/08|
|           2|          3| 2016/06/08|
|           3|          4| 2016/06/09|
+------------+-----------+-----------+



In [137]:
requestaccepted_df.createOrReplaceTempView('requestaccepteddf')

spark.sql(''' 
  with acc as (select '0' as requester_id, requester_id as accepter_id from 
                    (select requester_id,accepter_id,count(accepter_id) over(partition by accepter_id) as cnt from requestaccepteddf)
               where cnt == 1),
       req as (select '0' as  requester_id,accepter_id from 
                    (select requester_id,accepter_id,count(accepter_id) over(partition by accepter_id) as cnt from requestaccepteddf)
               where cnt > 1)
              select accepter_id,count(accepter_id) as count from 
                   ((select * from req) union all (select * from acc))
              group by accepter_id having count > 1
         ''').show()

+-----------+-----+
|accepter_id|count|
+-----------+-----+
|          3|    3|
+-----------+-----+



#### 14 40 Tree Node M

In [22]:
tree_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/14m_tree.csv'))

In [139]:
tree_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- p_id: integer (nullable = true)



In [188]:
root = tree_df.filter(col('p_id').isNull()).select('id',lit('Root').alias('type'))

x = tree_df.filter(~col('p_id').isNull()).select('id')
y = tree_df.filter(~col('p_id').isNull()).select('p_id').collect()

leaf = x.filter(~col('id').isin(set([y[i][0] for i in range(0,len(y))]))).select('id',lit('leaff').alias('type'))

both = root.union(leaf).select('id').collect()

inner = tree_df.filter(~col('id').isin([ both[i][0] for i in range(0,len(both))])).select('id',lit('Inner').alias('type'))

root.union(leaf).union(inner).show()

+---+-----+
| id| type|
+---+-----+
|  1| Root|
|  3|leaff|
|  4|leaff|
|  5|leaff|
|  2|Inner|
+---+-----+



In [276]:
tree_df.createOrReplaceTempView('treedf')

spark.sql('''
  with main as ( (select id,'Root' as type from treedf where p_id is null) union all
                 (select id,'leaf' as type from treedf where id not in
                 (select p_id from treedf where p_id is not null)) ),
      inner as (select id,'Inner' as Type from treedf where id  not in (select id from main))
               (select * from main) union all (select * from inner)
         ''').show()

+---+-----+
| id| type|
+---+-----+
|  1| Root|
|  3| leaf|
|  4| leaf|
|  5| leaf|
|  2|Inner|
+---+-----+



#### 15 42 Shortest Distance in a Plane M

In [23]:
point2d_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/15m_point2d.csv'))

In [212]:
point2d_df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)



In [275]:
point2d_df.alias('df1').crossJoin(point2d_df.alias('df2'))\
.withColumn(
    "distance",
    sqrt(
        pow(col("df2.x") - col("df1.x"), 2) + pow(col("df2.y") - col("df1.y"), 2)
    )
).filter(col('distance') > 0 ).orderBy('distance').limit(1).show()

+---+---+---+---+--------+
|  x|  y|  x|  y|distance|
+---+---+---+---+--------+
| -1| -1| -1| -2|     1.0|
+---+---+---+---+--------+



In [308]:
point2d_df.createOrReplaceTempView('point2ddf')

spark.sql('''
 with main as (select x1,y1,x2,y2,sqrt(pow((x2 - x1),2) + pow((y2 - y1),2)) as distance  from 
                    (select df1.x as x1,df1.y as y1,df2.x as x2,df2.y as y2  from point2ddf as df1 cross join point2ddf as df2))
               select x1,y1,x2,y2,distance from main where distance > 0 order by distance limit 1


         ''').show()

+---+---+---+---+--------+
| x1| y1| x2| y2|distance|
+---+---+---+---+--------+
| -1| -1| -1| -2|     1.0|
+---+---+---+---+--------+



#### 16 44 Second Degree Follower M

In [24]:
follow_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/16m_follow.csv'))

In [310]:
follow_df.printSchema()

root
 |-- followee: string (nullable = true)
 |-- follower: string (nullable = true)



In [350]:
x = follow_df.select('follower').collect()

follower_num = follow_df.filter(col('followee').isin([ x[i][0] for i in range(0,len(x))]))\
.groupBy('followee').agg(count(col('follower')).alias('num'))
follower_num.show()

y = follower_num.select('followee').collect()
follow_df.filter(col('follower').isin([ y[i][0] for i in range(0,len(y))])).select('followee','follower').show()


+--------+---+
|followee|num|
+--------+---+
|     Bob|  2|
|  Donald|  1|
+--------+---+

+--------+--------+
|followee|follower|
+--------+--------+
|   Alice|     Bob|
|     Bob|  Donald|
+--------+--------+



In [355]:
follow_df.createOrReplaceTempView('followdf')


spark.sql('''
  with num_followee as (select followee,count(*) as num from followdf where followee in (select follower from followdf)
                             group by followee)
                       select followee from followdf where follower in (select followee from num_followee)
         ''').show()

+--------+
|followee|
+--------+
|   Alice|
|     Bob|
+--------+



#### 17 49 Exchange Seats M

In [25]:
seat_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/17m_seat.csv'))

In [357]:
seat_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- student: string (nullable = true)



In [418]:
window_spec = Window.orderBy("id").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing) # (0,10)

seat_df.select('id','student',last(col('id')).over(window_spec).alias('last'))\
.withColumn('ids',when( ( (col('last')%2 != 0) & (col('id') == col('last')) ),col('id') )
                 .when(col('id')%2 == 0,col('id') - 1).otherwise(col('id') + 1))\
.select(col('ids').alias('id'),'student').orderBy('id').show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



24/09/04 22:07:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [441]:
seat_df.createOrReplaceTempView('seatdf')

spark.sql('''
 with main as (select id,student,last(id) over(order by id rows between 
               UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) as idlast from seatdf),
       one as (select id,student,idlast,case when idlast%2 != 0 and idlast = id then id
                                             when id%2 == 0 then id - 1
                                             else id + 1 end ids from main)
               select ids as id,student from one order by id
          ''').show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



24/09/04 22:26:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 18 51 Customers Who Bought All Products M

In [26]:
customer_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/18m_customer.csv'))

In [27]:
product_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/18m_product.csv'))

In [444]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_key: integer (nullable = true)



In [445]:
product_df.printSchema()

root
 |-- product_key: integer (nullable = true)



In [456]:
prd_key = product_df.collect()
len_key = product_df.count()

customer_df.filter(col('product_key').isin([prd_key[i][0] for i in range(0,len(prd_key))]))\
.groupBy('customer_id').agg(count(col('product_key')).alias('cnt')).filter(col('cnt') == len_key).select('customer_id').show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+



In [459]:
customer_df.createOrReplaceTempView('customerdf')
product_df.createOrReplaceTempView('productdf')

spark.sql('''
 with main as (select customer_id from customerdf where product_key in (select product_key from productdf))
               select customer_id from main group by customer_id having count(customer_id) 
                    in (select count(*) from productdf)
        ''').show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+



#### 19 55 Product Sales Analysis III M

In [28]:
sales_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/19m_sales.csv'))

In [29]:
product_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/19m_product.csv'))

In [462]:
sales_df.printSchema()

root
 |-- sale_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: integer (nullable = true)



In [464]:
product_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)



In [470]:
window_spec = Window.partitionBy('product_id').orderBy(col('year').asc())

sales_df.join(product_df,sales_df.product_id == product_df.product_id,'right' )\
.select(sales_df.product_id,'year','quantity','price').withColumn('rnk',rank().over(window_spec))\
.filter( (~col('product_id').isNull()) & (col('rnk') == 1))\
.select('product_id',col('year').alias('first_year'),'quantity','price').show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+



In [481]:
sales_df.createOrReplaceTempView('salesdf')
product_df.createOrReplaceTempView('productdf')

spark.sql('''
 with main as (select salesdf.product_id as product_id,year,quantity,price from salesdf
                    right join productdf on salesdf.product_id = productdf.product_id),
       one as (select product_id,year as first_year, quantity,price, 
                    rank() over(partition by product_id order by year) as rnk from main)
               select product_id,first_year,quantity,price from one where product_id is not null and rnk =1
         

        ''').show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+



#### 20 58 Project Employees III M

In [30]:
products_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/20m_product.csv'))

In [31]:
employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/20m_employee.csv'))

In [485]:
products_df.printSchema()

root
 |-- project_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)



In [486]:
employee_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- experience_years: integer (nullable = true)



In [487]:
products_df.join(employee_df,products_df.employee_id == employee_df.employee_id,'inner').show()

+----------+-----------+-----------+------+----------------+
|project_id|employee_id|employee_id|  name|experience_years|
+----------+-----------+-----------+------+----------------+
|         2|          1|          1|Khaled|               3|
|         1|          1|          1|Khaled|               3|
|         1|          2|          2|   Ali|               2|
|         1|          3|          3|  John|               3|
|         2|          4|          4|   Doe|               2|
+----------+-----------+-----------+------+----------------+



In [497]:
window_spec = Window.partitionBy('project_id').orderBy(col(('experience_years')).desc())

products_df.join(employee_df,products_df.employee_id == employee_df.employee_id,'inner')\
.select('project_id',products_df.employee_id,'name','experience_years')\
.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1).select('project_id','employee_id')\
.orderBy(col('project_id').asc(),col('experience_years').desc()).show()

+----------+-----------+
|project_id|employee_id|
+----------+-----------+
|         1|          1|
|         1|          3|
|         2|          1|
+----------+-----------+



In [505]:
products_df.createOrReplaceTempView('productsdf')
employee_df.createOrReplaceTempView('employeedf')

spark.sql('''
  with main as (select project_id,productsdf.employee_id as employee_id,name,experience_years,
                     rank() over(partition by project_id order by experience_years desc) as rnk from productsdf
                     join employeedf on productsdf.employee_id = employeedf.employee_id)      
               select project_id,employee_id from main where rnk = 1 order by project_id asc,experience_years desc
          ''').show()

+----------+-----------+
|project_id|employee_id|
+----------+-----------+
|         1|          1|
|         1|          3|
|         2|          1|
+----------+-----------+



#### 21 63 Unpopular Books  M

In [32]:
books_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/21m_books.csv'))

In [33]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/21m_orders.csv'))
# Assume today is 2019-06-23.

In [5]:
books_df.printSchema()

root
 |-- book_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- available_from: string (nullable = true)



In [7]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- dispatch_date: string (nullable = true)



In [51]:
despatched_one_year_age = orders_df.select('order_id','book_id','quantity','dispatch_date',
abs(datediff(to_date(col('dispatch_date'),'yyyy-MM-dd'),to_date(lit('2019-06-23'),'yyyy-MM-dd'))).alias('diff') )\
.filter(col(('diff')) >= lit('365'))

In [52]:
one_month_ago = '2019-05-23'
available_one_month_ago = books_df.filter(to_date(col('available_from'), 'yyyy-MM-dd') <= lit(one_month_ago) )

In [54]:
available_one_month_ago.show()

+-------+----------------+--------------+
|book_id|            name|available_from|
+-------+----------------+--------------+
|      1|Kalila And Demna|    2010-01-01|
|      2|      28 Letters|    2012-05-12|
|      5|The Hunger Games|    2008-09-21|
+-------+----------------+--------------+



#### 22 64 New Users Daily Count M

In [34]:
traffic_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/22m_traffic.csv'))
# Assume today is 2019-06-30

In [58]:
traffic_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- activity: string (nullable = true)
 |-- activity_date: string (nullable = true)



In [82]:
user_before_one_month = traffic_df.filter(to_date(col('activity_date'),'yyyy-MM-dd') <= date_add(to_date(lit('2019-06-30'),'yyyy-MM-dd'),-90))\
.select('user_id').distinct().collect()

traffic_df.filter( (to_date(col('activity_date'),'yyyy-MM-dd') >= date_add(to_date(lit('2019-06-30'),'yyyy-MM-dd'),-90))
                & (~col('user_id').isin([ user_before_one_month[i][0] for i in range(0,len(user_before_one_month))])))\
.groupBy('activity_date').agg(countDistinct(col('user_id')).alias('user_count')).show()


+-------------+----------+
|activity_date|user_count|
+-------------+----------+
|   2019-05-01|         1|
|   2019-06-21|         2|
+-------------+----------+



In [90]:
traffic_df.createOrReplaceTempView('trafficdf')

spark.sql('''
 with main as (select activity_date,user_id from trafficdf where 
                (to_date(activity_date,'yyyy-MM-dd')  >= date_add(to_date('2019-06-30','yyyy-MM-dd'),-90)) and
                user_id not in (select user_id from trafficdf where to_date(activity_date,'yyyy-MM-dd')  <= date_add(to_date('2019-06-30','yyyy-MM-dd'),-90)) )
              select activity_date, count(distinct user_id) as user_count from main group by  activity_date  
         ''').show()

+-------------+----------+
|activity_date|user_count|
+-------------+----------+
|   2019-05-01|         1|
|   2019-06-21|         2|
+-------------+----------+



#### 23 65 Highest Grade For Each Student M

In [35]:
input_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/23m_input.csv'))

In [92]:
input_df.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- course_id: integer (nullable = true)
 |-- grade: integer (nullable = true)



In [93]:
input_df.show()

+----------+---------+-----+
|student_id|course_id|grade|
+----------+---------+-----+
|         2|        2|   95|
|         2|        3|   95|
|         1|        1|   90|
|         1|        2|   99|
|         3|        1|   80|
|         3|        2|   75|
|         3|        3|   82|
+----------+---------+-----+



In [117]:
window_spec = Window.partitionBy('student_id').orderBy(col('grade').desc())
min_spec = Window.partitionBy('student_id').orderBy(col('course_id').desc())

input_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1)\
.withColumn('minx',max('course_id').over(min_spec))\
.withColumn('tie',when( (count('student_id').over(window_spec)) >=2,col('student_id') ))\
.withColumn('xx',when( ((col('tie') >= 2) & (col('minx') == col('course_id'))),col('student_id') ))\
.filter(col('xx').isNull()).select('student_id','course_id','grade').orderBy(col('student_id').asc()).show()

+----------+---------+-----+
|student_id|course_id|grade|
+----------+---------+-----+
|         1|        2|   99|
|         2|        2|   95|
|         3|        3|   82|
+----------+---------+-----+



In [129]:
input_df.createOrReplaceTempView('inputdf')

spark.sql('''
 with main as (select student_id,course_id,grade,rank() over(partition by student_id order by grade desc) as rnk from inputdf),
       one as (select student_id,course_id,grade,count(student_id) over(partition by student_id ) as cnt,
                   min(course_id) over(partition by student_id order by course_id)  as minx from main where rnk = 1)
              select student_id,course_id,grade from one where minx = course_id order by student_id asc    
         ''').show()

+----------+---------+-----+
|student_id|course_id|grade|
+----------+---------+-----+
|         1|        2|   99|
|         2|        2|   95|
|         3|        3|   82|
+----------+---------+-----+



#### 24 67 Active Businesses M 

In [36]:
events_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/24m_events.csv'))

In [131]:
events_df.printSchema()

root
 |-- business_id: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- occurrences: integer (nullable = true)



In [132]:
events_df.show()

+-----------+----------+-----------+
|business_id|event_type|occurrences|
+-----------+----------+-----------+
|          1|   reviews|          7|
|          3|   reviews|          3|
|          1|       ads|         11|
|          2|       ads|          7|
|          3|       ads|          6|
|          1|page views|          3|
|          2|page views|         12|
+-----------+----------+-----------+



In [136]:
window_spec = Window.partitionBy('event_type')

events_df.withColumn('cnt',count('occurrences').over(window_spec)).withColumn('sm',sum('occurrences').over(window_spec)).show()

+-----------+----------+-----------+---+---+
|business_id|event_type|occurrences|cnt| sm|
+-----------+----------+-----------+---+---+
|          1|       ads|         11|  3| 24|
|          2|       ads|          7|  3| 24|
|          3|       ads|          6|  3| 24|
|          1|   reviews|          7|  2| 10|
|          3|   reviews|          3|  2| 10|
|          1|page views|          3|  2| 15|
|          2|page views|         12|  2| 15|
+-----------+----------+-----------+---+---+



In [142]:
events_df.withColumn('avg',sum('occurrences').over(window_spec)/count('occurrences').over(window_spec))\
.filter(col('occurrences') > col('avg')).groupBy('business_id').agg(count(col('business_id')).alias('buss_cnt'))\
.filter(col('buss_cnt') > 1).select('business_id').show()




+-----------+
|business_id|
+-----------+
|          1|
+-----------+



In [156]:
events_df.createOrReplaceTempView('eventsdf')

spark.sql('''
 with main as (select business_id,occurrences,sum(occurrences) over(partition by event_type)/count(occurrences) over(partition by event_type) as avg
                    from eventsdf),
       one as (select business_id,occurrences,avg,count(business_id) over(partition by business_id) bus_cnt from main where occurrences > avg)
               select distinct business_id from one where bus_cnt > 1  
        ''').show()

+-----------+
|business_id|
+-----------+
|          1|
+-----------+



#### 25 69 Reported Posts II M

In [37]:
actions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/25m_actions.csv'))

In [39]:
removals_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/25m_removals.csv'))

In [158]:
actions_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- post_id: integer (nullable = true)
 |-- action_date: string (nullable = true)
 |-- action: string (nullable = true)
 |-- extra: string (nullable = true)



In [160]:
removals_df.printSchema()

root
 |-- post_id: integer (nullable = true)
 |-- remove_date: string (nullable = true)



In [198]:
window_spec = Window.partitionBy('action_date')


actions_df.filter(col('extra') == lit('spam')).join(removals_df,actions_df.post_id == removals_df.post_id,'left')\
.withColumn('cnt',count('extra').over(window_spec))\
.withColumn('cnt_remove',count(removals_df.post_id).over(window_spec)).filter(~col('remove_date').isNull())\
.select(expr("avg(cnt_remove/cnt)").alias('average_daily_percent')).show()


+---------------------+
|average_daily_percent|
+---------------------+
|                 0.75|
+---------------------+



In [225]:
actions_df.createOrReplaceTempView('actionsdf')
removals_df.createOrReplaceTempView('removalsdf')

spark.sql('''
  with main as (select user_id,actionsdf.post_id as act_post_id,action_date,action,extra,removalsdf.post_id as rm_postid,
                     removalsdf.remove_date as rm_remove_date from actionsdf left join removalsdf on actionsdf.post_id = removalsdf.post_id
                     where extra = "spam"),
        one as (select user_id,act_post_id,action_date,rm_remove_date,count(extra) over(partition by action_date) as cnt, 
                     count(rm_postid) over(partition by action_date) as rm_cnt from main)
               select avg(rm_cnt/cnt) as average_daily_percent from one where rm_remove_date is not null

         ''').show()

+---------------------+
|average_daily_percent|
+---------------------+
|                 0.75|
+---------------------+



#### 26 73 Article Views II M

In [40]:
views_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/26m_views.csv'))

In [227]:
views_df.printSchema()

root
 |-- article_id: integer (nullable = true)
 |-- author_id: integer (nullable = true)
 |-- viewer_id: integer (nullable = true)
 |-- view_date: string (nullable = true)



In [233]:
window_spec = Window.partitionBy('view_date','viewer_id').orderBy(col('article_id').asc())

views_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 2).select(col('viewer_id').alias('id')).show()

+---+
| id|
+---+
|  5|
|  6|
+---+



In [236]:
views_df.createOrReplaceTempView('viewsdf')

spark.sql('''
        select viewer_id as id from
        (select viewer_id, rank() over(partition by view_date,viewer_id order by article_id) as rnk from viewsdf)
              where rnk = 2
       ''').show()

+---+
| id|
+---+
|  5|
|  6|
+---+



#### 27 76 Product Price at a Given Date M

In [41]:
products_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/27m_products.csv'))

In [238]:
products_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- new_price: integer (nullable = true)
 |-- change_date: string (nullable = true)



In [258]:
window_spec = Window.partitionBy('product_id')
window_rnk = Window.partitionBy('product_id').orderBy(col('change_date').desc())

products_df.withColumn('prd_cnt',count('product_id').over(window_spec))\
.withColumn('new_price',when(col('prd_cnt') == 1,10).otherwise(col('new_price')))\
.withColumn('change_date',when(col('prd_cnt') == 1,lit('2019-08-16') ).otherwise(col('change_date')))\
.filter((to_date(col('change_date'),'yyyy-MM-dd') <= lit('2019-08-16') ) )\
.withColumn('rnk',rank().over(window_rnk)).filter(col('rnk') == 1)\
.select('product_id',col('new_price').alias('price')).orderBy('product_id').show()



+----------+-----+
|product_id|price|
+----------+-----+
|         1|   35|
|         2|   50|
|         3|   10|
+----------+-----+



In [272]:
products_df.createOrReplaceTempView('productsdf')

spark.sql('''
  with main as (select product_id,new_price,change_date,count(product_id) over(partition by product_id) as cnt from productsdf),
        one as (select product_id,case when cnt = 1 then 10 else new_price end as new_price,
                                  case when cnt = 1 then "2019-08-16" else change_date end as change_date from main),       
        two as (select product_id,new_price,change_date from one where change_date <= "2019-08-16"),
      three as (select product_id,new_price,rank() over(partition by product_id order by change_date desc) rnk from two)
               select product_id,new_price as price from three where rnk = 1 order by product_id
         ''').show()

+----------+-----+
|product_id|price|
+----------+-----+
|         1|   35|
|         2|   50|
|         3|   10|
+----------+-----+



#### 28 78 Immediate Food Delivery II M

In [42]:
delivery_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/28m_delivery.csv'))

In [277]:
delivery_df.printSchema()

root
 |-- delivery_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_pref_delivery_date: string (nullable = true)



In [317]:
window_spec = Window.partitionBy('customer_id').orderBy(col('order_date').asc())
window_cnt = Window.partitionBy('rnk')

total_cnt = delivery_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1)\
.withColumn('schedule_cnt',when( (col('order_date') != col('customer_pref_delivery_date')),col('rnk')) )\
.withColumn('immediate',when( (col('order_date') == col('customer_pref_delivery_date')),col('rnk')))\
.select(sum(col('rnk')).alias('total_cnt')).collect()

scheddule_cnt = delivery_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1)\
.withColumn('schedule_cnt',when( (col('order_date') != col('customer_pref_delivery_date')),col('rnk')) )\
.withColumn('immediate',when( (col('order_date') == col('customer_pref_delivery_date')),col('rnk')))\
.filter(col('schedule_cnt') == 1 ).select(sum('schedule_cnt').alias('schedule_cnt')).collect()

immediate_cnt = delivery_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1)\
.withColumn('schedule_cnt',when( (col('order_date') != col('customer_pref_delivery_date')),col('rnk')) )\
.withColumn('immediate',when( (col('order_date') == col('customer_pref_delivery_date')),col('rnk')))\
.filter(col('immediate') == 1 ).select(sum('immediate').alias('immediate')).collect()

(scheddule_cnt[0][0]/total_cnt[0][0])*100

50.0

In [335]:
delivery_df.createOrReplaceTempView('productsdf')

spark.sql('''
 with main as (select delivery_id,customer_id,order_date,customer_pref_delivery_date,
                 rank() over(partition by customer_id order by order_date) as rnk from productsdf),
       one as (select delivery_id,customer_id,order_date,customer_pref_delivery_date,rnk from main where rnk = 1),
       two as (select  distinct count(rnk) over(partition by rnk) as total_cnt from one),  
 immediate as (select count(*) as immediate from one where order_date = customer_pref_delivery_date),
 scheduled as (select count(*) as scheduled from one where order_date != customer_pref_delivery_date)
               select (immediate/total_cnt * 100) as immediate_percentage  from two,immediate
         ''').show()

+--------------------+
|immediate_percentage|
+--------------------+
|                50.0|
+--------------------+



#### 29 80 Monthly Transactions I M

In [43]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/29m_transactions.csv'))

In [337]:
transactions_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- trans_date: string (nullable = true)



In [359]:
transactions_df.withColumn('trans_date',date_format('trans_date','yyyy-MM'))\
.groupBy('trans_date','country').agg(count('*').alias('trans_count'),
                                     sum(when(col('state') == lit('approved'),1).otherwise(0)).alias('approved_count'),
                                     sum(col('amount')).alias('trans_total_amount'),
                                     sum(when(col('state') == lit('approved'),col('amount')).otherwise(0)).alias('approved_total_amount')
                                    ).orderBy('trans_date').show()

+----------+-------+-----------+--------------+------------------+---------------------+
|trans_date|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+----------+-------+-----------+--------------+------------------+---------------------+
|   2018-12|     US|          2|             1|              3000|                 1000|
|   2019-01|     US|          1|             1|              2000|                 2000|
|   2019-01|     DE|          1|             1|              2000|                 2000|
+----------+-------+-----------+--------------+------------------+---------------------+



In [370]:
transactions_df.createOrReplaceTempView('transactionsdf')

spark.sql('''
 with main as (select date_format(trans_date,"yyyy-MM") as trans_date,country,state,amount from transactionsdf)
               select trans_date,country,count(*) as trans_count,
                        sum(case when state = "approved" then 1 else 0 end ) as approved_count,
                        sum(amount) as trans_total_amount,
                        sum(case when state = "approved" then amount else 0 end) as approved_total_amount 
                from main group by trans_date,country order by trans_date
         ''').show()

+----------+-------+-----------+--------------+------------------+---------------------+
|trans_date|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+----------+-------+-----------+--------------+------------------+---------------------+
|   2018-12|     US|          2|             1|              3000|                 1000|
|   2019-01|     US|          1|             1|              2000|                 2000|
|   2019-01|     DE|          1|             1|              2000|                 2000|
+----------+-------+-----------+--------------+------------------+---------------------+



#### 30 82 Last Person to Fit in the Bus M

In [44]:
queue_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/30m_queue.csv'))

In [372]:
queue_df.printSchema()

root
 |-- person_id: integer (nullable = true)
 |-- person_name: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- turn: integer (nullable = true)



In [390]:
window_spec = Window.orderBy("turn")

queue_df.withColumn('row_total',sum('weight').over(window_spec)).filter(col('row_total') == 1000)\
.select('person_name').show()

+-----------+
|person_name|
+-----------+
|  John Cena|
+-----------+



24/09/05 22:53:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [393]:
queue_df.createOrReplaceTempView('queue_df')

spark.sql('''
  with main as (select person_name,weight,turn,sum(weight) over(order by turn) as row_total from queue_df)
               select person_name from main where row_total = 1000
          ''').show()

+-----------+
|person_name|
+-----------+
|  John Cena|
+-----------+



24/09/05 22:56:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 31 83 Monthly Transactions II M

In [45]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/31m_transactions.csv'))

In [46]:
chargebacks_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/31m_chargebacks.csv'))

In [5]:
transactions_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- trans_date: string (nullable = true)



In [6]:
chargebacks_df.printSchema()

root
 |-- trans_id: integer (nullable = true)
 |-- trans_date: string (nullable = true)



In [494]:
transactions_df = transactions_df.withColumn('trans_date',date_format('trans_date','yyyy-MM'))
chargebacks_df = chargebacks_df.withColumn('trans_date',date_format('trans_date','yyyy-MM'))

transactions = transactions_df.groupby('trans_date','country').agg(sum(when( col('state') == lit('approved'),1).otherwise(0)).alias('approved_count'),
                                     sum(when( col('state') == lit('approved'),col('amount')).otherwise(0)).alias('approved_amount'))\

chargebacks = chargebacks_df.join(transactions_df,chargebacks_df.trans_id == transactions_df.id, 'left')\
.groupBy(chargebacks_df.trans_date,'country').agg(count('*').alias('chargeback_count'),
                                                 sum('amount').alias('chargeback_amount'))

transactions.join(chargebacks,['trans_date', 'country'],'full_outer' ).na.fill(0)\
.select('trans_date', 'country', 'approved_count', 'approved_amount', 'chargeback_count', 'chargeback_amount').orderBy('trans_date').show()


+----------+-------+--------------+---------------+----------------+-----------------+
|trans_date|country|approved_count|approved_amount|chargeback_count|chargeback_amount|
+----------+-------+--------------+---------------+----------------+-----------------+
|   2019-05|     US|             1|           1000|               1|             2000|
|   2019-06|     US|             2|           8000|               1|             1000|
|   2019-09|     US|             0|              0|               1|             5000|
+----------+-------+--------------+---------------+----------------+-----------------+



In [495]:
transactions_df = transactions_df.withColumn('trans_date',date_format('trans_date','yyyy-MM'))
chargebacks_df = chargebacks_df.withColumn('trans_date',date_format('trans_date','yyyy-MM'))

transactions = transactions_df.groupby('trans_date','country')\
                                .agg(sum(when( col('state') == lit('approved'),1).otherwise(0)).alias('approved_count'),
                                     sum(when( col('state') == lit('approved'),col('amount')).otherwise(0)).alias('approved_amount'))\
                                .withColumn('chargeback_count',lit('0'))\
                                .withColumn('chargeback_amount',lit('0'))

chargebacks = chargebacks_df.join(transactions_df,chargebacks_df.trans_id == transactions_df.id, 'left')\
.groupBy(chargebacks_df.trans_date,'country')\
                                .agg(count('*').alias('chargeback_count'),
                                     sum('amount').alias('chargeback_amount'))\
                                .withColumn('approved_count',lit('0'))\
                                .withColumn('approved_amount',lit('0'))\
                                .select('trans_date','country','approved_count','approved_amount','chargeback_count','chargeback_amount')

transactions.unionAll(chargebacks).groupBy('trans_date','country')\
              .agg(sum('approved_count').cast('int').alias('approved_count'),
                   sum('approved_amount').cast('int').alias('approved_amount'),
                   sum('chargeback_count').cast('int').alias('chargeback_count'),
                   sum('chargeback_amount').cast('int').alias('chargeback_amount')).orderBy('trans_date').show()


+----------+-------+--------------+---------------+----------------+-----------------+
|trans_date|country|approved_count|approved_amount|chargeback_count|chargeback_amount|
+----------+-------+--------------+---------------+----------------+-----------------+
|   2019-05|     US|             1|           1000|               1|             2000|
|   2019-06|     US|             2|           8000|               1|             1000|
|   2019-09|     US|             0|              0|               1|             5000|
+----------+-------+--------------+---------------+----------------+-----------------+



In [39]:
transactions_df.createOrReplaceTempView('transactionsdf')
chargebacks_df.createOrReplaceTempView('chargebacksdf')

spark.sql('''
  with main as (select date_format(trans_date,"yyyy-MM") as month,country,count(*) as approved_count,sum(amount) as approved_amount
                     from transactionsdf where  state = "approved" group by month,country),
        one as (select date_format(chargebacksdf.trans_date,"yyyy-MM") as month,country,amount
                     from chargebacksdf right join transactionsdf on chargebacksdf.trans_id = transactionsdf.id),
        two as (select month,country,count(month) as chargeback_count,sum(amount) as chargeback_amount
                     from one where month is not null group by month,country)
                select two.month,two.country,coalesce(approved_count,0) as approved_count,coalesce(approved_amount,0) as approved_amount,chargeback_count,chargeback_amount 
                     from main full outer join two on main.month = two.month and main.country = two.country
                     order by month
         ''').show()

+-------+-------+--------------+---------------+----------------+-----------------+
|  month|country|approved_count|approved_amount|chargeback_count|chargeback_amount|
+-------+-------+--------------+---------------+----------------+-----------------+
|2019-05|     US|             1|           1000|               1|             2000|
|2019-06|     US|             2|           8000|               1|             1000|
|2019-09|     US|             0|              0|               1|             5000|
+-------+-------+--------------+---------------+----------------+-----------------+



#### 32 85 Team Scores in Football Tournament M 

In [47]:
teams_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/32m_teams.csv'))

In [48]:
matches_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/32m_matches.csv'))

In [42]:
teams_df.printSchema()

root
 |-- team_id: integer (nullable = true)
 |-- team_name: string (nullable = true)



In [43]:
matches_df.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- host_team: integer (nullable = true)
 |-- guest_team: integer (nullable = true)
 |-- host_goals: integer (nullable = true)
 |-- guest_goals: integer (nullable = true)



In [69]:
one = matches_df.withColumnRenamed('host_team','team')\
                        .withColumn('num_points',when((col('host_goals') > col('guest_goals')),3)\
                        .when((col('host_goals') == col('guest_goals')),1)) \
                        .select('team','num_points')

two = matches_df.withColumnRenamed('guest_team','team') \
                        .withColumn('num_points',when((col('host_goals') < col('guest_goals')),3)\
                        .when((col('host_goals') == col('guest_goals')),1).otherwise(0)) \
                        .select('team','num_points')

one.union(two).groupBy(col('team')).agg(sum(col('num_points')).alias('num_points'))\
  .join(teams_df,one.team == teams_df.team_id,'right')\
  .select('team_id','team_name',coalesce(col('num_points'),lit('0')).alias('num_points')).show()


+-------+-----------+----------+
|team_id|  team_name|num_points|
+-------+-----------+----------+
|     10|Leetcode FC|         7|
|     20| NewYork FC|         3|
|     30| Atlanta FC|         1|
|     40| Chicago FC|         0|
|     50| Toronto FC|         3|
+-------+-----------+----------+



In [60]:
matches_df.createOrReplaceTempView('matchesdf')
teams_df.createOrReplaceTempView('teamsdf')

spark.sql('''
    with one as (select host_team as team, case when host_goals > guest_goals then 3 
                                             when host_goals == guest_goals then 1 end as num_points from matchesdf),
         two as (select guest_team as team,case when host_goals < guest_goals then 3 
                                              when host_goals == guest_goals then 1 end as num_points from matchesdf),
       three as (select team, sum(num_points) as num_points from (select * from one union all select * from two rigt) 
                                        group by team)
                 select team_id,team_name,coalesce(num_points,'0') as num_points from teamsdf left join three  on team_id == team
         ''').show()

+-------+-----------+----------+
|team_id|  team_name|num_points|
+-------+-----------+----------+
|     10|Leetcode FC|         7|
|     20| NewYork FC|         3|
|     30| Atlanta FC|         1|
|     40| Chicago FC|         0|
|     50| Toronto FC|         3|
+-------+-----------+----------+



#### 33 89 Page Recommendations M

In [49]:
friendship_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/33m_friendship.csv'))

In [50]:
likes_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/33m_likes.csv'))

In [72]:
friendship_df.printSchema()

root
 |-- user1_id: integer (nullable = true)
 |-- user2_id: integer (nullable = true)



In [73]:
likes_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- page_id: integer (nullable = true)



In [91]:
friendship_df.filter(col('user1_id') == 1).select(col('user2_id').alias('like'))\
.unionAll(friendship_df.filter(col('user2_id') == 1).select(col('user1_id').alias('like')))\
.join(likes_df,col('like') == likes_df.user_id,'inner' )\
.filter(~col('page_id').isin(likes_df.filter(col('user_id') == 1).select('page_id').collect()[0][0]))\
.select(col('page_id').alias('recommended_page')).distinct().orderBy('recommended_page').show()

+----------------+
|recommended_page|
+----------------+
|              23|
|              24|
|              33|
|              56|
|              77|
+----------------+



In [100]:
friendship_df.createOrReplaceTempView('friendshipdf')
likes_df.createOrReplaceTempView('likesdf')

spark.sql('''
 with main as ( (select user2_id as like from friendshipdf where user1_id == 1)
                     union(select user1_id as like from friendshipdf where user2_id == 1) )
            select distinct page_id from likesdf join main on likesdf.user_id = main.like 
                 where page_id not in (select page_id from likesdf where user_id = 1)
                 order by page_id
         ''').show()

+-------+
|page_id|
+-------+
|     23|
|     24|
|     33|
|     56|
|     77|
+-------+



#### 34 90 All People Report to the Given Manager M

In [51]:
employees_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/34m_employees.csv'))

In [102]:
employees_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- manager_id: integer (nullable = true)



In [132]:
one  = employees_df.filter( (col('manager_id') == 1) & (col('employee_id') != 1) ).select('employee_id').collect()
first = [one[i][0] for i in range(0,len(one))]

two  = employees_df.filter( (col('manager_id')).isin(first)).select('employee_id').collect()
second = [two[i][0] for i in range(0,len(two))]

three = employees_df.filter( (col('manager_id')).isin(second)).select('employee_id').collect()
third = [three[i][0] for i in range(0,len(three))]

spark.createDataFrame(one).union(spark.createDataFrame(two)).union(spark.createDataFrame(three))\
.orderBy('employee_id').show()


+-----------+
|employee_id|
+-----------+
|          2|
|          4|
|          7|
|         77|
+-----------+



[Row(employee_id=2), Row(employee_id=77)]

In [130]:
employees_df.show()

+-----------+-------------+----------+
|employee_id|employee_name|manager_id|
+-----------+-------------+----------+
|          1|         Boss|         1|
|          3|        Alice|         3|
|          2|          Bob|         1|
|          4|       Daniel|         2|
|          7|         Luis|         4|
|          8|         Jhon|         3|
|          9|       Angela|         8|
|         77|       Robert|         1|
+-----------+-------------+----------+



In [152]:
employees_df.createOrReplaceTempView('employeesdf')

spark.sql('''
  with one as (select employee_id from employeesdf where manager_id = 1 and employee_id != 1),
       two as (select employee_id from employeesdf where manager_id in (select * from one)),
     three as (select employee_id from employeesdf where manager_id in (select * from two))
               (select * from one)union(select * from two)union(select * from three) order by employee_id
         ''').show()

+-----------+
|employee_id|
+-----------+
|          2|
|          4|
|          7|
|         77|
+-----------+



#### 35 92  Find the Start and End Number of Continuous Ranges M

In [52]:
logs_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/35m_logs.csv'))

In [155]:
logs_df.printSchema()

root
 |-- log_id: integer (nullable = true)



In [193]:
window_spec = Window.orderBy("log_id")

df_with_lag = logs_df.withColumn("prev_log_id", lag("log_id").over(window_spec))

df_with_breaks = df_with_lag.withColumn("is_new_group", when((col("log_id") - col("prev_log_id")) > 1, 1).otherwise(0))

df_with_groups = df_with_breaks.withColumn("group_id", sum("is_new_group").over(Window.orderBy("log_id")))

logic_df = df_with_groups.withColumn('strend',rank().over(Window.partitionBy('group_id').orderBy('log_id')))\
.withColumn('start',when(col('strend') == min('strend').over(Window.partitionBy('group_id')),col('log_id')).otherwise(0))\
.withColumn('end',when(col('strend')   == max('strend').over(Window.partitionBy('group_id')),col('log_id')).otherwise(0))

start_df = logic_df.select('start').withColumn('rnk',dense_rank().over(Window.orderBy(col('start')))).filter(col('rnk') != 1)
end_df = logic_df.select('end').withColumn('rnk',dense_rank().over(Window.orderBy(col('end')))).filter(col('rnk') != 1)

start_df.join(end_df,start_df.rnk == end_df.rnk,'inner').select('start','end').show()

24/09/06 14:52:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 14:52:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 14:52:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 14:52:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 14:52:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 14:52:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+---+
|start|end|
+-----+---+
|    1|  3|
|    7|  8|
|   10| 10|
+-----+---+



In [197]:
logs_df.createOrReplaceTempView('logsdf')

spark.sql('''
  with main as (select log_id, sum(is_new_group) OVER(ORDER BY log_id) as group_id  from 
                    (select log_id,case when (log_id - prev_log_id) > 1 then 1 else 0  end is_new_group from 
                        (select log_id,lag(log_id) OVER(ORDER BY log_id) as prev_log_id from logsdf)) group by log_id,is_new_group),    
      start as (select distinct min(log_id) OVER(PARTITION BY group_id ) as start from main),
        end as (select distinct max(log_id) OVER(PARTITION BY group_id ) as end from main),
     end_id as (select end,row_number() OVER(ORDER BY end) as row_id from end), 
   start_id as (select start,row_number() OVER(ORDER BY start) as row_id from start)
                select start,end from start_id join end_id on start_id.row_id == end_id.row_id
         ''').show()

24/09/06 15:00:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 15:00:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 15:00:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 15:00:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 15:00:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 15:00:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+---+
|start|end|
+-----+---+
|    1|  3|
|    7|  8|
|   10| 10|
+-----+---+



#### 36 95 Running Total for Different Genders M

In [53]:
scores_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/36m_scores.csv'))

In [199]:
scores_df.printSchema()

root
 |-- player_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- day: string (nullable = true)
 |-- score_points: integer (nullable = true)



In [247]:
window_spec = Window.partitionBy('gender').orderBy(col('day'))
window_rnk  = Window.partitionBy('gender').orderBy(col('rnk').asc())

scores_df.withColumn('rnk',row_number().over(window_spec))\
.withColumn('total',sum('score_points').over(window_rnk)).select('gender','day','total').show()


+------+----------+-----+
|gender|       day|total|
+------+----------+-----+
|     F|2019-12-30|   17|
|     F|2019-12-31|   40|
|     F|2020-01-01|   57|
|     F|2020-01-07|   80|
|     M|2019-12-18|    2|
|     M|2019-12-25|   13|
|     M|2019-12-30|   26|
|     M|2019-12-31|   29|
|     M|2020-01-07|   36|
+------+----------+-----+



In [251]:
scores_df.createOrReplaceTempView('scoresdf')

spark.sql('''
          select gender,day,sum(score_points) over(partition by gender order by rnk) as total from
          (select gender,day,score_points,row_number() over(partition by gender order by day asc) as rnk from scoresdf)
        ''').show()

+------+----------+-----+
|gender|       day|total|
+------+----------+-----+
|     F|2019-12-30|   17|
|     F|2019-12-31|   40|
|     F|2020-01-01|   57|
|     F|2020-01-07|   80|
|     M|2019-12-18|    2|
|     M|2019-12-25|   13|
|     M|2019-12-30|   26|
|     M|2019-12-31|   29|
|     M|2020-01-07|   36|
+------+----------+-----+



#### 37 96 Restaurant Growth M

In [54]:
customer_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/37m_customer.csv'))

In [254]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- visited_on: string (nullable = true)
 |-- amount: integer (nullable = true)



In [286]:
window_spec = Window.orderBy(col('visited_on').asc()).rowsBetween(0,6)

customer_df.groupBy('visited_on').agg(sum(col('amount')).alias('amount')).orderBy('visited_on')\
.withColumn('xxx',sum('amount').over(window_spec))\
.withColumn('yyy',lead('visited_on',6,0).over(Window.orderBy('visited_on')))\
.withColumn('average_amount',round(col('xxx')/7,2)  ).filter(col('yyy') != lit('0') )\
.select(col('yyy').alias('visited_on'),col('xxx').alias('amount'),'average_amount').show()

24/09/06 18:49:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 18:49:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+------+--------------+
|visited_on|amount|average_amount|
+----------+------+--------------+
|2019-01-07|   860|        122.86|
|2019-01-08|   840|         120.0|
|2019-01-09|   840|         120.0|
|2019-01-10|  1000|        142.86|
+----------+------+--------------+



In [306]:
customer_df.createOrReplaceTempView('customerdf')

spark.sql('''
 with main as (select visited_on,sum(amount) as amount from customerdf group by visited_on order by visited_on asc),
       one as (select lead(visited_on,6,0) over(order by visited_on) as visited_on,
                    sum(amount) over(order by visited_on rows between 0 PRECEDING and 6 FOLLOWING) as amount from main)
              select visited_on,amount,round((amount/7),2) as average_amount from one where visited_on != "0"
         ''').show()

24/09/06 19:05:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+------+--------------+
|visited_on|amount|average_amount|
+----------+------+--------------+
|2019-01-07|   860|        122.86|
|2019-01-08|   840|         120.0|
|2019-01-09|   840|         120.0|
|2019-01-10|  1000|        142.86|
+----------+------+--------------+



#### 38 100 Movie Rating M

In [55]:
movies_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/38m_movies.csv'))

In [56]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/38m_users.csv'))

In [57]:
movierating_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/38m_movierating.csv'))

In [348]:
movies_df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)



In [349]:
users_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [350]:
movierating_df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- created_at: string (nullable = true)



In [366]:

result = users_df.join(movierating_df.select('user_id'),on = 'user_id')\
.groupBy(users_df.name).agg(count(users_df.name).alias('cnt'))\
.withColumn('rnk',rank().over(Window.orderBy(col('cnt').desc()))).filter(col('rnk') == 1)\
.select('name').orderBy('name').first()[0]

window_spec = Window.partitionBy('movie_id')

movierating = movierating_df.withColumn('created_at',date_format('created_at','yyyy-MM')).filter(col('created_at') ==lit('2020-02') )\
           .select('movie_id','rating','created_at')

moviename = movies_df.join(movierating,on = 'movie_id').withColumn('cnt',count('movie_id').over(window_spec))\
.withColumn('sm',sum('rating').over(window_spec)).select('title',expr("sm/cnt").alias('average'))\
.withColumn('rnk',rank().over(Window.orderBy(col('average').desc()))).filter(col('rnk') == 1)\
.select('title').orderBy('title').first()[0]

print(result,moviename)
                                                                                            

24/09/06 21:03:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 21:03:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Daniel Frozen 2


In [398]:
movies_df.createOrReplaceTempView('moviesdf')
users_df.createOrReplaceTempView('usersdf')
movierating_df.createOrReplaceTempView('movieratingdf')


spark.sql('''
  with name as (select name,cnt ,rank() over(order by cnt desc) as rnk from 
                    (select name,count(name) as cnt from usersdf join movieratingdf on usersdf.user_id = movieratingdf.user_id
                         group by name) ),
  user_name as (select name as result from name where rnk = 1 order by name limit 1),
      movie as (select moviesdf.movie_id,title,x.rating from moviesdf join 
                     (select movieratingdf.movie_id,rating,date_format(created_at,'yyyy-MM') as created_at from movieratingdf) x
                           on moviesdf.movie_id = x.movie_id where x.created_at = "2020-02"),
  movie_avg as (select title,sum(rating) over(partition by movie_id)/count(movie_id) over(partition by movie_id) as average from movie),
  movie_rng as (select title as result, rank() over(order by average desc) as rnk from movie_avg),
  res_movie as (select result from movie_rng where rnk = 1 order by result limit 1)
               (select * from user_name)union(select * from res_movie)

        ''').show()

24/09/06 22:21:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 22:21:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+
|  result|
+--------+
|Frozen 2|
|  Daniel|
+--------+



#### 39 102 Activity Participants M

In [58]:
friends_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/39m_friends.csv'))

In [59]:
activities_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/39m_activities.csv'))

In [402]:
friends_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- activity: string (nullable = true)



In [403]:
activities_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [412]:
window_spec = Window.partitionBy('activity')
window_rnk = Window.orderBy(col('cnt').desc())

friends_df.withColumn('cnt',count('activity').over(window_spec))\
.withColumn('rnk',dense_rank().over(window_rnk)).filter(col('rnk') == 2).select('activity').distinct().show()

24/09/06 22:42:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+
|activity|
+--------+
| Singing|
+--------+



In [419]:
friends_df.createOrReplaceTempView('friendsdf')
activities_df.createOrReplaceTempView('activitiesdf')

spark.sql('''
  with main as (select activity, dense_rank() over(order by cnt desc) as rnk from 
         (select activity, count(activity) over(partition by activity) as cnt from friendsdf))
         select distinct activity from main where rnk = 2
         ''').show()

24/09/06 22:48:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+
|activity|
+--------+
| Singing|
+--------+



#### 40 103  Number of Trusted Contacts of a Customer M

In [64]:
customers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/40m_customers.csv'))

In [65]:
contacts_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/40m_contacts.csv'))

In [66]:
invoices_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/40m_invoices.csv'))

In [424]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- email: string (nullable = true)



In [425]:
contacts_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- contact_name: string (nullable = true)
 |-- contact_email: string (nullable = true)



In [426]:
invoices_df.printSchema()

root
 |-- invoice_id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- user_id: integer (nullable = true)



In [472]:
invoice_info = customers_df.join(invoices_df,customers_df.customer_id == invoices_df.user_id,'inner')\
.select('invoice_id','customer_name','price','customer_id')

contact_cnt = customers_df.join(contacts_df,customers_df.customer_id == contacts_df.user_id,'left')\
.select('customer_id','customer_name','user_id','contact_name').withColumn('contacts_cnt',count('user_id').over(Window.partitionBy('user_id')))\
.select('customer_id','customer_name','contacts_cnt').distinct()

anti_info = contacts_df.join(customers_df,customers_df.customer_name == contacts_df.contact_name,'leftanti')\
.withColumn('anticnt',count('user_id').over(Window.partitionBy('user_id'))).select('user_id','anticnt').distinct()

invoice_info.join(contact_cnt,invoice_info.customer_id == contact_cnt.customer_id,'inner')\
.select('invoice_id',invoice_info.customer_name,'price','contacts_cnt',invoice_info.customer_id)\
.join(anti_info,invoice_info.customer_id == anti_info.user_id,'left')\
.withColumn('trusted_contacts_cnt',expr("contacts_cnt - coalesce(anticnt,0) "))\
.select('invoice_id','customer_name','price','contacts_cnt','trusted_contacts_cnt').orderBy('invoice_id').show()


+----------+-------------+-----+------------+--------------------+
|invoice_id|customer_name|price|contacts_cnt|trusted_contacts_cnt|
+----------+-------------+-----+------------+--------------------+
|        44|         Alex|   60|           1|                   1|
|        55|         John|  500|           0|                   0|
|        66|          Bob|  400|           2|                   0|
|        77|        Alice|  100|           3|                   2|
|        88|        Alice|  200|           3|                   2|
|        99|          Bob|  300|           2|                   0|
+----------+-------------+-----+------------+--------------------+



In [506]:
customers_df.createOrReplaceTempView('customersdf')
contacts_df.createOrReplaceTempView('contactsdf')
invoices_df.createOrReplaceTempView('invoicesdf')

spark.sql('''
  with invoice as (select invoice_id,customer_name,price,customer_id from customersdf join invoicesdf
                        on customersdf.customer_id = invoicesdf.user_id),
  contacts_cnt as (select distinct customer_id,count(user_id) over(partition by user_id) as contacts_cnt from customersdf 
                        left join contactsdf on customersdf.customer_id = contactsdf.user_id),
      anti_cnt as (select distinct user_id,count(user_id) over(partition by user_id) as cnt_ant from contactsdf  left anti join customersdf 
                        on contactsdf.contact_name = customersdf.customer_name) 
                  select invoice_id,customer_name,price,contacts_cnt, (contacts_cnt - coalesce(cnt_ant,0)) as trusted_contacts_cnt from 
                       invoice  join contacts_cnt on invoice.customer_id = contacts_cnt.customer_id
                                left join anti_cnt on anti_cnt.cnt_ant = invoice.customer_id
                       order by invoice_id                             
        ''').show()

+----------+-------------+-----+------------+--------------------+
|invoice_id|customer_name|price|contacts_cnt|trusted_contacts_cnt|
+----------+-------------+-----+------------+--------------------+
|        44|         Alex|   60|           1|                   1|
|        55|         John|  500|           0|                   0|
|        66|          Bob|  400|           2|                   0|
|        77|        Alice|  100|           3|                   2|
|        88|        Alice|  200|           3|                   2|
|        99|          Bob|  300|           2|                   0|
+----------+-------------+-----+------------+--------------------+



In [488]:
contacts_df.join(customers_df,customers_df.customer_name == contacts_df.contact_name,'leftanti').show()

+-------+------------+-----------------+
|user_id|contact_name|    contact_email|
+-------+------------+-----------------+
|      1|         Jal| jal@leetcode.com|
|      2|        Omar|omar@leetcode.com|
|      2|        Meir|meir@leetcode.com|
+-------+------------+-----------------+



#### 41 107 Capital Gain/Loss M

In [67]:
stocks_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/41m_stocks.csv'))

In [4]:
stocks_df.printSchema()

root
 |-- stock_name: string (nullable = true)
 |-- operation: string (nullable = true)
 |-- operation_day: integer (nullable = true)
 |-- price: integer (nullable = true)



In [36]:
window_spec = Window.partitionBy('stock_name','operation')
window_rnk =  Window.partitionBy('stock_name','operation').orderBy(col('operation_day').desc())

buy = stocks_df.withColumn('sumprice',sum('price').over(window_spec))\
.withColumn('rnk',rank().over(window_rnk)).filter( (col('rnk') == 1) & (col('operation') == lit('Buy'))  ).select('stock_name','operation',col('sumprice').alias('buy_price'),'rnk')

sell = stocks_df.withColumn('sumprice',sum('price').over(window_spec))\
.withColumn('rnk',rank().over(window_rnk)).filter( (col('rnk') == 1) & (col('operation') == lit('Sell'))  ).select('stock_name','operation',col('sumprice').alias('sell_price'),'rnk')

sell.join(buy,['stock_name','rnk'],'inner').select('stock_name',expr("sell_price - buy_price").alias('capital_gain_loss')).show()

+------------+-----------------+
|  stock_name|capital_gain_loss|
+------------+-----------------+
|    Handbags|           -23000|
|Corona Masks|             9500|
|    Leetcode|             8000|
+------------+-----------------+



In [51]:
stocks_df.createOrReplaceTempView('stocksdf')

spark.sql('''
 with main as (select stock_name,operation,sumprice, rank() over(partition by stock_name,operation order by sumprice desc) as rnk from 
                   (select stock_name,operation,sum(price) as sumprice from stocksdf group by stock_name,operation)),
      sell as (select stock_name,operation,sumprice as sell_price,rnk from main where operation = "Sell" ),
      buy  as (select stock_name,operation,sumprice as buy_price,rnk from main where operation = "Buy") 
              select sell.stock_name, (sell_price - buy_price) as capital_gain_loss from sell join buy on 
                   sell.stock_name = buy.stock_name and sell.rnk = buy.rnk
         ''').show()

+------------+-----------------+
|  stock_name|capital_gain_loss|
+------------+-----------------+
|    Handbags|           -23000|
|Corona Masks|             9500|
|    Leetcode|             8000|
+------------+-----------------+



#### 42 108 Customers Who Bought Products A and B but Not C M

In [68]:
customers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/42m_customers.csv'))

In [69]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/42m_orders.csv'))

In [56]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)



In [57]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_name: string (nullable = true)



In [93]:
window_spec = Window.partitionBy('customer_id')

orders_df.filter(col('product_name').isin(['A','B','C'])).select('customer_id','product_name')\
.withColumn('cnt',count('customer_id').over(window_spec)).filter(col('cnt') == 2).select('customer_id').distinct()\
.join(customers_df,on = 'customer_id').select('customer_name').show()

+-------------+
|customer_name|
+-------------+
|    Elizabeth|
+-------------+



In [103]:
customers_df.createOrReplaceTempView('customersdf')
orders_df.createOrReplaceTempView('ordersdf')

spark.sql('''
  with main as (select distinct customer_id from 
                     (select customer_id,product_name,count(customer_id) over(partition by customer_id) as cnt
                          from ordersdf where product_name in ('A','B','C'))
                     where cnt = 2)
               select customer_name from customersdf join main on customersdf.customer_id = main.customer_id
        ''').show()

+-------------+
|customer_name|
+-------------+
|    Elizabeth|
+-------------+



#### 43 113 Evaluate Boolean Expression M

In [70]:
variables_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/43m_variables.csv'))

In [71]:
expressions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/43m_expressions.csv'))

In [106]:
variables_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- value: integer (nullable = true)



In [107]:
expressions_df.printSchema()

root
 |-- left_operand: string (nullable = true)
 |-- operator: string (nullable = true)
 |-- right_operand: string (nullable = true)



In [110]:
x = 66
y = 77
expressions_df.withColumn('xx',when(col('left_operand') == lit('x'),66) \
                              .when(col('left_operand') == lit('y'),77)) \
              .withColumn('yy',when(col('right_operand') == lit('x'),66) \
                              .when(col('right_operand') == lit('y'),77)) \
              .withColumn('value',when(col('operator') == '>', col('xx') > col('yy')) \
                                 .when(col('operator') == '<', col('xx') < col('yy')) \
                                 .when(col('operator') == '=', col('xx') == col('yy'))) \
                                 .select('left_operand','operator','right_operand','value').show()

+------------+--------+-------------+-----+
|left_operand|operator|right_operand|value|
+------------+--------+-------------+-----+
|           x|       >|            y|false|
|           x|       <|            y| true|
|           x|       =|            y|false|
|           y|       >|            x| true|
|           y|       <|            x|false|
|           x|       =|            x| true|
+------------+--------+-------------+-----+



In [111]:
expressions_df.createOrReplaceTempView('expressionsdf')

spark.sql(''' 
   with base as (select left_operand,operator,right_operand,case when left_operand == "x" then 66 else 77 end as xx,  
                        case when right_operand == "y" then 77 else 66 end as yy from expressionsdf) 
            select left_operand,operator,right_operand, case when operator = ">" then "xx" > "yy"
                                                            when operator = "<" then "xx" < "yy"
                                                            when operator = "=" then "xx" = "yy" end as value 
                                                             from base
  ''').show()

+------------+--------+-------------+-----+
|left_operand|operator|right_operand|value|
+------------+--------+-------------+-----+
|           x|       >|            y|false|
|           x|       <|            y| true|
|           x|       =|            y|false|
|           y|       >|            x|false|
|           y|       <|            x| true|
|           x|       =|            x|false|
+------------+--------+-------------+-----+



#### 44 114 Apples & Oranges M

In [72]:
sales_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/44m_sales.csv'))

In [113]:
sales_df.printSchema()

root
 |-- sale_date: string (nullable = true)
 |-- fruit: string (nullable = true)
 |-- sold_num: integer (nullable = true)



In [120]:
apple = sales_df.filter(col('fruit') == lit('apples')).select('sale_date',col('sold_num').alias('apple_sold'))
orange = sales_df.filter(col('fruit') == lit('oranges')).select('sale_date',col('sold_num').alias('orange_sold'))

apple.join(orange, on = 'sale_date').select('sale_date',expr("apple_sold - orange_sold").alias('diff')).show()

+----------+----+
| sale_date|diff|
+----------+----+
|2020-05-01|   2|
|2020-05-02|   0|
|2020-05-03|  20|
|2020-05-04|  -1|
+----------+----+



In [125]:
sales_df.createOrReplaceTempView('salesdf')

spark.sql('''
           select apple.sale_date as sale_date, (apple_sold - orange_sold) as diff from 
                     (select sale_date,sold_num as apple_sold from salesdf where fruit = "apples") as apple join
                     (select sale_date,sold_num as orange_sold from salesdf where fruit = "oranges") as orange
                on apple.sale_date =orange.sale_date

         ''').show()

+----------+----+
| sale_date|diff|
+----------+----+
|2020-05-01|   2|
|2020-05-02|   0|
|2020-05-03|  20|
|2020-05-04|  -1|
+----------+----+



#### 45 115 Active Users M

In [73]:
logins_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/45m_logins.csv'))

In [74]:
accounts_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/45m_accounts.csv'))

In [5]:
logins_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- login_date: string (nullable = true)



In [6]:
accounts_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [7]:
window_spec = Window.partitionBy('id').orderBy(col('login_date').asc())

logins_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') ==lit('6'))\
.join(accounts_df,on = 'id').select('id','name').show()

+---+--------+
| id|    name|
+---+--------+
|  7|Jonathan|
+---+--------+



In [8]:
accounts_df.createOrReplaceTempView('accountsdf')
logins_df.createOrReplaceTempView('loginsdf')

spark.sql('''
          select x.id,name from accountsdf join
          (select id,login_date,rank() over(partition by id order by login_date asc) as rnk from loginsdf) as x
          on accountsdf.id = x.id where rnk = 6
        ''').show()

+---+--------+
| id|    name|
+---+--------+
|  7|Jonathan|
+---+--------+



#### 46 116 Rectangles Area M

In [75]:
points_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/46m_points.csv'))

In [11]:
points_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- x_value: integer (nullable = true)
 |-- y_value: integer (nullable = true)



In [44]:
x = points_df.select('x_value')
y = points_df.select('y_value')

window_spec = Window.orderBy(col('id').asc())

points_df.withColumn('1xst',lead('x_value',1,0).over(window_spec))\
.withColumn('1yxt',lead('y_value',1,0).over(window_spec))\
.withColumn('1idxt',lead('id',1,0).over(window_spec))\
.withColumn('2xst',lead('x_value',2,0).over(window_spec))\
.withColumn('2yxt',lead('y_value',2,0).over(window_spec))\
.withColumn('2idxt',lead('id',2,0).over(window_spec))\
.select('id','1idxt','2idxt','x_value','1xst','2xst','y_value','1yxt','2yxt',
        expr("abs((x_value - 1xst) * (y_value - 1yxt ))").alias('area1'),
        expr("abs((x_value - 2xst) * (y_value - 2yxt ))").alias('area2'),
       ).filter(col('1idxt') != 0).select(col('id').alias('p1'),col('1idxt').alias('p2'),col('area1').alias('area')  ).show()      

+---+---+----+
| p1| p2|area|
+---+---+----+
|  1|  2|   2|
|  2|  3|   4|
+---+---+----+



24/09/08 00:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [68]:
points_df.createOrReplaceTempView('pointsdf')

spark.sql('''
 with main as (   select id,lead(id, 1, 0) over(order by id) as id1,
                           lead(id, 2, 0) over(order by id) as id2,
                           x_value, 
                           lead(x_value, 1, 0) over(order by id) as x1,
                           lead(x_value, 2, 0) over(order by id) as x2, 
                           y_value, 
                           lead(y_value, 1, 0) over(order by id) as y1, 
                           lead(y_value, 2, 0) over(order by id) as y2
                        from pointsdf),
       one as (select id,id1,id2, abs((x_value - x1) * (y_value - y1)) as area,
                    abs((x_value - x2) * (y_value - y2)) as area2 from main)
              select id as p1,id1 as p2,area from one where id != 3
         ''').show()

+---+---+----+
| p1| p2|area|
+---+---+----+
|  1|  2|   2|
|  2|  3|   4|
+---+---+----+



24/09/08 00:45:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 47 117 Calculate Salaries M

In [76]:
salaries_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/47m_salaries.csv'))

In [4]:
salaries_df.printSchema()

root
 |-- company_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- salary: integer (nullable = true)



In [18]:
window_spec = Window.partitionBy('company_id').orderBy(col('salary').desc())

salaries_df.withColumn('rnk',max('salary').over(window_spec))\
.withColumn('per',when((col('rnk') > lit('10000')),col('salary') - (49/100 * col('salary')) )
                 .when((col('rnk') < lit('1000')),col('salary'))
                 .when(((col('rnk') <= lit('10000')) & (col('rnk') > lit('1000'))),col('salary') - (24/100 * col('salary'))))\
.select('company_id','employee_id','employee_name',col('per').cast('int').alias('salary')).orderBy('company_id','employee_id').show()
                          

+----------+-----------+-------------+------+
|company_id|employee_id|employee_name|salary|
+----------+-----------+-------------+------+
|         1|          1|         Tony|  1020|
|         1|          2|       Pronub| 10863|
|         1|          3|       Tyrrox|  5508|
|         2|          1|          Pam|   300|
|         2|          7|       Bassem|   450|
|         2|          9|     Hermione|   700|
|         3|          2|       Ognjen|  1672|
|         3|          7|      Bocaben|    76|
|         3|         13|      Nyancat|  2508|
|         3|         15|  Morninngcat|  5910|
+----------+-----------+-------------+------+



In [28]:
salaries_df.createOrReplaceTempView('salariesdf')

spark.sql('''
 with main as (select company_id,employee_id,employee_name,salary,max(salary) over(partition by company_id order by salary desc) as max_sal from salariesdf)
           select company_id,employee_id,employee_name,case when max_sal > 10000 then salary - (49/100 * salary) 
                                                            when max_sal < 1000 then salary
                                                            when max_sal <= 10000 and max_sal > 1000 then salary - (24/100 * salary) 
                                                            end as percentage from main
                 order by company_id,employee_id
         ''').show()

+----------+-----------+-------------+----------+
|company_id|employee_id|employee_name|percentage|
+----------+-----------+-------------+----------+
|         1|          1|         Tony|    1020.0|
|         1|          2|       Pronub|   10863.0|
|         1|          3|       Tyrrox|    5508.0|
|         2|          1|          Pam|     300.0|
|         2|          7|       Bassem|     450.0|
|         2|          9|     Hermione|     700.0|
|         3|          2|       Ognjen|    1672.0|
|         3|          7|      Bocaben|      76.0|
|         3|         13|      Nyancat|    2508.0|
|         3|         15|  Morninngcat|   5910.52|
+----------+-----------+-------------+----------+



#### 48 121 Countries You Can Safely Invest In M

In [77]:
person_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/48m_person.csv'))

In [81]:
country_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/48m_country.csv'))

In [82]:
calls_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/48m_calls.csv'))

In [80]:
person_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- phone_number: string (nullable = true)



In [7]:
country_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- country_code: integer (nullable = true)



In [8]:
calls_df.printSchema()

root
 |-- caller_id: integer (nullable = true)
 |-- callee_id: integer (nullable = true)
 |-- duration: integer (nullable = true)



In [20]:
total_df =calls_df.withColumn('callarrays',array_sort(array(col('caller_id'),col('callee_id')))) \
  .agg((sum(col('duration'))/count(col('duration'))).alias('total_avg'))

final_df= calls_df.withColumn('callarrays',array_sort(array(col('caller_id'),col('callee_id')))).groupBy(col('callarrays'))\
  .agg(count(col('duration')).alias('cnt'),sum(col('duration')).alias('sumduration'))

israel_df = final_df.withColumn("exploded_callarrays", explode(final_df["callarrays"])).filter(col('exploded_callarrays').isin([7,9]))\
.select( (sum(col('sumduration'))/sum(col('cnt'))).alias('israel')).select(lit('israel').alias('country'),col('israel').alias('avg'))

Morocco_df = final_df.withColumn("exploded_callarrays", explode(final_df["callarrays"])).filter(col('exploded_callarrays').isin([1,2]))\
.select( (sum(col('sumduration'))/sum(col('cnt'))).alias('Morocco')).select(lit('Morocco').alias('country'),col('Morocco').alias('avg'))

peru_df = final_df.withColumn("exploded_callarrays", explode(final_df["callarrays"])).filter(col('exploded_callarrays').isin([3,12]))\
.select( (sum(col('sumduration'))/sum(col('cnt'))).alias('Peru')).select(lit('Peru').alias('country'),col('Peru').alias('avg'))

israel_df.union(Morocco_df).union(peru_df).orderBy(desc('avg')).first()[0]

'Peru'

In [21]:
person_df.createOrReplaceTempView('persondf')
country_df.createOrReplaceTempView('countrydf')
calls_df.createOrReplaceTempView('callsdf')

spark.sql('''
  with code_list as (select collect_list(id) as coldelst,countrydf.name from person_df join countrydf on 
                       substring(person_df.phone_number,1,3) == countrydf.country_code group by countrydf.name),
      code_array as (select array_sort(array(caller_id,callee_id)) as code, count(duration) as count_call, 
                       sum(duration) as total_duration from callsdf group by code),
    explode_code as (select code,count_call,total_duration,explode(code) as ex_code from code_array),
           logic as (select country,sum(avg)/count(country) as avg from (select case when ex_code in (3,12) then "Peru"
                                 when ex_code in (1,2)  then "Morocco"
                                 when ex_code in (7,9)  then "Israel" end as country,
                            case when ex_code in (3,12) then round(sum(total_duration)/sum(count_call),2) 
                                 when ex_code in (1,2) then round(sum(total_duration)/sum(count_call),2) 
                                 when ex_code in (7,9) then round(sum(total_duration)/sum(count_call),2) end as avg 
                            from explode_code  group by ex_code) group by country),
      global_avg as (select sum(duration)/count(duration) as global_avg from callsdf )
                    select * from logic where avg > (select global_avg from  global_avg)
         ''').show()

+-------+------+
|country|   avg|
+-------+------+
|   Peru|145.67|
+-------+------+



#### 49 125 The Most Recent Three Orders M

In [83]:
customers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/49m_customers.csv'))

In [84]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/49m_orders.csv'))

In [33]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [34]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- cost: integer (nullable = true)



In [40]:
window_spec = Window.partitionBy('customer_id').orderBy(col('order_date').desc())

orders_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') < 4).join(customers_df,on = 'customer_id')\
.select(col('name').alias('customer_name'),'customer_id','order_id','order_date')\
.orderBy(col('customer_name').asc(),col('customer_id').asc()).show()

+-------------+-----------+--------+----------+
|customer_name|customer_id|order_id|order_date|
+-------------+-----------+--------+----------+
|    Annabelle|          3|       7|2020-08-01|
|    Annabelle|          3|       3|2020-07-31|
|     Jonathan|          2|       6|2020-08-01|
|     Jonathan|          2|       2|2020-07-30|
|     Jonathan|          2|       9|2020-08-07|
|       Marwan|          4|       4|2020-07-29|
|      Winston|          1|       8|2020-08-03|
|      Winston|          1|       1|2020-07-31|
|      Winston|          1|      10|2020-07-15|
+-------------+-----------+--------+----------+



In [47]:
orders_df.createOrReplaceTempView('ordersdf')
customers_df.createOrReplaceTempView('customersdf')

spark.sql('''
         select name as customer_name,x.customer_id,x.order_id,x.order_date from 
         (select order_id,order_date,customer_id, rank() over(partition by customer_id order by order_date desc) as rnk from ordersdf) x
              join customersdf on x.customer_id  = customersdf.customer_id
              where x.rnk < 4
              order by customer_name asc,customer_id asc
        ''').show()

+-------------+-----------+--------+----------+
|customer_name|customer_id|order_id|order_date|
+-------------+-----------+--------+----------+
|    Annabelle|          3|       7|2020-08-01|
|    Annabelle|          3|       3|2020-07-31|
|     Jonathan|          2|       6|2020-08-01|
|     Jonathan|          2|       2|2020-07-30|
|     Jonathan|          2|       9|2020-08-07|
|       Marwan|          4|       4|2020-07-29|
|      Winston|          1|       8|2020-08-03|
|      Winston|          1|       1|2020-07-31|
|      Winston|          1|      10|2020-07-15|
+-------------+-----------+--------+----------+



#### 50 127 he Most Recent Orders for Each Product  M

In [85]:
customers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/50m_customers.csv'))

In [86]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/50m_orders.csv'))

In [87]:
products_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/50m_products.csv'))

In [51]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [52]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)



In [53]:
products_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: integer (nullable = true)



In [57]:
window_spec = Window.partitionBy('product_id').orderBy(col('order_date').desc())

orders_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1).join(products_df,on = 'product_id')\
.select('product_name','product_id','order_id','order_date').orderBy(col('product_id').asc(),col('product_id').asc()).show()

+------------+----------+--------+----------+
|product_name|product_id|order_id|order_date|
+------------+----------+--------+----------+
|    keyboard|         1|       6|2020-08-01|
|    keyboard|         1|       7|2020-08-01|
|       mouse|         2|       8|2020-08-03|
|      screen|         3|       3|2020-08-29|
+------------+----------+--------+----------+



In [61]:
customers_df.createOrReplaceTempView('customersdf')
orders_df.createOrReplaceTempView('ordersdf')
products_df.createOrReplaceTempView('productsdf')

spark.sql('''
           select product_name,x.product_id,x.order_id,x.order_date from 
                      (select product_id,order_id,order_date, rank() over(partition by product_id order by order_date desc) as rnk from ordersdf) x
                 join productsdf on productsdf.product_id = x.product_id 
                 where x.rnk = 1
                 order by product_name,x.product_id
        ''').show()

+------------+----------+--------+----------+
|product_name|product_id|order_id|order_date|
+------------+----------+--------+----------+
|    keyboard|         1|       6|2020-08-01|
|    keyboard|         1|       7|2020-08-01|
|       mouse|         2|       8|2020-08-03|
|      screen|         3|       3|2020-08-29|
+------------+----------+--------+----------+



#### 51 128 Bank Account Summary M


In [88]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/51m_users.csv'))

In [89]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/51m_transactions.csv'))

In [7]:
users_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- user_name: string (nullable = true)
 |-- credit: integer (nullable = true)



In [8]:
transactions_df.printSchema()

root
 |-- trans_id: integer (nullable = true)
 |-- paid_by: integer (nullable = true)
 |-- paid_to: integer (nullable = true)
 |-- amount: integer (nullable = true)
 |-- transacted_on: string (nullable = true)



In [37]:
transactions_df.show()

+--------+-------+-------+------+-------------+
|trans_id|paid_by|paid_to|amount|transacted_on|
+--------+-------+-------+------+-------------+
|       1|      1|      3|   400|   2020-08-01|
|       2|      3|      2|   500|   2020-08-02|
|       3|      2|      1|   200|   2020-08-03|
+--------+-------+-------+------+-------------+



In [62]:
paid_by = transactions_df.groupBy('paid_by').agg((- sum('amount')).alias('chage_amt')).withColumnRenamed('paid_by','user_id')
paid_to = transactions_df.groupBy('paid_to').agg((sum('amount')).alias('chage_amt')).withColumnRenamed('paid_by','user_id')

paid_by.union(paid_to).groupby('user_id').agg(sum('chage_amt').alias('change_rate'))\
.join(users_df,'user_id','right').withColumn('credit',(coalesce(col('change_rate'),lit('0')) + col('credit')))\
.withColumn('credit_limit_breached',when(col('credit') < 0,lit('Yes')).otherwise(lit('No')))\
.select('user_id','user_name',col('credit').cast('int').alias('credit'),'credit_limit_breached').show()

+-------+---------+------+---------------------+
|user_id|user_name|credit|credit_limit_breached|
+-------+---------+------+---------------------+
|      1| Moustafa|  -100|                  Yes|
|      2| Jonathan|   500|                   No|
|      3|  Winston|  9900|                   No|
|      4|     Luis|   800|                   No|
+-------+---------+------+---------------------+



In [90]:
users_df.createOrReplaceTempView('usersdf')
transactions_df.createOrReplaceTempView('transactionsdf')

spark.sql('''
 with main as (select user_id,sum(change_amt) as change_amt from (
                   (select paid_by as user_id,sum(amount) * -1 as change_amt from transactionsdf group by paid_by)
                   union
                   (select paid_to as user_id,sum(amount) as change_amt from transactionsdf group by paid_to))
                   group by user_id),
       one as (select usersdf.user_id,user_name,credit,(coalesce(change_amt,0)) as change_amt from main
                   right join usersdf on main.user_id == usersdf.user_id),
       two as (select  user_id,user_name, (credit + change_amt) as credit from one)
               select user_id,user_name,credit,case when credit < 0 then "Yes" else "No" end credit_limit_breached from two
         ''').show()


+-------+---------+------+---------------------+
|user_id|user_name|credit|credit_limit_breached|
+-------+---------+------+---------------------+
|      1| Moustafa|  -100|                  Yes|
|      2| Jonathan|   500|                   No|
|      3|  Winston|  9900|                   No|
|      4|     Luis|   800|                   No|
+-------+---------+------+---------------------+



#### 52 133 The Most Frequently Ordered Products for Each Customer M

In [90]:
customers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/52m_customers.csv'))

In [93]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/52m_orders.csv'))

In [94]:
products_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/52m_products.csv'))

In [104]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [105]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)



In [106]:
products_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: integer (nullable = true)



In [116]:
window_spec = Window.partitionBy('customer_id').orderBy(col('cnt').desc())
orders_df.groupby('customer_id','product_id').agg(count('product_id').alias('cnt'))\
.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1).join(products_df,'product_id','inner')\
.select('customer_id','product_id','product_name').orderBy('customer_id','product_id').show()

+-----------+----------+------------+
|customer_id|product_id|product_name|
+-----------+----------+------------+
|          1|         2|       mouse|
|          2|         1|    keyboard|
|          2|         2|       mouse|
|          2|         3|      screen|
|          3|         3|      screen|
|          4|         1|    keyboard|
+-----------+----------+------------+



In [122]:
customers_df.createOrReplaceTempView('customersdf')
orders_df.createOrReplaceTempView('ordersdf')
products_df.createOrReplaceTempView('productsdf')


range_df = spark.range()

spark.sql('''
 with main as (select customer_id,product_id,count(product_id) as cnt from ordersdf group by customer_id,product_id),
       rnk as (select customer_id,product_id, rank() over(partition by customer_id order by cnt desc) as rnk from main)
               select customer_id,rnk.product_id,product_name from rnk join productsdf on rnk.product_id = productsdf.product_id
                    where rnk = 1 order by customer_id,product_id
        ''').show()

+-----------+----------+------------+
|customer_id|product_id|product_name|
+-----------+----------+------------+
|          1|         2|       mouse|
|          2|         1|    keyboard|
|          2|         2|       mouse|
|          2|         3|      screen|
|          3|         3|      screen|
|          4|         1|    keyboard|
+-----------+----------+------------+



#### 53 135 Find the Missing IDs M

In [95]:
customers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/53m_customers.csv'))

In [125]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)



In [178]:
min_num = customers_df.select(min(col('customer_id')).alias('min_num')).collect()[0][0]
max_num = customers_df.select(max(col('customer_id')).alias('max_num')).collect()[0][0]

df = spark.createDataFrame([(spark.range(min_num,max_num+1).collect()[i][0],)  for i in range(0,len(spark.range(min_num,max_num+1).collect()))],['rng'])

cus_id  = [customers_df.select('customer_id').collect()[i][0] for i in range(0,len(customers_df.select('customer_id').collect()))]
                                                                             
df.filter(~col('rng').isin(cus_id)  ).show()




+---+
|rng|
+---+
|  2|
|  3|
+---+



In [196]:
customers_df.createOrReplaceTempView('customersdf')
range_df.createOrReplaceTempView('rangedf')
range_df = spark.range(min_num,max_num + 1)

spark.sql('''
          select id from rangedf where id not in (select customer_id  from customersdf)
           ''').show()



+---+
| id|
+---+
|  2|
|  3|
+---+



#### 54 143 Number of Calls Between Two Persons M

In [96]:
calls_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/54m_calls.csv'))

In [198]:
calls_df.printSchema()

root
 |-- from_id: integer (nullable = true)
 |-- to_id: integer (nullable = true)
 |-- duration: integer (nullable = true)



In [222]:
window_spec = Window.partitionBy('arr_id')
window_rnk = Window.partitionBy('arr_id').orderBy(col('from_id').asc())

calls_df.withColumn('arr_id',array_sort(array('from_id','to_id')))\
.withColumn('gp_id',sum('duration').over(window_spec)).withColumn('cnt_id',count('duration').over(window_spec))\
.withColumn('rnk',rank().over(window_rnk)).filter(col('rnk') == 1)\
.select(col('from_id').alias('person1'),col('to_id').alias('person2'),col('cnt_id').alias('call_count'),col('gp_id').alias('total_duration'))\
.distinct().orderBy('person1').show()


+-------+-------+----------+--------------+
|person1|person2|call_count|total_duration|
+-------+-------+----------+--------------+
|      1|      2|         2|            70|
|      1|      3|         1|            20|
|      3|      4|         4|           999|
+-------+-------+----------+--------------+



In [239]:
calls_df.createOrReplaceTempView('callsdf')

spark.sql('''
 with main as (select arr_id,from_id,to_id,duration, count(duration) over(partition by arr_id) as call_count,sum(duration) over(partition by arr_id) as total_duration from  
                    (select from_id,to_id,duration,array_sort(array(from_id,to_id)) as arr_id from callsdf)),
      one  as (select  from_id as person1,to_id as person2,duration,call_count,total_duration,arr_id,rank() over(partition by arr_id order by from_id) as rnk  from main)
               select distinct person1,person2,call_count,total_duration from one where rnk = 1
                     order by person1 
         ''').show()

+-------+-------+----------+--------------+
|person1|person2|call_count|total_duration|
+-------+-------+----------+--------------+
|      1|      3|         1|            20|
|      1|      2|         2|            70|
|      3|      4|         4|           999|
+-------+-------+----------+--------------+



#### 55 144 Biggest Window Between Visits M

In [97]:
usevisits_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/55m_useviisits.csv'))

#today's date is '2021-1-1'

In [244]:
usevisits_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- visit_date: string (nullable = true)



In [254]:
window_spec = Window.partitionBy('user_id').orderBy(col('visit_date').asc())
window_rnk = Window.partitionBy('user_id').orderBy(col('biggest_window').desc())

usevisits_df.withColumn('lead_date',lead('visit_date',1,0).over(window_spec))\
.withColumn('start_date',when(col('lead_date') == 0,lit("2021-1-1")).otherwise(col('lead_date')))\
.withColumn('biggest_window',datediff(col('start_date'),col('visit_date')))\
.withColumn('rnk',rank().over(window_rnk)).filter(col('rnk') == 1).select('user_id','biggest_window').orderBy('user_id').show()

+-------+--------------+
|user_id|biggest_window|
+-------+--------------+
|      1|            39|
|      2|            65|
|      3|            51|
+-------+--------------+



In [265]:
usevisits_df.createOrReplaceTempView('usevisitsdf')

spark.sql('''
 with main as (select user_id,visit_date,lead(visit_date,1,0) over(partition by user_id order by visit_date asc) start_date from usevisitsdf),
       one as (select user_id,visit_date,start_date,case when start_date = 0 then "2021-1-1" else start_date end as date from main),
       two as (select user_id, datediff(date,visit_date) as biggest_window,rank() over(partition by user_id order by datediff(date,visit_date) desc) as rnk from one)
               select user_id,biggest_window from two where rnk = 1 order by user_id asc
         ''').show()

+-------+--------------+
|user_id|biggest_window|
+-------+--------------+
|      1|            39|
|      2|            65|
|      3|            51|
+-------+--------------+



#### 56 145 Count Apples and Oranges M

In [98]:
boxes_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/56m_boxes.csv'))

In [99]:
chests_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/56m_chests.csv'))

In [268]:
boxes_df.printSchema()

root
 |-- box_id: integer (nullable = true)
 |-- chest_id: integer (nullable = true)
 |-- apple_count: integer (nullable = true)
 |-- orange_count: integer (nullable = true)



In [269]:
chests_df.printSchema()

root
 |-- chest_id: integer (nullable = true)
 |-- apple_count: integer (nullable = true)
 |-- orange_count: integer (nullable = true)



In [335]:
main_df = boxes_df.join(chests_df,boxes_df.chest_id == chests_df.chest_id,'left')\
.select('box_id',boxes_df.chest_id,boxes_df.apple_count.alias('apples'),boxes_df.orange_count.alias('oranges'),
        coalesce(chests_df.apple_count,lit('0')).alias('chest_apple'),
        coalesce(chests_df.orange_count,lit('0')).alias('chest_orange'))
        
one_df = main_df.withColumn('applecnt',(col('apples')+col('chest_apple') ))\
.withColumn('orangecnt',(col('oranges')+ col('chest_orange')))\

one_df.select(expr("sum(applecnt)").cast('int').alias('apple_count'),lit('0').alias('orange_count'))\
.union(one_df.select(lit('0').alias('apple_count'),expr("sum(orangecnt)").cast('int').alias('orange_count')))\
.select(expr("sum(apple_count)").cast('int').alias('apple_count'),expr("sum(orange_count)").cast('int').alias('orange_count')).show()

+-----------+------------+
|apple_count|orange_count|
+-----------+------------+
|        151|         123|
+-----------+------------+



In [374]:
boxes_df.createOrReplaceTempView('boxesdf')
chests_df.createOrReplaceTempView('chestsdf')

spark.sql('''
 with main as (select boxesdf.chest_id,boxesdf.apple_count,coalesce(chestsdf.apple_count,0) as chest_apple,
                     boxesdf.orange_count,coalesce(chestsdf.orange_count,0) as chest_orange 
                     from boxesdf left join chestsdf on boxesdf.chest_id = chestsdf.chest_id),
       one as ((select sum((apple_count+chest_apple)) as apple_count,'0' as orange_count from main) union(select '0' as apple_count ,
                     sum((orange_count+chest_orange)) as orange_count from main))
                     select cast(sum(apple_count) as int) as apple_count, cast(sum(orange_count) as int) as orange_count from one
         ''').show()

+-----------+------------+
|apple_count|orange_count|
+-----------+------------+
|        151|         123|
+-----------+------------+



#### 57 149 Leetflex Banned Accounts M

In [100]:
loginfo_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/57m_loginfo.csv'))

In [376]:
loginfo_df.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- ip_address: integer (nullable = true)
 |-- login: string (nullable = true)
 |-- logout: string (nullable = true)



In [461]:
window_spec = Window.partitionBy('account_id').orderBy(col('ip_address').asc())

odd_df = loginfo_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1)
even_df = loginfo_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 2)

evendf = even_df.withColumnRenamed('account_id','even_account_id').withColumnRenamed('ip_address','even_ip_address')\
.withColumnRenamed('login','even_login').withColumnRenamed('logout','even_logout')

evendf.join(odd_df,evendf.even_account_id == odd_df.account_id,'inner')\
.select(odd_df.account_id,odd_df.ip_address,evendf.even_ip_address
       ,unix_timestamp(odd_df.login).alias('odd_login')
       ,unix_timestamp(odd_df.logout).alias('odd_logout')
       ,unix_timestamp(evendf.even_login).alias('even_login')
       ,unix_timestamp(evendf.even_logout).alias('even_logout')
    ,(unix_timestamp(odd_df.logout) - unix_timestamp(odd_df.login)).alias('odd_diff')
        ,(unix_timestamp(evendf.even_logout) - unix_timestamp(evendf.even_login) ).alias('even_diff')
      ).filter( (col('ip_address') != col('even_ip_address')) & (col('odd_diff') != col('even_diff')))\
.select('account_id').show()

+----------+
|account_id|
+----------+
|         1|
|         4|
+----------+



In [462]:
loginfo_df.alias('d1').join(loginfo_df.alias('d2'),(col('d1.account_id') == col('d2.account_id') )
                            & (unix_timestamp(col('d1.login')) < unix_timestamp(col('d2.logout')) )
                            & (unix_timestamp(col('d1.logout')) > unix_timestamp(col('d2.login')) )
                            & (col('d1.ip_address') != col('d2.ip_address')), 'inner')\
          .select(col('d1.account_id')).show()

+----------+
|account_id|
+----------+
|         1|
|         1|
+----------+



#### 58 152 Grand Slam Titles M

In [101]:
players_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/58m_players.csv'))

In [102]:
championships_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/58m_championships.csv'))

In [5]:
players_df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- player_name: string (nullable = true)



In [6]:
championships_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- wimbledon: integer (nullable = true)
 |-- Fr_open: integer (nullable = true)
 |-- US_open: integer (nullable = true)
 |-- Au_open: integer (nullable = true)



In [497]:
championships_df.withColumn('arr',array(col('wimbledon'),col('Fr_open'),col('US_open'),col('Au_open')))\
.groupBy().agg(explode(sort_array(flatten(collect_list(col('arr'))))).alias('lst')).groupby ('lst').agg(count('lst').alias('grand_slams_count'))\
.join(players_df,col('lst') == players_df.player_id,'inner').select('player_id','player_name','grand_slams_count').show()

+---------+-----------+-----------------+
|player_id|player_name|grand_slams_count|
+---------+-----------+-----------------+
|        1|      Nadal|                7|
|        2|    Federer|                5|
+---------+-----------+-----------------+



In [509]:
players_df.createOrReplaceTempView('playersdf')
championships_df.createOrReplaceTempView('championshipsdf')

spark.sql('''
 with main as (select explode(arr) as arrcol from 
                    (select array(wimbledon,Fr_open,US_open,Au_open) as arr from championshipsdf)),
       one as (select arrcol,count(arrcol) as grand_slams_count from main group by arrcol)
              select player_id,player_name,grand_slams_count from playersdf join one on one.arrcol = playersdf.player_id
        ''').show()

+---------+-----------+-----------------+
|player_id|player_name|grand_slams_count|
+---------+-----------+-----------------+
|        1|      Nadal|                7|
|        2|    Federer|                5|
+---------+-----------+-----------------+



#### 59 156 Find Interview Candidates M

In [103]:
contests_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/59m_contests.csv'))

In [104]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/59m_users.csv'))

In [5]:
contests_df.printSchema()

root
 |-- contest_id: integer (nullable = true)
 |-- gold_medal: integer (nullable = true)
 |-- silver_medal: integer (nullable = true)
 |-- bronze_medal: integer (nullable = true)



In [6]:
users_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- mail: string (nullable = true)
 |-- name: string (nullable = true)



In [30]:
window_spec = Window.orderBy(col('contest_id').asc()).rowsBetween(0,2)
window_one = Window.orderBy(col('contest_id').asc()).rowsBetween(0,1)

first_df = contests_df.withColumn('arr',array(col('gold_medal'),col('silver_medal'),col(('bronze_medal'))))

twodf = first_df.withColumn('arrcollect',collect_list('arr').over(window_spec)).select('contest_id','arrcollect')

threedf = twodf.withColumn('ln',size('arrcollect')).filter(col('ln') >= 3).select('contest_id','arrcollect')\
.groupby('contest_id','arrcollect').agg(explode(flatten('arrcollect')).alias('flt'))\
.groupby('contest_id','flt').agg(count('flt').alias('cnt')).filter(col('cnt') >= 3).select(col('flt').alias('userid'))

threemedaldf = threedf.join(users_df,threedf.userid == users_df.user_id,'inner').select('user_id','mail').distinct()

window_spec_gold = Window.orderBy(col('user_id').desc())

golddf = contests_df.groupBy('gold_medal').agg(count('gold_medal').alias('user_id')).withColumn('rnk',rank().over(window_spec_gold))\
.filter(col('rnk') == 1).select(col('gold_medal').alias('user_id')).join(users_df,'user_id','inner')\
.select('user_id','mail')

threemedaldf.union(golddf).orderBy('user_id').show()

24/09/12 10:39:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/12 10:39:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+------------------+
|user_id|              mail|
+-------+------------------+
|      1|sarah@leetcode.com|
|      2|  bob@leetcode.com|
|      3|alice@leetcode.com|
|      5|quarz@leetcode.com|
+-------+------------------+



In [83]:
contests_df.createOrReplaceTempView('contestsdf')
users_df.createOrReplaceTempView('usersdf')

spark.sql('''
  with arr as (select contest_id,collect_list(array(gold_medal,silver_medal,bronze_medal)) over(order by contest_id asc rows between 
                0 PRECEDING and 2 FOLLOWING) as arrcollect from contestsdf),
       one as (select contest_id,arrcollect from arr where size(arrcollect) >=3),
       two as (select contest_id,explode(flatten(arrcollect)) as ids from one),
     three as (select ids,count(ids) as id from two group by contest_id,ids),
      four as (select distinct ids as user_id,mail from three join usersdf on usersdf.user_id = three.ids where id >= 3),
   goldcnt as (select gold_medal,count(gold_medal) as cnt from contestsdf group by gold_medal),
   goldrnk as (select gold_medal,cnt,rank() over(order by cnt desc) as rnk from goldcnt),
      gold as (select gold_medal,mail as user_id from goldrnk join usersdf on usersdf.user_id = goldrnk.gold_medal where rnk = 1)
               (select * from four) union(select * from gold) order by user_id
        ''').show()

24/09/12 11:14:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/12 11:14:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+------------------+
|user_id|              mail|
+-------+------------------+
|      1|sarah@leetcode.com|
|      2|  bob@leetcode.com|
|      3|alice@leetcode.com|
|      5|quarz@leetcode.com|
+-------+------------------+



#### 60 158 Maximum Transaction Each Day M

In [105]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/60m_transactions.csv'))

In [525]:
transactions_df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- amount: integer (nullable = true)



In [543]:
window_spec = Window.partitionBy('datecol').orderBy(col('amount').desc())

transactions_df.withColumn('datecol',substring(col('day'),1,9)).withColumn('mx',rank().over(window_spec))\
.filter(col('mx') == 1).select('transaction_id').show()

+--------------+
|transaction_id|
+--------------+
|             1|
|             6|
|             5|
|             8|
+--------------+



In [550]:
transactions_df.createOrReplaceTempView('transactionsdf')


spark.sql('''
 with main as (select transaction_id,substring(day,1,9) as day, amount from transactionsdf),
       one as (select transaction_id,amount, rank() over(partition by day order by amount desc) as rnk from main)
              select transaction_id from one where rnk = 1
         ''').show()

+--------------+
|transaction_id|
+--------------+
|             1|
|             6|
|             5|
|             8|
+--------------+



#### 61 159 League Statistics M

In [106]:
teams_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/61m_teams.csv'))

In [107]:
matches_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/61m_matches.csv'))

In [86]:
teams_df.printSchema()

root
 |-- team_id: integer (nullable = true)
 |-- team_name: string (nullable = true)



In [87]:
matches_df.printSchema()

root
 |-- home_team_id: integer (nullable = true)
 |-- away_team_id: integer (nullable = true)
 |-- home_team_goals: integer (nullable = true)
 |-- away_team_goals: integer (nullable = true)



In [299]:
matches_df.show()

+------------+------------+---------------+---------------+
|home_team_id|away_team_id|home_team_goals|away_team_goals|
+------------+------------+---------------+---------------+
|           1|           4|              0|              1|
|           1|           6|              3|              3|
|           4|           1|              5|              2|
|           6|           1|              0|              0|
+------------+------------+---------------+---------------+



In [348]:
name = matches_df.withColumn('match_played',array('home_team_id','away_team_id'))\
.groupby().agg(explode(flatten(collect_list(col('match_played')))).alias('team_id'))\
.groupby('team_id').agg(count(col('team_id')).alias('matches_played')).join(teams_df,'team_id','inner')\
.select('team_id','team_name','matches_played')

#####
home_point = matches_df.withColumnRenamed('home_team_id','team').withColumn('point',when( (col('home_team_goals') - col('away_team_goals') ) < 0,0 )
                                  .when( ( col('home_team_goals') > col('away_team_goals') ),3 )
                                  .when(( col('home_team_goals') == col('away_team_goals') ),1))\
                                  .select('team','point')

away_point = matches_df.withColumnRenamed('away_team_id','team').withColumn('point',when( (col('away_team_goals') - col('home_team_goals') ) < 0,0 )
                                  .when( ( col('away_team_goals') > col('home_team_goals') ),3 )
                                  .when(( col('home_team_goals') == col('away_team_goals') ),1))\
                                  .select('team','point')

one = home_point.union(away_point).groupby('team').agg(sum(col('point')).alias('points'))\
.select('team','points',lit('0').alias('goal_for'),lit('0').alias('goal_against'))
####
home_goal = matches_df.groupby('home_team_id').agg(sum(col('home_team_goals')).alias('goal_for'))\
.select(col('home_team_id').alias('team'),'goal_for')

away_goal = matches_df.groupby('away_team_id').agg(sum(col('away_team_goals')).alias('goal_for'))\
.select(col('away_team_id').alias('team'),'goal_for')

two = home_goal.union(away_goal).groupby('team').agg(sum('goal_for').alias('goal_for'))\
.select('team',lit('0').alias('points'),'goal_for',lit('0').alias('goal_against'))
####
home_agnt = matches_df.groupby('home_team_id').agg(sum(col('away_team_goals')).alias('goal_against'))\
.select(col('home_team_id').alias('team'),'goal_against')

away_agnt = matches_df.groupby('away_team_id').agg(sum(col('home_team_goals')).alias('goal_against'))\
.select(col('away_team_id').alias('team'),'goal_against')

three = home_agnt.union(away_agnt).groupby('team').agg(sum('goal_against').alias('goal_against'))\
.select('team',lit('0').alias('points'),lit('0').alias('goal_for'),'goal_against')
####
logic = one.union(two).union(three).groupby('team').agg(sum('points').alias('points'),sum('goal_for').alias('goal_for'),sum('goal_against').alias('goal_against'))

logic.join(name,name.team_id == logic.team,'inner').withColumn('goal_diff',expr("goal_for - goal_against"))\
.select('team_name','matches_played',col('points').cast('int').alias('points'),col('goal_for').cast('int').alias('goal_for')
,col('goal_against').cast('int').alias('goal_against'),col('goal_diff').cast('int').alias('goal_diff')).orderBy(col('goal_diff').desc()).show()

+---------+--------------+------+--------+------------+---------+
|team_name|matches_played|points|goal_for|goal_against|goal_diff|
+---------+--------------+------+--------+------------+---------+
| Dortmund|             2|     6|       6|           2|        4|
|  Arsenal|             2|     2|       3|           3|        0|
|     Ajax|             4|     2|       5|           9|       -4|
+---------+--------------+------+--------+------------+---------+



#### 62 160 Suspicious Bank Accounts M

In [108]:
accounts_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/62m_accounts.csv'))

In [109]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/62m_transactions.csv'))

In [90]:
accounts_df.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- max_income: integer (nullable = true)



In [91]:
transactions_df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- account_id: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- day: string (nullable = true)



In [136]:
window_spec = Window.partitionBy('account_id','month')
window_row = Window.partitionBy('account_id').orderBy(col('month').asc())

transactions_df.filter(col('type') == lit('Creditor')).withColumn('month',substring('day',1,7))\
.select('account_id','amount','month').withColumn('excedamt',sum('amount').over(window_spec))\
.select('account_id','month','excedamt').distinct()\
.withColumn('comp',when( ((col('account_id') == lit('3')) & (col('excedamt') > 21000 )),lit('0'))
                   .when( ((col('account_id') == lit('4')) & (col('excedamt') > 10400 )),lit('0')).otherwise(lit('1')))\
.withColumn('ls',lead('comp',1).over(window_row)).filter(col('comp') == col('ls')).select('account_id').show()



+----------+
|account_id|
+----------+
|         3|
+----------+



In [152]:
transactions_df.createOrReplaceTempView('transactionsdf')
accounts_df.createOrReplaceTempView('accountsdf')

spark.sql('''
 with main as (select account_id,amount,substring(day,1,7) as month from transactionsdf where type = "Creditor" ),
       one as (select account_id,month,sum(amount) as amt from main group by account_id,month 
                    order by account_id,month),
       two as (select account_id,month, case when (account_id = 3) and (amt > 21000) then "0"
                                           when (account_id = 4) and (amt > 10400) then "0"
                                           else "1" end as flag from one),
     three as (select account_id,month,flag,lead(flag) over(partition by account_id order by month asc) as led from two)
               select account_id from three where flag = led
        ''').show()

+----------+
|account_id|
+----------+
|         3|
+----------+



#### 63 162 Orders With Maximum Quantity Above Average M

In [110]:
orderdetail_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/63m_orderdetail.csv'))

In [154]:
orderdetail_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)



In [158]:
orderdetail_df.groupby('order_id').agg((sum(col('quantity'))/count(col('product_id'))).alias('avg'),max('quantity').alias('max_qty'))\
.filter(col('max_qty') > col('avg')).orderBy('order_id').show()
                                                                                                        

+--------+------------------+-------+
|order_id|               avg|max_qty|
+--------+------------------+-------+
|       1|12.333333333333334|     15|
|       2|               5.5|      8|
|       3|14.333333333333334|     20|
|       4|               5.0|      8|
+--------+------------------+-------+



In [161]:
orderdetail_df.createOrReplaceTempView('orderdetaildf')

spark.sql('''
          select order_id,sum(quantity)/count(product_id) as avg, max(quantity) as mxqty 
               from orderdetaildf group by order_id having avg < mxqty
       ''').show()

+--------+------------------+-----+
|order_id|               avg|mxqty|
+--------+------------------+-----+
|       1|12.333333333333334|   15|
|       3|14.333333333333334|   20|
|       4|               5.0|    8|
|       2|               5.5|    8|
+--------+------------------+-----+



#### 64 164 Group Employees of the Same Salary M

In [111]:
employees_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/64m_employees.csv'))

In [164]:
employees_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: integer (nullable = true)



In [173]:
window_spec = Window.partitionBy('salary')
window_rnk = Window.orderBy(col('salary').asc())

employees_df.withColumn('gp',count('salary').over(window_spec)).filter(col('gp') > 1)\
.withColumn('rank',dense_rank().over(window_rnk)).select('employee_id','name','salary','rank').orderBy('rank').show()

24/09/12 15:00:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-------+------+----+
|employee_id|   name|salary|rank|
+-----------+-------+------+----+
|          2|   Meir|  3000|   1|
|          3|Michael|  3000|   1|
|          7|Addilyn|  7400|   2|
|          9| Kannon|  7400|   2|
+-----------+-------+------+----+



In [183]:
employees_df.createOrReplaceTempView('employeesdf')

spark.sql('''
 with main as (select employee_id,name,salary,count(salary) over(partition by salary) as gp from employeesdf)
               select employee_id,name,salary,dense_rank() over(order by salary asc) as rank from main where gp > 1
                    order by rank

         ''').show()

24/09/12 15:07:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-------+------+----+
|employee_id|   name|salary|rank|
+-----------+-------+------+----+
|          2|   Meir|  3000|   1|
|          3|Michael|  3000|   1|
|          7|Addilyn|  7400|   2|
|          9| Kannon|  7400|   2|
+-----------+-------+------+----+



#### 65 166 Count Salary Categories M

In [112]:
accounts_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/65m_accounts.csv'))

In [185]:
accounts_df.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- income: integer (nullable = true)



In [208]:
accounts_df.withColumn('category',when( ((col('income') > lit(20000)) & (col('income') < lit(50000))),lit('Average Salary'))
                                  .when( (col('income') < lit(20000)),lit('Low Salary'))
                                 .when( (col('income') > lit(50000)),lit('High Salary'))     
                      ).show()


+----------+------+-----------+
|account_id|income|   category|
+----------+------+-----------+
|         3|108939|High Salary|
|         2| 12747| Low Salary|
|         8| 87709|High Salary|
|         6| 91796|High Salary|
+----------+------+-----------+



In [236]:
catagory_df = spark.createDataFrame([('Low Salary',),('Average Salary',),('High Salary',)],["category"] )

accounts_df.withColumn('category',when( col('income').between(20000,50000),lit('Average Salary'))
                                  .when( (col('income') < lit(20000)),lit('Low Salary'))
                                 .when( (col('income') > lit(50000)),lit('High Salary')).otherwise(0)    
                                ).groupby('category').agg(count('category').alias('accounts_count'))\
.join(catagory_df,'category','right').select('category',coalesce('accounts_count',lit('0')).alias('accounts_count')).show()

+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             1|
|Average Salary|             0|
|   High Salary|             3|
+--------------+--------------+



In [261]:
accounts_df.createOrReplaceTempView('accountsdf')
catagory_df.createOrReplaceTempView('catagorydf')

spark.sql('''
  with main as (select case when ((income > 20000) and (income < 50000)) then "Average Salary"
                  when income < 20000 then "Low Salary"
                  when income > 50000 then "High Salary"
                  else "0" end as category from accountsdf),
        one as (select category,count(category) as accounts_count from main group by category)
                select catagorydf.category,coalesce(accounts_count,'0') as accounts_count from one right join catagorydf on catagorydf.category = one.category
         ''').show()

+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             1|
|Average Salary|             0|
|   High Salary|             3|
+--------------+--------------+



#### 66 167  Confirmation Rate M

In [113]:
signups_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/66m_signups.csv'))

In [114]:
confirmations_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/66m_confirmations.csv'))

In [352]:
signups_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- time_stamp: string (nullable = true)



In [353]:
confirmations_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- time_stamp: string (nullable = true)
 |-- action: string (nullable = true)



In [375]:
window_spec = Window.partitionBy('user_id')
window_cnt = Window.partitionBy('user_id','action')

confirmations_df.withColumn('cnt',count('user_id').over(window_spec))\
.withColumn('concnt',when( (col('action') == lit('confirmed')),count('action').over(window_cnt)).otherwise(lit('0')))\
.withColumn('timcnt',when( (col('action') == lit('timeout')),count('action').over(window_cnt)).otherwise(lit('0')))\
.withColumn('preagg',when(col('concnt') != 0,col('concnt')/col('cnt')).otherwise(0)).filter(col('preagg') > 0)\
.select('user_id','preagg').join(signups_df,'user_id','right')\
.select(signups_df.user_id,round(coalesce(col('preagg'),lit('0')),2).alias('confirmation_rate')).distinct().show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      2|              0.5|
|      7|              1.0|
|      6|              0.0|
|      3|              0.0|
+-------+-----------------+



In [396]:
signups_df.createOrReplaceTempView('signupsdf')
confirmations_df.createOrReplaceTempView('confirmationsdf')

spark.sql('''
 with main as (select user_id,count(user_id) over(partition by user_id) as cnt,
                    case when action = "confirmed" then count(action) over(partition by user_id,action) else 0 end  as concnt,
                    case when action = "timeout" then count(action) over(partition by user_id,action) else 0 end  as timcnt from confirmationsdf),
       one as (select distinct user_id,concnt/cnt as aggper from main where concnt > 0 )
               select signupsdf.user_id,round(coalesce(aggper,0),2) as confirmation_rate  from one right join signupsdf on signupsdf.user_id = one.user_id   
          ''').show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+



#### 67 169 Strong Friendship M

In [115]:
friendship_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/67m_friendship.csv'))

In [398]:
friendship_df.printSchema()

root
 |-- user1_id: integer (nullable = true)
 |-- user2_id: integer (nullable = true)



In [590]:
window_arrcomp = Window.orderBy(col('user1_id').asc())

right = friendship_df.withColumn('rightarr',collect_list(col('user2_id')).over(window_arr)).select('user1_id','rightarr').distinct()
left = friendship_df.withColumn('leftarr',collect_list(col('user1_id')).over(window_arr)).select('user2_id','leftarr').distinct()

one = right.join(left,right.user1_id == left.user2_id,'left')\
.withColumn('distleftarr',coalesce(element_at(array_distinct('leftarr'),1),lit(0)))\
.withColumn('arr',array('distleftarr'))\
.withColumn('arryunion',array_union('rightarr','arr'))\
.withColumn('arrremove',array_remove('arryunion',1))\
.select('user1_id','user2_id','arrremove')

main = one.withColumn('arrremove',array_remove('arrremove',0))\
.withColumn('sizearr',size('arrremove')).filter(col('sizearr') > 2).select('user1_id','arrremove')

c_one = main.withColumn('user2_id',lead('user1_id',1).over(window_arrcomp))\
.withColumn('arrtwo',lead('arrremove',1).over(window_arrcomp))\
.withColumn('common_friend',size(array_intersect('arrremove','arrtwo'))).filter(col('common_friend') >= 3)\
.select('user1_id','user2_id','common_friend')

c_two = main.withColumn('user2_id',lead('user1_id',2).over(window_arrcomp))\
.withColumn('arrtwo',lead('arrremove',2).over(window_arrcomp))\
.withColumn('common_friend',size(array_intersect('arrremove','arrtwo'))).filter(col('common_friend') >= 3)\
.select('user1_id','user2_id','common_friend')

c_one.union(c_two).show()                                                                

24/09/13 00:04:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/13 00:04:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+--------+-------------+
|user1_id|user2_id|common_friend|
+--------+--------+-------------+
|       1|       2|            4|
|       1|       3|            3|
+--------+--------+-------------+



#### 68 170 All the Pairs With the Maximum Number of Common Followers M

In [116]:
relations_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/68m_relatiions.csv'))

In [593]:
relations_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- follower_id: integer (nullable = true)



In [630]:
window_spec = Window.partitionBy('user_id').orderBy(col('user_id').asc())
window_arr = Window.partitionBy('user_id')
window_ord = Window.orderBy(col('user_id').asc())

main = relations_df.withColumn('gp_lead',lead('follower_id',1,0).over(window_spec))\
.withColumn('arraylist',collect_list('follower_id').over(window_arr)).select('user_id','arraylist').distinct()

one = main.withColumn('user2_id',lead(col('user_id'),1).over(window_ord))\
.withColumn('commarr',lead('arraylist',1).over(window_ord))\
.withColumn('common_friend',size(array_intersect('arraylist','commarr'))).filter(col('common_friend') > 0)\
.select('user_id','user2_id','common_friend')

two = main.withColumn('user2_id',lead(col('user_id'),2).over(window_ord))\
.withColumn('commarr',lead('arraylist',2).over(window_ord))\
.withColumn('common_friend',size(array_intersect('arraylist','commarr'))).filter(col('common_friend') > 0)\
.select('user_id','user2_id','common_friend')

one.union(two).withColumn('rnk',rank().over(Window.orderBy(col('common_friend').desc()))).filter(col('rnk') == 1)\
.select('user_id','user2_id','common_friend').show()


24/09/13 00:41:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/13 00:41:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/13 00:41:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+--------+-------------+
|user_id|user2_id|common_friend|
+-------+--------+-------------+
|      1|       7|            3|
+-------+--------+-------------+



In [649]:
main.alias('d1').crossJoin(main.alias('d2'))\
.select(col('d1.user_id').alias('d1userid'),col('d2.user_id').alias('d2userid'),
        col('d1.arraylist').alias('d1arrlst'),col('d2.arraylist').alias('d2arrlst'))\
.withColumn('arr',size(array_intersect('d1arrlst','d2arrlst'))).filter(col('arr') > 2).filter(col('d1userid') != col('d2userid'))\
.select(col('d1userid').alias('user1_id'),col('d2userid').alias('user2_id')).show()


+--------+--------+
|user1_id|user2_id|
+--------+--------+
|       1|       7|
|       7|       1|
+--------+--------+



In [664]:
relations_df.createOrReplaceTempView('relationsdf')

spark.sql('''
  with main as (select user_id,collect_list(follower_id) over(partition by user_id) as arr from relationsdf),
        one as (select d1.user_id as d1user,d2.user_id as d2user,d1.arr as d1arr,d2.arr as d2arr from main d1 cross join main d2),
        two as (select d1user,d2user,size(array_intersect(d1arr,d2arr)) as size from one)
                select distinct d1user,d2user from two where size > 2 and d1user != d2user limit 1
         ''').show()

+------+------+
|d1user|d2user|
+------+------+
|     1|     7|
+------+------+



#### 69 173 Find Cutoff Score for Each School M

In [117]:
schools_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/69m_schools.csv'))

In [118]:
exam_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/69m_exam.csv'))

In [5]:
schools_df.printSchema()

root
 |-- school_id: integer (nullable = true)
 |-- capacity: integer (nullable = true)



In [6]:
exam_df.printSchema()

root
 |-- score: integer (nullable = true)
 |-- student_count: integer (nullable = true)



In [45]:
window_spec = Window.partitionBy("school_id").orderBy(col("score").asc())

schools_df.crossJoin(exam_df).filter(col('student_count') <=col('capacity') )\
.withColumn('divs',floor(expr("score/capacity")))\
.withColumn('rnk',row_number().over(window_spec)).filter(col('rnk') == 1).select('school_id','score')\
.join(schools_df,'school_id','right').select('school_id',coalesce(col('score'),lit('-1')).alias('score')).show()

+---------+-----+
|school_id|score|
+---------+-----+
|       11|  744|
|        5|  975|
|        9|   -1|
|       10|  749|
+---------+-----+



In [74]:
schools_df.createOrReplaceTempView('schools_df')
exam_df.createOrReplaceTempView('exam_df')

spark.sql('''
 with main as (select * from schools_df cross join exam_df where student_count <= capacity),
       one as (select school_id,capacity,score,student_count, row_number() over(partition by school_id order by score asc) as rnk from main),
       two as (select school_id,score from one  where rnk = 1)
               select schools_df.school_id,coalesce(score,-1) as score from two right join schools_df on two.school_id = schools_df.school_id
         ''').show()

+---------+-----+
|school_id|score|
+---------+-----+
|       11|  744|
|        5|  975|
|        9|   -1|
|       10|  749|
+---------+-----+



#### 70 174 Count the Number of Experiments M

In [119]:
experiments_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/70m_experiments.csv'))

In [76]:
experiments_df.printSchema()

root
 |-- experiment_id: integer (nullable = true)
 |-- platform: string (nullable = true)
 |-- experiment_name: string (nullable = true)



In [138]:
window_spec = Window.partitionBy('platform','exname').orderBy(col('platform').asc(),col('exname').asc())

exp = experiments_df.select(col('platform').alias('explatform'),col('experiment_name').alias('exname'))
main = experiment_name.crossJoin(platform).select('platform','experiment_name')

main.join(exp, (main.platform == exp.explatform) & 
          (main.experiment_name == exp.exname) ,'left')\
.withColumn('num_experiments',count('exname').over(window_spec))\
.withColumn('experiment_name',when( (col('exname').isNull() ),col('experiment_name')).otherwise(col('exname')))\
.select('platform','experiment_name','num_experiments').distinct().orderBy(col('platform').asc(),col('experiment_name').asc()).show()

+--------+---------------+---------------+
|platform|experiment_name|num_experiments|
+--------+---------------+---------------+
| Android|    Programming|              0|
| Android|        Reading|              1|
| Android|         Sports|              0|
|     IOS|    Programming|              1|
|     IOS|        Reading|              0|
|     IOS|         Sports|              1|
|     Web|    Programming|              1|
|     Web|        Reading|              2|
|     Web|         Sports|              0|
+--------+---------------+---------------+



In [153]:
experiments_df.createOrReplaceTempView('experimentsdf')

spark.sql('''
 with one as (select distinct platform from experimentsdf),
      two as (select distinct experiment_name from experimentsdf),
    three as (select * from one cross join two),
  expname as (select platform as explatform,experiment_name as exname from experimentsdf ),
    logic as (select platform,experiment_name,explatform,exname,
                   case when exname is null then experiment_name else exname end experimentname
                   from three left join expname on platform = explatform and experiment_name = exname)
              select platform,experimentname,count(exname) as num_experiments from logic 
                   group by platform,experimentname order by platform asc,experimentname asc
         ''').show()

+--------+--------------+---------------+
|platform|experimentname|num_experiments|
+--------+--------------+---------------+
| Android|   Programming|              0|
| Android|       Reading|              1|
| Android|        Sports|              0|
|     IOS|   Programming|              1|
|     IOS|       Reading|              0|
|     IOS|        Sports|              1|
|     Web|   Programming|              1|
|     Web|       Reading|              2|
|     Web|        Sports|              0|
+--------+--------------+---------------+



#### 71 175 Number of Accounts That Did Not Stream M

In [120]:
subscriptions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/71m_subscriptions.csv'))

In [121]:
streams_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/71m_streams.csv'))

In [157]:
subscriptions_df.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [158]:
streams_df.printSchema()

root
 |-- session_id: integer (nullable = true)
 |-- account_id: integer (nullable = true)
 |-- stream_date: string (nullable = true)



In [170]:
in_year = subscriptions_df.filter(year('end_date') == lit('2021')).select('account_id')
instream_year = streams_df.filter(year('stream_date') == lit('2021'))
instream_year.join(in_year,'account_id','right').select(coalesce(col('session_id'),lit(-1)).alias('session_id'))\
.groupby('session_id').agg(count(col('session_id')).alias('accounts_count')).filter(col('session_id') == -1).select('accounts_count').show()

+--------------+
|accounts_count|
+--------------+
|             2|
+--------------+



In [179]:
subscriptions_df.createOrReplaceTempView('subscriptionsdf')
streams_df.createOrReplaceTempView('streamsdf')

spark.sql('''
 with one as (select account_id from subscriptionsdf where year(end_date) = "2021"),
      two as (select account_id,session_id from streamsdf where year(stream_date) = "2021"),
    three as (select one.account_id,coalesce(session_id,-1) as session_id from one left join two on one.account_id = two.account_id)
              select count(session_id) as accounts_count from three where session_id = -1
         ''').show()

+--------------+
|accounts_count|
+--------------+
|             2|
+--------------+



#### 72 177 Accepted Candidates From the Interviews M

In [122]:
candidates_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/72m_candidates.csv'))

In [123]:
rounds_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/72m_rounds.csv'))

In [182]:
candidates_df.printSchema()

root
 |-- candidate_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- years_of_exp: integer (nullable = true)
 |-- interview_id: integer (nullable = true)



In [183]:
rounds_df.printSchema()

root
 |-- interview_id: integer (nullable = true)
 |-- round_id: integer (nullable = true)
 |-- score: integer (nullable = true)



In [188]:
one = candidates_df.filter(col('years_of_exp') >= 2)
two = rounds_df.groupby('interview_id').agg(sum(col('score')).alias('score'))
one.join(two,one.interview_id == two.interview_id,'inner').filter(col('score') > 15).select('candidate_id').show()

+------------+
|candidate_id|
+------------+
|           9|
+------------+



In [194]:
candidates_df.createOrReplaceTempView('candidatesdf')
rounds_df.createOrReplaceTempView('roundsdf')

spark.sql('''
  with one as (select candidate_id,interview_id from candidatesdf where years_of_exp >= 2),
       two as (select interview_id,sum(score) as score from roundsdf group by interview_id having score > 15)
              select candidate_id from one join two on one.interview_id = two.interview_id
        ''').show()

+------------+
|candidate_id|
+------------+
|           9|
+------------+



#### 73 178 The Category of Each Member in the Store M

In [124]:
members_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/73m_members.csv'))

In [125]:
visits_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/73m_visits.csv'))

In [126]:
purchases_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/73m_purchases.csv'))

In [198]:
members_df.printSchema()

root
 |-- member_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [199]:
visits_df.printSchema()

root
 |-- visit_id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- visit_date: string (nullable = true)



In [200]:
purchases_df.printSchema()

root
 |-- visit_id: integer (nullable = true)
 |-- charged_amount: integer (nullable = true)



In [245]:
window_spec = Window.partitionBy('member_id')
window_visit = Window.partitionBy('visit_id')

one = members_df.join(visits_df,members_df.member_id == visits_df.member_id,'left')\
.select(members_df.member_id,members_df.name,coalesce(col('visit_id'),lit(0)).alias('visit_id'))\
.withColumn('cntmem',count('member_id').over(window_spec))\
.withColumnRenamed('visit_id','visitid')

purchases_df.join(one,one.visitid == purchases_df.visit_id,'right')\
.withColumn('cntvisit',count('visit_id').over(window_visit))\
.withColumn('pre', round((100 * col('cntvisit'))/ col('cntmem'),2))\
.filter(~( (col('cntmem') >= 2) & (col('pre') == 0) )) \
.withColumn('category',when( (col('visitid') == 0 ),lit('Bronze'))
                      .when( (col('pre') >= 80),lit('Diamond'))
                      .when( (col('pre').between(50,80)),lit('Gold'))
                      .when( (col('pre') < 50 ),lit('Silver')) )\
.select('member_id','name','category').distinct().orderBy(col('member_id').asc()).show()

+---------+-------+--------+
|member_id|   name|category|
+---------+-------+--------+
|        1|Narihan|  Bronze|
|        3|Winston|  Silver|
|        8|  Hercy| Diamond|
|        9|  Alice|    Gold|
|       11|    Bob|  Silver|
+---------+-------+--------+



In [289]:
members_df.createOrReplaceTempView('membersdf')
visits_df.createOrReplaceTempView('visitsdf')
purchases_df.createOrReplaceTempView('purchasesdf')

spark.sql('''
 with one as (select membersdf.member_id as member_id,name,coalesce(visit_id,0) as visitid 
                   from membersdf left join visitsdf on membersdf.member_id = visitsdf.member_id),
   cntmem as (select member_id,name,visitid,count(member_id) over(partition by member_id) as cntmem from one),
 prelogic as (select member_id,name,visitid,cntmem,charged_amount,count(visit_id) over(partition by visit_id) as cntamt
                   from cntmem left join purchasesdf on cntmem.visitid = purchasesdf.visit_id),
  conrate as (select member_id,name,cntmem,cntamt,visitid,round((100 * cntamt/cntmem),2) as conrate  from prelogic),
   result as (select member_id,name,cntmem,conrate,visitid,case when visitid  == 0  then "Bronze"
                                         when conrate >= 80 then "Diamond"
                                         when conrate between 50 and 80 then "Gold"
                                         when conrate < 50 then "Silver" else 0 end as category 
                    from conrate where not (cntmem >= 2 and conrate = 0))
              select member_id,name,category from result order by member_id
        ''').show()

+---------+-------+--------+
|member_id|   name|category|
+---------+-------+--------+
|        1|Narihan|  Bronze|
|        3|Winston|  Silver|
|        8|  Hercy| Diamond|
|        9|  Alice|    Gold|
|       11|    Bob|  Silver|
+---------+-------+--------+



#### 74 179 Account Balance M

In [127]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/74m_transactions.csv'))

In [291]:
transactions_df.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: integer (nullable = true)



In [309]:
window_lead = Window.partitionBy('account_id').orderBy(col('day').asc())

transactions_df.withColumn('ntvamt',when(col('type') == lit('Withdraw'),col('amount') * -1).otherwise(col('amount')))\
.withColumn('balance',sum('ntvamt').over(window_lead)).select('account_id','day','balance').show()

+----------+----------+-------+
|account_id|       day|balance|
+----------+----------+-------+
|         1|2021-11-07|   2000|
|         1|2021-11-09|   1000|
|         1|2021-11-11|   4000|
|         2|2021-12-07|   7000|
|         2|2021-12-12|      0|
+----------+----------+-------+



In [314]:
transactions_df.createOrReplaceTempView('transactionsdf')

spark.sql('''
  with main as (select account_id,day,case when type == "Withdraw" then amount * -1 else amount end amt 
                     from transactionsdf)
                select account_id,day, sum(amt) over(partition by account_id order by day asc) as balance from main
          ''').show()

+----------+----------+-------+
|account_id|       day|balance|
+----------+----------+-------+
|         1|2021-11-07|   2000|
|         1|2021-11-09|   1000|
|         1|2021-11-11|   4000|
|         2|2021-12-07|   7000|
|         2|2021-12-12|      0|
+----------+----------+-------+



#### 75 182 Drop Type 1 Orders for Customers With Type 0 Orders M

In [128]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/75m_orders.csv'))

In [316]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_type: integer (nullable = true)



In [322]:
window_spec = Window.partitionBy('customer_id','order_type').orderBy(col('customer_id').asc())

orders_df.withColumn('cnt',count('order_type').over(window_spec))\
.withColumn('flag',when( col('cnt') == 2,lit('1'))
                  .when( ((col('order_type') == lit('0')) & (col('cnt') == lit('1'))),lit('1')))\
.filter(col('flag') == 1).select('order_id','customer_id','order_type').show()

+--------+-----------+----------+
|order_id|customer_id|order_type|
+--------+-----------+----------+
|       1|          1|         0|
|       2|          1|         0|
|      11|          2|         0|
|      22|          3|         0|
|      31|          4|         1|
|      32|          4|         1|
+--------+-----------+----------+



In [328]:
orders_df.createOrReplaceTempView('ordersdf')

spark.sql('''
 with main as (select order_id,customer_id,order_type,count(order_type) over(partition by customer_id,order_type) as cnt 
                    from ordersdf),
       one as (select order_id,customer_id,order_type,case when cnt = 2 then 1
                                                          when (order_type = 0) and (cnt = 1) then 1 
                                                          end as flag from main)
               select order_id,customer_id,order_type from one where flag = 1
        ''').show()

+--------+-----------+----------+
|order_id|customer_id|order_type|
+--------+-----------+----------+
|       1|          1|         0|
|       2|          1|         0|
|      11|          2|         0|
|      22|          3|         0|
|      31|          4|         1|
|      32|          4|         1|
+--------+-----------+----------+



#### 76 183 The Airport With the Most Traffic M

In [129]:
flights_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/76m_flights.csv'))

In [330]:
flights_df.printSchema()

root
 |-- departure_airport: integer (nullable = true)
 |-- arrival_airport: integer (nullable = true)
 |-- flights_count: integer (nullable = true)



In [347]:
dep = flights_df.groupby('departure_airport').agg(sum(col('flights_count')).alias('flight_cnt')).withColumnRenamed('departure_airport','airport')
arr = flights_df.groupby('arrival_airport').agg(sum(col('flights_count')).alias('flight_cnt')).withColumnRenamed('arrival_airport','airport')

dep.union(arr).groupby('airport').agg(sum((col('flight_cnt'))).alias('flight_cnt'))\
.withColumn('rnk',rank().over(Window.orderBy(col('flight_cnt').desc()))).filter(col('rnk') == 1).select('airport').show()

24/09/14 00:07:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+
|airport|
+-------+
|      2|
+-------+



In [362]:
flights_df.createOrReplaceTempView('flightsdf')

spark.sql('''
  with main as ((select departure_airport as airport,sum(flights_count) as flights_count 
                          from flightsdf group by departure_airport)
                      union(select arrival_airport as airport,sum(flights_count) as flights_count 
                          from flightsdf group by arrival_airport)),
        one as (select airport,sum(flights_count) as flights_count from main group by airport),
        two as (select airport,rank() over(order by flights_count desc) as rnk from one)
                select airport from two where rnk = 1
        ''').show()

24/09/14 00:21:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+
|airport|
+-------+
|      2|
+-------+



#### 77 184 The Number of Passengers in Each Bus I M

In [130]:
buses_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/77m_buses.csv'))

In [131]:
passengers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/77m_passengers.csv'))

In [365]:
buses_df.printSchema()

root
 |-- bus_id: integer (nullable = true)
 |-- arrival_time: integer (nullable = true)



In [366]:
passengers_df.printSchema()

root
 |-- passenger_id: integer (nullable = true)
 |-- arrival_time: integer (nullable = true)



In [387]:
one = buses_df.join(passengers_df,passengers_df.arrival_time <= buses_df.arrival_time,'left' )\
.groupBy("passenger_id").agg(min("bus_id").alias("earliest_bus_id"))
one.join(buses_df,one.earliest_bus_id == buses_df.bus_id,'right')\
.groupby('bus_id').agg(count('earliest_bus_id').alias('passengers_cnt')).show()

+------+--------------+
|bus_id|passengers_cnt|
+------+--------------+
|     1|             1|
|     3|             3|
|     2|             0|
+------+--------------+



In [405]:
buses_df.createOrReplaceTempView('busesdf')
passengers_df.createOrReplaceTempView('passengersdf')

spark.sql('''
  with main as (select * from busesdf left join 
                     passengersdf where passengersdf.arrival_time  <= busesdf.arrival_time),
        one as (select passenger_id, min(bus_id) as earlier_bus from main group by passenger_id)
                select distinct bus_id,coalesce(earlier_bus,0) as passengers_cnt from one right join busesdf on one.earlier_bus = busesdf.bus_id    
         ''').show()

+------+--------------+
|bus_id|passengers_cnt|
+------+--------------+
|     1|             1|
|     3|             3|
|     2|             0|
+------+--------------+



#### 78 185 Order Two Columns Independently M

In [132]:
data_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/78m_data.csv'))

In [7]:
data_df.printSchema()

root
 |-- first_col: integer (nullable = true)
 |-- second_col: integer (nullable = true)



In [8]:
data_df.show()

+---------+----------+
|first_col|second_col|
+---------+----------+
|        4|         2|
|        2|         3|
|        3|         1|
|        1|         4|
+---------+----------+



In [14]:
window_rnk1  = Window.orderBy(col('first_col').asc())
window_rnk2  = Window.orderBy(col('second_col').desc())

one = data_df.select('first_col').orderBy(col('first_col').asc()).withColumn('rnk',rank().over(window_rnk1))
two = data_df.select('second_col').orderBy(col('second_col').desc()).withColumn('rnk',rank().over(window_rnk2))

one.join(two,'rnk','inner').select('first_col','second_col').show()

24/09/15 15:24:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/15 15:24:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+----------+
|first_col|second_col|
+---------+----------+
|        1|         4|
|        2|         3|
|        3|         2|
|        4|         1|
+---------+----------+



In [23]:
data_df.createOrReplaceTempView('datadf')

spark.sql('''
  with one as (select first_col,rank() over(order by first_col asc) as rnk from datadf),
       two as (select second_col, rank() over (order by second_col desc) as rnk from datadf)
               select one.first_col,two.second_col from one,two where one.rnk=two.rnk
          ''').show()

24/09/15 15:31:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/15 15:31:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+----------+
|first_col|second_col|
+---------+----------+
|        1|         4|
|        2|         3|
|        3|         2|
|        4|         1|
+---------+----------+



#### 79 186 The Change in Global Rankings M

In [133]:
teampoints_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/79m_teampoints.csv'))

In [134]:
pointschange_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/79m_pointschange.csv'))

In [26]:
teampoints_df.printSchema()

root
 |-- team_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- points: integer (nullable = true)



In [27]:
pointschange_df.printSchema()

root
 |-- team_id: integer (nullable = true)
 |-- points_change: integer (nullable = true)



In [28]:
teampoints_df.show()

+-------+-----------+------+
|team_id|       name|points|
+-------+-----------+------+
|      3|    Algeria|  1431|
|      1|    Senegal|  2132|
|      2|New Zealand|  1402|
|      4|    Croatia|  1817|
+-------+-----------+------+



In [39]:
window_spec = Window.orderBy(col('points').desc())
window_rnk = Window.orderBy(col('diff').desc(),col('name').asc())

teampoints_df.withColumn('rnkglobal',rank().over(window_spec)).join(pointschange_df,'team_id','inner')\
.withColumn('diff',expr("points + points_change")).withColumn('rnkdiff',rank().over(window_rnk))\
.withColumn('rank_diff',col('rnkglobal') - col('rnkdiff')).select('team_id','name',col('diff').alias('points'),'rank_diff').show()


+-------+-----------+------+---------+
|team_id|       name|points|rank_diff|
+-------+-----------+------+---------+
|      1|    Senegal|  2110|        0|
|      3|    Algeria|  1830|        1|
|      4|    Croatia|  1830|       -1|
|      2|New Zealand|  1402|        0|
+-------+-----------+------+---------+



24/09/15 15:47:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/15 15:47:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [52]:
teampoints_df.createOrReplaceTempView('teampointsdf')
pointschange_df.createOrReplaceTempView('pointschangedf')

spark.sql('''
 with one as (select teampointsdf.team_id,name,points, rank() over(order by points desc) as rnk,points_change, (points + points_change) as pntchg
                   from teampointsdf,pointschangedf where teampointsdf.team_id = pointschangedf.team_id),
      two as (select team_id,name,pntchg as point,rnk,rank() over(order by pntchg desc, name asc) as rnktwo from one)
              select team_id,name,point, (rnk - rnktwo) as rank_diff from two


        ''').show()

+-------+-----------+-----+---------+
|team_id|       name|point|rank_diff|
+-------+-----------+-----+---------+
|      1|    Senegal| 2110|        0|
|      3|    Algeria| 1830|        1|
|      4|    Croatia| 1830|       -1|
|      2|New Zealand| 1402|        0|
+-------+-----------+-----+---------+



24/09/15 15:59:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/15 15:59:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 80 188 Users With Two Purchases Within Seven Days M

In [135]:
purchases_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/80m_purchases.csv'))

In [55]:
purchases_df.printSchema()

root
 |-- purchase_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- purchase_date: string (nullable = true)



In [71]:
window_spec = Window.partitionBy('user_id').orderBy(col('purchase_date').asc())

purchases_df.withColumn('leaddate',lead('purchase_date',1).over(window_spec))\
.withColumn('dffdate',datediff(col('purchase_date'),col('leaddate'))).filter(col('dffdate') <= 7)\
.select('user_id').distinct().show()

+-------+
|user_id|
+-------+
|      7|
|      2|
+-------+



In [81]:
purchases_df.createOrReplaceTempView('purchasesdf')

spark.sql('''
 with one as (select user_id,purchase_date,lead(purchase_date,1) over(partition by user_id order by purchase_date asc) as leaddate 
               from purchasesdf),
      two as (select user_id,datediff(purchase_date,leaddate) as diff from one)
              select distinct user_id from two where diff <= 7
         ''').show()

+-------+
|user_id|
+-------+
|      7|
|      2|
+-------+



#### 81 190 Number of Times a Driver Was a Passenger M

In [136]:
rides_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/81m_rides.csv'))

In [85]:
rides_df.printSchema()

root
 |-- ride_id: integer (nullable = true)
 |-- driver_id: integer (nullable = true)
 |-- passenger_id: integer (nullable = true)



In [97]:
one = rides_df.select('driver_id').distinct()
two = rides_df.groupby('passenger_id').agg(count('passenger_id').alias('cnt'))

one.join(two,one.driver_id == two.passenger_id,'left')\
.select('driver_id',coalesce(col('cnt'),lit('0')).alias('cnt')).show()

+---------+---+
|driver_id|cnt|
+---------+---+
|        7|  2|
|       11|  0|
+---------+---+



In [101]:
rides_df.createOrReplaceTempView('ridesdf')

spark.sql('''
  with one as (select distinct driver_id from ridesdf),
       two as (select passenger_id,count('passenger_id') as cnt from ridesdf group by passenger_id)
               select driver_id,coalesce(cnt,0) as cnt from one left join two on one.driver_id = two.passenger_id 

         ''').show()

+---------+---+
|driver_id|cnt|
+---------+---+
|        7|  2|
|       11|  0|
+---------+---+



#### 82 191 Products With Three or More Orders in Two Consecutive Years M

In [137]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/82m_orders.csv'))

In [106]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- purchase_date: string (nullable = true)



In [111]:
window_spec = Window.partitionBy('product_id').orderBy(col('purchase_date').asc())

orders_df.withColumn('rownum',row_number().over(window_spec)).filter(col('rownum') >=3)\
.select('product_id').distinct().show()

+----------+
|product_id|
+----------+
|         1|
+----------+



In [115]:
orders_df.createOrReplaceTempView('ordersdf')

spark.sql('''
  with one as (select product_id,row_number() over(partition by product_id order by purchase_date asc) as rownum
                    from ordersdf)
               select distinct product_id from one where rownum >=3
         ''').show()

+----------+
|product_id|
+----------+
|         1|
+----------+



#### 83 192 Tasks Count in the Weekend M

In [138]:
tasks_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/83m_task.csv'))

In [118]:
tasks_df.printSchema()

root
 |-- task_id: integer (nullable = true)
 |-- assignee_id: integer (nullable = true)
 |-- submit_date: string (nullable = true)



In [147]:
window_spec = Window.partitionBy('weekend')

tasks_df.withColumn('dayweek',dayofweek('submit_date'))\
.withColumn('weekend',when ( ((col('dayweek') >=2) & (col('dayweek') <= 6)),lit('0') ).otherwise(lit('1')))\
.withColumn('weekend_cnt',when(col('weekend') == lit(1),count('weekend').over(window_spec)))\
.withColumn('working_cnt',when(col('weekend') == lit(0),count('weekend').over(window_spec)))\
.select(coalesce('weekend_cnt',lit(0)).alias('weekend_cnt'),coalesce('working_cnt',lit(0)).alias('working_cnt'))\
.distinct().select(sum(col('weekend_cnt')).alias('weekend_cnt'),sum(col('working_cnt')).alias('working_cnt')).show()

+-----------+-----------+
|weekend_cnt|working_cnt|
+-----------+-----------+
|          3|          3|
+-----------+-----------+



In [161]:
tasks_df.createOrReplaceTempView('tasksdf')

spark.sql('''
 with main as (select submit_date,dayofweek(submit_date) as day from tasksdf),
       one as (select case when day between 2 and 6 then 0 else 1 end range from main),
       two as (select case when range = 0 then count(range) end as working_cnt,
                      case when range = 1 then count(range) end as weekend_cnt from one group by range)
               select sum(coalesce(weekend_cnt,0)) as weekend_cnt,sum(coalesce(working_cnt,0)) as working_cnt from two
         ''').show()

+-----------+-----------+
|weekend_cnt|working_cnt|
+-----------+-----------+
|          3|          3|
+-----------+-----------+



#### 84 193 Arrange Table by Gender M

In [139]:
genders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/84m_genders.csv'))

In [8]:
genders_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- gender: string (nullable = true)



In [25]:
winddow_spec = Window.partitionBy('flag').orderBy(col('flag').desc(),col('user_id').asc())

main = genders_df.withColumn('flag',when(col('gender') == lit('female'),lit('1'))
                            .when(col('gender') == lit('other'), lit('2'))
                            .when(col('gender') == lit('male'),  lit('3')))
one = main.withColumn('rnk',rank().over(winddow_spec)).filter(col('rnk') == 1).orderBy('flag').select('user_id','gender')
two = main.withColumn('rnk',rank().over(winddow_spec)).filter(col('rnk') == 2).orderBy('flag').select('user_id','gender')
three = main.withColumn('rnk',rank().over(winddow_spec)).filter(col('rnk') == 3).orderBy('flag').select('user_id','gender')

one.union(two).union(three).show()

+-------+------+
|user_id|gender|
+-------+------+
|      3|female|
|      1| other|
|      4|  male|
|      7|female|
|      2| other|
|      5|  male|
|      9|female|
|      6| other|
|      8|  male|
+-------+------+



In [153]:
genders_df.createOrReplaceTempView('gendersdf')

spark.sql('''
 with main as (select user_id,gender,case when gender = "female" then 1
                                      when gender = "other"  then 2
                                      when gender =  "male"  then 3
                                      end as flag from gendersdf),
       one as (select user_id,gender,flag,rank() over(partition by flag order by flag desc,user_id asc) as rnk from main),
       two as ((select user_id,gender,flag,1 as ord from one where rnk = 1 order by flag)
               union
               (select user_id,gender,flag,2 as ord from one where rnk = 2 order by flag)
               union
               (select user_id,gender,flag,3 as ord from one where rnk = 3 order by flag)
               order by ord asc,flag asc)
               select user_id,gender from two  
          ''').show()

+-------+------+
|user_id|gender|
+-------+------+
|      3|female|
|      1| other|
|      4|  male|
|      7|female|
|      2| other|
|      5|  male|
|      9|female|
|      6| other|
|      8|  male|
+-------+------+



#### 85 194 The First Day of the Maximum Recorded Degree in Each City M

In [140]:
weather_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/85m_weather.csv'))

In [166]:
weather_df.printSchema()

root
 |-- city_id: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- degree: integer (nullable = true)



In [175]:
window_spec = Window.partitionBy('city_id')
window_rnk = Window.partitionBy('city_id').orderBy(col('day').asc())

weather_df.withColumn('maxtemp',max('degree').over(window_spec))\
.withColumn('rnk',rank().over(window_rnk)).filter(col('rnk') == 1)\
.select('city_id','day',col('maxtemp').alias('degree')).orderBy('city_id').show()

+-------+----------+------+
|city_id|       day|degree|
+-------+----------+------+
|      1|2022-01-07|    24|
|      2|2022-08-07|    37|
|      3|2022-02-07|    -6|
+-------+----------+------+



In [179]:
weather_df.createOrReplaceTempView('weatherdf')

spark.sql('''
 with main as (select city_id,day,max(degree) over(partition by city_id) as degree,
                     rank() over(partition by city_id order by day asc)  as rnk from weatherdf)
            select city_id,day,degree from main where rnk = 1 order by city_id
         ''').show()

+-------+----------+------+
|city_id|       day|degree|
+-------+----------+------+
|      1|2022-01-07|    24|
|      2|2022-08-07|    37|
|      3|2022-02-07|    -6|
+-------+----------+------+



#### 86 195 Product Sales Analysis IV M

In [141]:
product_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/86m_product.csv'))

In [142]:
sales_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/86m_sales.csv'))

In [182]:
product_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- price: integer (nullable = true)



In [183]:
sales_df.printSchema()

root
 |-- sale_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)



In [206]:
window_spec = Window.partitionBy('user_id').orderBy(col('saleprice').desc())
window_sumpid = Window.partitionBy('user_id','product_id')

sales_df.join(product_df,'product_id','inner').withColumn('sumqty',sum('quantity').over(window_sumpid))\
.withColumn('saleprice',expr("sumqty * price")).withColumn('mstspent',rank().over(window_spec)).filter(col('mstspent') == 1)\
.select('user_id','product_id').distinct().show()

+-------+----------+
|user_id|product_id|
+-------+----------+
|    101|         3|
|    102|         3|
|    102|         2|
|    102|         1|
+-------+----------+



In [221]:
product_df.createOrReplaceTempView('productdf')
sales_df.createOrReplaceTempView('salesdf')

spark.sql('''
  with main as (select productdf.product_id,user_id,quantity,price,sum(quantity) over(partition by user_id,productdf.product_id) as qty 
                     from productdf,salesdf where productdf.product_id = salesdf.product_id),
                     
        one as (select product_id,user_id, (qty * price) as salesprice from main),
        two as (select product_id,user_id,rank() over(partition by user_id order by salesprice desc) as rnk from one)
                select distinct user_id,product_id from two where rnk = 1
         ''').show()

+-------+----------+
|user_id|product_id|
+-------+----------+
|    101|         3|
|    102|         3|
|    102|         2|
|    102|         1|
+-------+----------+



#### 87 198 Compute the Rank as a Percentage M

In [143]:
students_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/87m_students.csv'))

In [241]:
students_df.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- mark: integer (nullable = true)



In [249]:
window_spec = Window.partitionBy('department_id')
window_rnk = Window.partitionBy('department_id').orderBy(col('mark').desc())

students_df.withColumn('cnt',count('student_id').over(window_spec))\
.withColumn('rnk',rank().over(window_rnk))\
.withColumn('percentage',round(expr(" (rnk - 1) * 100 /(cnt - 1)"),2))\
.select('student_id','department_id','percentage').show()

+----------+-------------+----------+
|student_id|department_id|percentage|
+----------+-------------+----------+
|         7|            1|       0.0|
|         1|            1|      50.0|
|         3|            1|     100.0|
|         2|            2|       0.0|
|         8|            2|       0.0|
+----------+-------------+----------+



In [252]:
students_df.createOrReplaceTempView('studentsdf')

spark.sql('''
  with main as (select student_id,department_id,mark,count(student_id) over(partition by department_id) cnt,
                     rank() over(partition by department_id order by mark desc) rnk from studentsdf)
                select student_id,department_id,((rnk - 1) * 100 /(cnt - 1)) as percentage from main
         ''').show()

+----------+-------------+----------+
|student_id|department_id|percentage|
+----------+-------------+----------+
|         7|            1|       0.0|
|         1|            1|      50.0|
|         3|            1|     100.0|
|         2|            2|       0.0|
|         8|            2|       0.0|
+----------+-------------+----------+



#### 88 200 Calculate the Influence of Each Salesperson M

In [144]:
salesperson_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/88m_salesperson.csv'))

In [145]:
customer_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/88m_customer.csv'))

In [146]:
sales_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/88m_sales.csv'))

In [256]:
salesperson_df.printSchema()

root
 |-- salesperson_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [257]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- salesperson_id: integer (nullable = true)



In [258]:
sales_df.printSchema()

root
 |-- sale_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- price: integer (nullable = true)



In [266]:
customer_df.join(sales_df,'customer_id','inner').groupby('salesperson_id').agg(sum(col('price')).alias('total'))\
.join(salesperson_df,'salesperson_id','right').select('salesperson_id','name',coalesce('total',lit('0')).alias('total')).show()

+--------------+-----+-----+
|salesperson_id| name|total|
+--------------+-----+-----+
|             1|Alice| 1246|
|             2|  Bob| 1844|
|             3|Jerry|    0|
+--------------+-----+-----+



In [272]:
salesperson_df.createOrReplaceTempView('salespersondf')
customer_df.createOrReplaceTempView('customerdf')
sales_df.createOrReplaceTempView('salesdf')

spark.sql('''
  with main as (select salesperson_id,sum(price) as total from customerdf,salesdf 
                     where customerdf.customer_id = salesdf.customer_id group by salesperson_id)
                select salespersondf.salesperson_id,name,coalesce(total,0) as total from salespersondf left join main on
                      salespersondf.salesperson_id = main.salesperson_id


         ''').show()

+--------------+-----+-----+
|salesperson_id| name|total|
+--------------+-----+-----+
|             1|Alice| 1246|
|             2|  Bob| 1844|
|             3|Jerry|    0|
+--------------+-----+-----+



#### 89 202 hange Null Values in a Table to the Previous Value M

In [147]:
coffeeshop_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/89m_coffeeshop.csv'))

In [28]:
coffeeshop_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- drink: string (nullable = true)



In [79]:
window_spec = Window.orderBy(col('index').asc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)

coffeeshop_df.withColumn('index',monotonically_increasing_id())\
.withColumn('drink',last('drink',ignorenulls=True).over(window_spec)).show()


+---+-----------------+-----+
| id|            drink|index|
+---+-----------------+-----+
|  9|     Rum and Coke|    0|
|  6|     Rum and Coke|    1|
|  7|     Rum and Coke|    2|
|  3|St Germain Spritz|    3|
|  1| Orange Margarita|    4|
|  2| Orange Margarita|    5|
+---+-----------------+-----+



24/09/15 22:22:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [83]:
window_spec = Window.rowsBetween(-2,0)

coffeeshop_df.withColumn('index',monotonically_increasing_id())\
.withColumn('drink',last('drink',ignorenulls=True).over(window_spec)).show()

+---+-----------------+-----+
| id|            drink|index|
+---+-----------------+-----+
|  9|     Rum and Coke|    0|
|  6|     Rum and Coke|    1|
|  7|     Rum and Coke|    2|
|  3|St Germain Spritz|    3|
|  1| Orange Margarita|    4|
|  2| Orange Margarita|    5|
+---+-----------------+-----+



24/09/15 22:28:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [141]:
coffeeshop_df.createOrReplaceTempView('coffeeshopdf')

spark.sql('''
           select id,last(drink,True) over(ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as drink from coffeeshopdf
         ''').show()

+---+-----------------+
| id|            drink|
+---+-----------------+
|  9|     Rum and Coke|
|  6|     Rum and Coke|
|  7|     Rum and Coke|
|  3|St Germain Spritz|
|  1| Orange Margarita|
|  2| Orange Margarita|
+---+-----------------+



24/09/16 00:34:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 90 203 Employees With Deductions M

In [148]:
employees_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/90m_employees.csv'))

In [149]:
logs_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/90m_logs.csv'))

In [289]:
employees_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- needed_hours: integer (nullable = true)



In [290]:
logs_df.printSchema() # unix_timestamp

root
 |-- employee_id: integer (nullable = true)
 |-- in_time: string (nullable = true)
 |-- out_time: string (nullable = true)



In [336]:
window_spec = Window.partitionBy('employee_id')

logs_df.withColumn('uintime',expr("unix_timestamp(in_time)")).withColumn('uouttime',expr("unix_timestamp(out_time)"))\
.withColumn('diff',round( (expr("uouttime - uintime ")/3600),4))\
.withColumn('to',sum('diff').over(window_spec)).join(employees_df,'employee_id','right')\
.select('employee_id',coalesce('to',lit('0')).alias('timeworked'),'needed_hours')\
.filter(col('timeworked') < col('needed_hours')).select('employee_id').show()

+-----------+
|employee_id|
+-----------+
|          2|
|          3|
+-----------+



In [360]:
employees_df.createOrReplaceTempView('employeesdf')
logs_df.createOrReplaceTempView('logsdf')

spark.sql('''
 with main as (select employee_id,
                     sum(round((unix_timestamp(out_time) - unix_timestamp(in_time))/3600,4)) as workedhours from logsdf
                     group by employee_id),
       one as (select employeesdf.employee_id,coalesce(workedhours,0) as workedhours,needed_hours from main right join employeesdf
                     on main.employee_id = employeesdf.employee_id)
               select employee_id from one where workedhours < needed_hours
         ''').show()

+-----------+
|employee_id|
+-----------+
|          2|
|          3|
+-----------+



#### 91 208 Immediate Food Delivery III M

In [150]:
delivery_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/91m_delivery.csv'))

In [5]:
delivery_df.printSchema()

root
 |-- delivery_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_pref_delivery_date: string (nullable = true)



In [6]:
window_spec = Window.partitionBy('order_date')
window_cnt = Window.partitionBy('diff','order_date')

one  = delivery_df.withColumn('cnt',count('order_date').over(window_spec))\
.withColumn('diff',datediff('customer_pref_delivery_date','order_date'))\
.withColumn('diffcnt',count('diff').over(window_cnt))\
.withColumn('immediate_percentage',round(expr("(diffcnt/cnt)*100"),2) )\
.filter(col('diff') == 0)\
.select('order_date','immediate_percentage').distinct()

delivery_df.select('order_date').distinct().join(one,'order_date','left')\
.select('order_date',coalesce('immediate_percentage',lit('0.00')).alias('immediate_percentage')).orderBy('order_date').show()

+----------+--------------------+
|order_date|immediate_percentage|
+----------+--------------------+
|2019-08-01|               66.67|
|2019-08-02|               66.67|
|2019-08-03|               100.0|
|2019-08-04|                0.00|
+----------+--------------------+



In [22]:
delivery_df.createOrReplaceTempView('deliverydf')

spark.sql('''
 with main as (select order_date,count(order_date) over(partition by order_date) cnt,
                    datediff(customer_pref_delivery_date,order_date) as diff from deliverydf),
       one as (select cnt,diff,count(diff) over(partition by diff,order_date) diffcnt,order_date from main),
       two as (select order_date,cnt,diff,diffcnt,round((diffcnt/cnt)*100,2) immediate_percentage from one),
     three as (select distinct order_date,immediate_percentage from two where diff = 0 )
               select distinct deliverydf.order_date,coalesce(immediate_percentage,0) as immediate_percentage from deliverydf 
                    left join three on deliverydf.order_date = three.order_date order by order_date
         ''').show()

+----------+--------------------+
|order_date|immediate_percentage|
+----------+--------------------+
|2019-08-01|               66.67|
|2019-08-02|               66.67|
|2019-08-03|               100.0|
|2019-08-04|                 0.0|
+----------+--------------------+



#### 92 210 Find Active Users M

In [151]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/92m_users.csv'))

In [30]:
users_df.printSchema() #greatest

root
 |-- user_id: integer (nullable = true)
 |-- item: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- amount: integer (nullable = true)



In [34]:
window_spec = Window.partitionBy('user_id').orderBy(col('created_at').asc())

users_df.withColumn('rnk',lead('created_at',1).over(window_spec))\
.withColumn('diff',datediff('rnk','created_at')).filter(col('diff') <= 7).select('user_id').show()

+-------+
|user_id|
+-------+
|      6|
+-------+



In [40]:
users_df.createOrReplaceTempView('usersdf')

spark.sql('''
  with main as (select user_id,created_at,lead(created_at,1) over(partition by user_id order by created_at asc) diff from usersdf)
                select user_id from main where datediff(diff,created_at) <= 7
        ''').show()

+-------+
|user_id|
+-------+
|      6|
+-------+



#### 93 211 Count Occurrences in Text M

In [152]:
file_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/93m_files.csv'))

In [45]:
file_df.printSchema()

root
 |-- file_name: string (nullable = true)
 |-- content: string (nullable = true)



In [96]:
file_df.withColumn("one",regexp_extract(col("content"), r"(bull)", 0)).select('one')\
.union(file_df.withColumn("one",regexp_extract(col("content"), r"(bear)", 0)).select('one'))\
.groupby('one').agg(count('one').alias('count')).filter((col("one") != "")).show()
  

+----+-----+
| one|count|
+----+-----+
|bear|    2|
|bull|    3|
+----+-----+



In [107]:
file_df.createOrReplaceTempView('filedf')

spark.sql('''
 with main as ((select file_name,regexp_extract(content,"(bull)",0) as one  from filedf)
                     union
                         (select file_name, regexp_extract(content,"(bear)",0) as one  from filedf))
             select one,count(one) as count from main group by one having one !=""
        ''').show()

+----+-----+
| one|count|
+----+-----+
|bear|    2|
|bull|    3|
+----+-----+



#### 94 212  Flight Occupancy and Waitlist Analysis M

In [153]:
flights_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/94m_flights.csv'))

In [154]:
passengers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/94m_passengers.csv'))

In [110]:
flights_df.printSchema()

root
 |-- flight_id: integer (nullable = true)
 |-- capacity: integer (nullable = true)



In [111]:
passengers_df.printSchema()

root
 |-- passenger_id: integer (nullable = true)
 |-- flight_id: integer (nullable = true)



In [119]:
window_spec = Window.partitionBy('flight_id')

flights_df.join(passengers_df,'flight_id','inner').withColumn('cnt',count('passenger_id').over(window_spec))\
.withColumn('waitlist_cnt',expr("cnt - capacity")).select('flight_id',col('capacity').alias('booked_cnt'),'waitlist_cnt')\
.distinct().orderBy('flight_id').show()

+---------+----------+------------+
|flight_id|booked_cnt|waitlist_cnt|
+---------+----------+------------+
|        1|         2|           1|
|        2|         2|           0|
|        3|         1|           1|
+---------+----------+------------+



In [126]:
passengers_df.createOrReplaceTempView('passengersdf')
flights_df.createOrReplaceTempView('flightsdf')

spark.sql('''
 with main as (select flightsdf.flight_id,capacity,passenger_id,count(passenger_id) over(partition by flightsdf.flight_id) as cnt 
                    from passengersdf join flightsdf on  passengersdf.flight_id = flightsdf.flight_id)
                select distinct flight_id, (cnt-capacity) as waitlist_cnt from main order by flight_id
         ''').show()

+---------+------------+
|flight_id|waitlist_cnt|
+---------+------------+
|        1|           1|
|        2|           0|
|        3|           1|
+---------+------------+



#### 95 213 Election Results M

In [155]:
votes_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/95m_votes.csv'))

In [128]:
votes_df.printSchema()

root
 |-- voter: string (nullable = true)
 |-- candidate: string (nullable = true)



In [151]:
window_spec = Window.partitionBy('voter')
window_rnk = Window.orderBy(col('cndidate').desc())

votes_df.withColumn('cnt',count('candidate').over(window_spec)).withColumn('cntg',count('voter').over(window_spec))\
.withColumn('logic',when(col('cntg') > 1,1/col('cntg')).otherwise(col('cnt') ))\
.withColumn('cndidate',round(sum('logic').over(Window.partitionBy('candidate')),2))\
.withColumn('rnk',rank().over(window_rnk)).filter(col('rnk') == 1).select('candidate').distinct().show()

24/09/16 15:24:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+
|candidate|
+---------+
|Christine|
|     Ryan|
+---------+



In [166]:
votes_df.createOrReplaceTempView('votesdf')

spark.sql('''
 with one as (select voter,candidate,count(voter) over(partition by voter) as cnt,
                   count(candidate) over(partition by candidate) as cancnt from votesdf),
      two as (select voter,candidate,cnt,cancnt,case when cnt > 1 then round(1/cnt,2) else cnt end as ratiop from one)
              select candidate from two group by candidate having sum(ratiop) > 2 
         ''').show()

+---------+
|candidate|
+---------+
|Christine|
|     Ryan|
+---------+



#### 96 216 Rolling Average Steps M

In [156]:
steps_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/96m_steps.csv'))

In [168]:
steps_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- steps_count: integer (nullable = true)
 |-- steps_date: string (nullable = true)



In [235]:
window_spec = Window.partitionBy('user_id').orderBy(col('steps_date').asc()).rowsBetween(0,2)
window_led = Window.partitionBy('user_id').orderBy(col('steps_date'))
window_cnt = Window.partitionBy('user_id')

steps_df.withColumn('rng',avg(col('steps_count')).over(window_spec))\
.withColumn('lead',lead('steps_date',1).over(window_led))\
.withColumn('diff',datediff('lead','steps_date')).filter(col('diff') == 1)\
.withColumn('cnt',count('user_id').over(window_cnt)).filter(col('cnt') > 1)\
.withColumn('lst',last('steps_count').over(window_cnt)).filter(col('lst') != col('steps_count'))\
.select('user_id','steps_date',round(col('rng'),2).alias('rolling_average')).orderBy('user_id').show()

+-------+----------+---------------+
|user_id|steps_date|rolling_average|
+-------+----------+---------------+
|      1|2021-09-04|         535.33|
|      1|2021-09-05|         595.67|
|      2|2021-09-06|         284.67|
|      3|2021-09-07|         505.67|
|      3|2021-09-08|         674.67|
+-------+----------+---------------+



In [270]:
steps_df.createOrReplaceTempView('stepsdf')

spark.sql('''
  with main as (select user_id,steps_count,steps_date,avg(steps_count) 
                     over(partition by user_id order by steps_date asc rows between 0 preceding and 2 Following) as avgrng,
                     lead(steps_date,1) over(partition by user_id order by steps_date asc) leaddt from stepsdf),
        one as (select user_id,steps_count,steps_date,avgrng,count(user_id) over(partition by user_id) as usrcnt,
                      last(steps_count) over(partition by user_id) as lst from main where datediff(leaddt,steps_date)  = 1)
                select distinct user_id,steps_date,round(avgrng,2) as rolling_average 
                      from one where lst != steps_count order by user_id
         ''').show()

+-------+----------+---------------+
|user_id|steps_date|rolling_average|
+-------+----------+---------------+
|      1|2021-09-04|         535.33|
|      1|2021-09-05|         595.67|
|      2|2021-09-06|         284.67|
|      3|2021-09-07|         505.67|
|      3|2021-09-08|         674.67|
+-------+----------+---------------+



#### 97 217 Calculate Orders Within Each Interval M

In [157]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/97m_orders.csv'))

In [272]:
orders_df.printSchema()

root
 |-- minute: integer (nullable = true)
 |-- order_count: integer (nullable = true)



In [288]:
window_spec = Window.orderBy(col('minute').asc()).rowsBetween(0,5)
window_lead = Window.orderBy(col('minute').asc())

orders_df.withColumn('total_orders',sum('order_count').over(window_spec))\
.withColumn('xx',lead('minute',6).over(window_lead))\
.withColumn('diff',expr("xx - minute"))\
.withColumn('rnk',expr("xx-diff")).filter(col('rnk') <= 2).select(col('rnk').alias('interval_no'),'total_orders').show()

+-----------+------------+
|interval_no|total_orders|
+-----------+------------+
|          1|          17|
|          2|          18|
+-----------+------------+



24/09/16 20:12:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/16 20:12:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [292]:
orders_df.createOrReplaceTempView('ordersdf')

spark.sql('''
 with main as (select minute,order_count,sum(order_count) 
                     over(order by minute asc rows between 0 preceding and 5 Following) as total_orders,
                     lead(minute,6) over(order by minute asc) as le from ordersdf),
       one as (select minute,order_count,total_orders,(le - minute) as diff,le - (le - minute) as rnk from main)
               select rnk as interval_no,total_orders from one where rnk <= 2
       ''').show()

+-----------+------------+
|interval_no|total_orders|
+-----------+------------+
|          1|          17|
|          2|          18|
+-----------+------------+



24/09/16 20:20:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 98 218 Market Analysis III M

In [158]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/98m_users.csv'))

In [159]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/98m_orders.csv'))

In [160]:
items_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/98m_items.csv'))

In [296]:
users_df.printSchema()

root
 |-- seller_id: integer (nullable = true)
 |-- join_date: string (nullable = true)
 |-- favorite_brand: string (nullable = true)



In [297]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- seller_id: integer (nullable = true)



In [300]:
items_df.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- item_brand: string (nullable = true)



In [310]:
orders_df.join(items_df,'item_id','inner').join(users_df,'seller_id','right')\
.filter(col('favorite_brand') == col('item_brand')).groupby('seller_id').agg(count(col('favorite_brand')).alias('num_items'))\
.orderBy('seller_id').show()


+---------+---------+
|seller_id|num_items|
+---------+---------+
|        2|        1|
|        3|        1|
+---------+---------+



In [317]:
users_df.createOrReplaceTempView('usersdf')
orders_df.createOrReplaceTempView('ordersdf')
items_df.createOrReplaceTempView('itemsdf')

spark.sql('''
  with one as (select ordersdf.seller_id,item_brand,favorite_brand from itemsdf 
                    join ordersdf on itemsdf.item_id = ordersdf.item_id
                    right join usersdf on usersdf.seller_id = ordersdf.seller_id)             
               select seller_id,count(item_brand) as num_items from one where item_brand = favorite_brand 
                    group by seller_id order by seller_id                                        
        ''').show()

+---------+---------+
|seller_id|num_items|
+---------+---------+
|        2|        1|
|        3|        1|
+---------+---------+



#### 99 219 Symmetric Coordinates M

In [161]:
coordinates_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/99m_coordinates.csv'))

In [319]:
coordinates_df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)



In [404]:
window_y = Window.orderBy(col('x').asc())
window_x = Window.orderBy(col('y').asc())
winddow_spec = Window.orderBy('x')
winddow_x = Window.orderBy('x2')

first = coordinates_df.orderBy('x')\
.withColumn('y2',lead('y',1).over(window_y))\
.withColumn('ro',row_number().over(winddow_spec))

second = coordinates_df.orderBy('x')\
.withColumn('x2',lead('x',1).over(window_x))\
.withColumn('ro',row_number().over(winddow_x)).select('x2','ro').orderBy(col('x2').asc_nulls_last())\
.select('x2',expr("ro - 1").alias('ro'))

first.join(second,'ro','inner').select(col('x').alias('x1'),'y2','x2',col('y').alias('y1'))\
.withColumn('flag',when( ((col('x1') == col('y2') ) & ( col('x2') == col('y1') )) ,lit('1')).otherwise(lit('0')))\
.filter(col('flag') == 1).select(col('x1').alias('x'),col('y1').alias('y')).show()


+---+---+
|  x|  y|
+---+---+
| 20| 20|
| 20| 21|
| 22| 23|
+---+---+



24/09/16 22:34:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/16 22:34:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/16 22:34:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 100 220 Find Peak Calling Hours for Each City M

In [162]:
calls_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/100m_calls.csv'))

In [406]:
calls_df.printSchema()

root
 |-- caller_id: integer (nullable = true)
 |-- recipient_id: integer (nullable = true)
 |-- call_time: string (nullable = true)
 |-- city: string (nullable = true)



In [419]:
window_spec = Window.partitionBy('peak_calling_hour')

calls_df.withColumn('peak_calling_hour',hour('call_time')).withColumn('number_of_calls',count('peak_calling_hour').over(window_spec))\
.select('city','peak_calling_hour','number_of_calls').distinct()\
.orderBy(col('peak_calling_hour').desc(),'city').show()


+--------+-----------------+---------------+
|    city|peak_calling_hour|number_of_calls|
+--------+-----------------+---------------+
| Houston|               22|              3|
| Houston|               21|              1|
|New York|               14|              1|
|New York|               13|              1|
+--------+-----------------+---------------+



In [423]:
calls_df.createOrReplaceTempView('callsdf')

spark.sql('''
 with main as (select city,hour(call_time) as peak_calling_hour from callsdf)
             select city,peak_calling_hour,count(peak_calling_hour) as number_of_calls from main
                  group by peak_calling_hour,city order by peak_calling_hour desc, city
       ''').show()

+--------+-----------------+---------------+
|    city|peak_calling_hour|number_of_calls|
+--------+-----------------+---------------+
| Houston|               22|              3|
| Houston|               21|              1|
|New York|               14|              1|
|New York|               13|              1|
+--------+-----------------+---------------+



#### 101 222 Find Third Transaction M

In [163]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/101m_transactions.csv'))

In [425]:
transactions_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- spend: double (nullable = true)
 |-- transaction_date: string (nullable = true)



In [438]:
window_spec = Window.partitionBy('user_id')
window_rnk = Window.partitionBy('user_id').orderBy(col('transaction_date').asc())

transactions_df.withColumn('cnt',count('user_id').over(window_spec)).filter(col('cnt') > 2)\
.withColumn('rnk',rank().over(window_rnk)).filter(col('rnk') <= 3)\
.withColumn('1st',lead('spend',1).over(window_rnk))\
.withColumn('1stdate',lead('transaction_date',1).over(window_rnk))\
.withColumn('2st',lead('1st',1).over(window_rnk))\
.withColumn('2stdate',lead('1stdate',1).over(window_rnk))\
.withColumn('logic',when( ((col('2st') > col('1st')) & (col('2st') > col('1st')) ),lit('1')).otherwise(lit('0')))\
.filter(col('logic') == 1).select('user_id',col('2st').alias('third_transaction_spend'),col('2stdate').alias('third_transaction_date')).show()

+-------+-----------------------+----------------------+
|user_id|third_transaction_spend|third_transaction_date|
+-------+-----------------------+----------------------+
|      1|                  65.56|   2023-11-18 13:49:42|
+-------+-----------------------+----------------------+



In [464]:
transactions_df.createOrReplaceTempView('transactionsdf')

spark.sql('''
   with main as (select user_id,spend,transaction_date,rank() over(partition by user_id order by transaction_date asc) as rnk from transactionsdf),
         one as (select user_id,spend,transaction_date,
                      lead(spend,1) over(partition by user_id order by transaction_date asc) as 1st,
                      lead(transaction_date,1) over(partition by user_id order by transaction_date asc) as 1stdate
                      from main where rnk <= 3),
         two as (select user_id,spend,transaction_date,1st,1stdate,
                      lead(1st,1) over(partition by user_id order by transaction_date asc) as 2st,
                      lead(1stdate,1) over(partition by user_id order by transaction_date asc) as 2stdate
                      from one),
       three as (select user_id,2st as third_transaction_spend,case when (2st > 1st) and (2st > spend) then 1 else 0 end as flag, 
                      2stdate as third_transaction_date from two)
                 select user_id,third_transaction_spend,third_transaction_date from three where flag = 1
         ''').show()

+-------+-----------------------+----------------------+
|user_id|third_transaction_spend|third_transaction_date|
+-------+-----------------------+----------------------+
|      1|                  65.56|   2023-11-18 13:49:42|
+-------+-----------------------+----------------------+



#### 102 224 Manager of the Largest Department M

In [164]:
employees_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/102m_employees.csv'))

In [466]:
employees_df.printSchema()

root
 |-- emp_id: integer (nullable = true)
 |-- emp_name: string (nullable = true)
 |-- dep_id: integer (nullable = true)
 |-- position: string (nullable = true)



In [471]:
window_spec = Window.partitionBy('dep_id')
window_rnk = Window.orderBy(col('cnt').desc())

employees_df.withColumn('cnt',count('dep_id').over(window_spec))\
.withColumn('rnk',rank().over(window_rnk)).filter( (col('rnk') == 1) & (col('position') == lit('Manager')))\
.select(col('emp_name').alias('manager_name'),'dep_id').show()

24/09/17 00:18:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+------+
|manager_name|dep_id|
+------------+------+
|    Isabella|   101|
|      Joseph|   100|
+------------+------+



In [477]:
employees_df.createOrReplaceTempView('employeesdf')

spark.sql('''
  with one as (select emp_id,emp_name,dep_id,position,count(dep_id) over(partition by dep_id) as cnt from employeesdf),
       two as (select emp_id,emp_name,dep_id,position,cnt,rank() over(order by cnt desc) as rnk from one)  
               select emp_name as manager_name,dep_id from two where position = "Manager" and rnk = 1
        ''').show()

24/09/17 00:25:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+------+
|manager_name|dep_id|
+------------+------+
|    Isabella|   101|
|      Joseph|   100|
+------------+------+



#### 103 225 Class Performance M

In [166]:
scores_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/103m_scores.csv'))

In [5]:
scores_df.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- student_name: string (nullable = true)
 |-- assignment1: integer (nullable = true)
 |-- assignment2: integer (nullable = true)
 |-- assignment3: integer (nullable = true)



In [40]:
scores_df.withColumn('sm',array('assignment1','assignment2','assignment3'))\
.withColumn('sum',aggregate('sm',lit(0.0),lambda acc, x: acc + x))\
.select( (expr("max(sum)").alias('mx') - expr("min(sum)").alias('mi')).alias('difference_in_score') ) .show()

+-------------------+
|difference_in_score|
+-------------------+
|              111.0|
+-------------------+



In [41]:
scores_df.createOrReplaceTempView('scoresdf')

spark.sql('''
 with main as (select (assignment1 + assignment2 + assignment3) as sum_array from scoresdf)
               select (max(sum_array) - min(sum_array)) as difference_in_score from main
        ''').show()

+-------------------+
|difference_in_score|
+-------------------+
|                111|
+-------------------+



#### 104 227 Friday Purchases I M

In [167]:
purchases_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/104m_purchases.csv'))

In [43]:
purchases_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- amount_spend: integer (nullable = true)



In [69]:
window_spec = Window.partitionBy('purchase_date')

purchases_df.withColumn('weekday',dayofweek('purchase_date'))\
.withColumn('weekofyear',weekofyear('purchase_date'))\
.withColumn('dayofmonth',dayofmonth('purchase_date'))\
.withColumn('firstdayofmonth',expr("date_sub(purchase_date, dayofmonth(purchase_date) - 1)"))\
.withColumn('fistweekofmont',weekofyear('firstdayofmonth'))\
.withColumn('weekofmonth',expr("weekofyear - fistweekofmont + 1"))\
.orderBy('purchase_date').filter(col('weekday') == 6 )\
.withColumn('total_amount',sum('amount_spend').over(window_spec))\
.select('weekofmonth','purchase_date','total_amount').distinct().show()

+-----------+-------------+------------+
|weekofmonth|purchase_date|total_amount|
+-----------+-------------+------------+
|          1|   2023-11-03|        5117|
|          4|   2023-11-24|       21692|
+-----------+-------------+------------+



In [89]:
purchases_df.createOrReplaceTempView('purchasesdf')

spark.sql('''
 with main as (select purchase_date,amount_spend,dayofweek(purchase_date) as weekday,
                                    weekofyear(purchase_date) as weekofyear,
                                    dayofmonth(purchase_date) as dayofmonth,
                                    date_sub(purchase_date,dayofmonth(purchase_date) -1) as firstdayofmonth
                                    from  purchasesdf),
       one as (select purchase_date,amount_spend,weekday,weekofyear,dayofmonth,firstdayofmonth,
                                    weekofyear(firstdayofmonth) as fistweekofmonth,
                                    (weekofyear - weekofyear(firstdayofmonth) + 1) as weekofmonth
                                    from main where weekday == 6)
               select distinct weekofmonth,purchase_date,sum(amount_spend) over(partition by purchase_date) as total_amount 
                    from one order by purchase_date
          ''').show()

+-----------+-------------+------------+
|weekofmonth|purchase_date|total_amount|
+-----------+-------------+------------+
|          1|   2023-11-03|        5117|
|          4|   2023-11-24|       21692|
+-----------+-------------+------------+



#### 105 228 Pizza Toppings Cost Analysis M

In [168]:
toppings_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/105m_toppings.csv'))

In [144]:
toppings_df.printSchema()

root
 |-- topping_name: string (nullable = true)
 |-- cost: double (nullable = true)



In [143]:
window_spec = Window.partitionBy('ln')
window_cnt = Window.partitionBy('srt')
window_rnk = Window.partitionBy('srt').orderBy(col('cntsrt').desc())

toppings_df.alias('d1').crossJoin(toppings_df.alias('d2')).crossJoin(toppings_df.alias('d3'))\
.select(array('d1.topping_name','d2.topping_name','d3.topping_name').alias('arr'))\
.withColumn('xxx',array_distinct('arr'))\
.withColumn('ln',size('xxx')).filter(col('ln') > 2)\
.withColumn('cnt',count('ln').over(window_spec)).select('arr').distinct().orderBy('arr')\
.withColumn('srt',array_sort('arr'))\
.withColumn('cntsrt',count('srt').over(window_cnt))\
.withColumn('rnk',row_number().over(window_rnk)).filter(col('rnk') == 1).select('arr')\
.withColumn('lst',array_join('arr',',')).select('lst')\
.withColumn('total_cost',when( col('lst') == lit("Chicken,Extra Cheese,Pepperoni"),expr("0.55+0.4+0.5"))
                        .when( col('lst') == lit("Chicken,Pepperoni,Sausage"),expr("0.55+0.5+0.7"))
                        .when( col('lst') == lit("Chicken,Extra Cheese,Sausage"),expr("0.55+0.4+0.7"))
                        .when(col('lst') == lit("Extra Cheese,Pepperoni,Sausage"),expr("0.4+0.5+0.7")))\
.show(truncate = False)

+------------------------------+----------+
|lst                           |total_cost|
+------------------------------+----------+
|Chicken,Extra Cheese,Pepperoni|1.45      |
|Chicken,Pepperoni,Sausage     |1.75      |
|Chicken,Extra Cheese,Sausage  |1.65      |
|Extra Cheese,Pepperoni,Sausage|1.60      |
+------------------------------+----------+



In [188]:
toppings_df.createOrReplaceTempView('toppingsdf')

spark.sql('''
 with one as (select d1.topping_name from toppingsdf as d1),
      two as (select d2.topping_name from toppingsdf as d2),
    three as (select d3.topping_name from toppingsdf as d3),
     four as (select one.topping_name,two.topping_name,three.topping_name,
                   array(one.topping_name,two.topping_name,three.topping_name) as arr_name
                   from one cross join two cross join three),
     five as (select distinct array_sort(arr_name) as arr_name from four where size(array_distinct(arr_name))  = 3 order by arr_name),
      six as (select array_join(arr_name,',') as pizza from five)
              select pizza, case when pizza = "Chicken,Extra Cheese,Pepperoni" then 0.55+0.4+0.5
                                when pizza = "Chicken,Extra Cheese,Sausage" then 0.55+0.5+0.7
                                when pizza = "Chicken,Pepperoni,Sausage" then 0.55+0.4+0.7
                                when pizza = "Extra Cheese,Pepperoni,Sausage" then 0.4+0.5+0.7
                                end total_cost
                    from six
       ''').show(truncate=False)

+------------------------------+----------+
|pizza                         |total_cost|
+------------------------------+----------+
|Chicken,Extra Cheese,Pepperoni|1.45      |
|Chicken,Extra Cheese,Sausage  |1.75      |
|Chicken,Pepperoni,Sausage     |1.65      |
|Extra Cheese,Pepperoni,Sausage|1.60      |
+------------------------------+----------+



#### 106 231 Binary Tree Nodes M

In [169]:
tree_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/106m_tree.csv'))

In [250]:
tree_df.printSchema()

root
 |-- N: integer (nullable = true)
 |-- P: integer (nullable = true)



In [251]:
tree_df = tree_df.select(col('N').alias('id'),coalesce(col('P'),lit('0')).alias('p_id'))

In [258]:
root_df =  tree_df.withColumn('type',when((col('p_id')) == 0,lit('root'))).filter(col('type') == lit('root'))\
.select('id','type')

leaf_df = tree_df.alias('d1').join(tree_df.alias('d2'),col('d1.id')==col('d2.p_id'),'anti')\
.withColumn('type',lit('leaf')).select('id','type')

inner_df = tree_df.alias('d1').join(tree_df.alias('d2'),col('d1.id')==col('d2.p_id'),'leftsemi')\
.withColumn('type',lit('inner')).select('id','type')\
.filter((col('id') != tree_df.filter(col('p_id') == 0).select('id').collect()[0][0]))

root_df.union(leaf_df).union(inner_df).orderBy('id').show()

+---+-----+
| id| type|
+---+-----+
|  1| leaf|
|  2|inner|
|  3| leaf|
|  5| root|
|  6| leaf|
|  8|inner|
|  9| leaf|
+---+-----+



In [260]:
tree_df.createOrReplaceTempView('treedf')

spark.sql('''
  with root as (select id,'root' as type from treedf where p_id = 0),
       leaf as (select id, 'leaf' as type from treedf as d1 anti join treedf as d2 on d1.id == d2.p_id ),
      inner as (select id, 'inner' as type from treedf as d1 left semi join treedf as d2 on d1.id == d2.p_id 
                    where id not in (select id from root))
                select * from root union(select * from leaf) union (select * from inner) order by id
         ''').show()

+---+-----+
| id| type|
+---+-----+
|  1| leaf|
|  2|inner|
|  3| leaf|
|  5| root|
|  6| leaf|
|  8|inner|
|  9| leaf|
+---+-----+



#### 107 232 Top Percentile Fraud M

In [170]:
fraud_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/107m_fraud.csv'))

In [262]:
fraud_df.printSchema()

root
 |-- policy_id: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- fraud_score: double (nullable = true)



In [272]:
window_spec = Window.partitionBy('state')

fraud_df.withColumn('per',percentile_approx('fraud_score',0.95).over(window_spec))\
.filter(col('fraud_score') == col('per')).select('policy_id','state','fraud_score').orderBy('state').show()

+---------+----------+-----------+
|policy_id|     state|fraud_score|
+---------+----------+-----------+
|        1|California|       0.92|
|       11|   Florida|       0.98|
|        4|  New York|       0.94|
|        7|     Texas|       0.98|
+---------+----------+-----------+



In [277]:
fraud_df.createOrReplaceTempView('frauddf')

spark.sql('''
 with main as (select policy_id,state,fraud_score,percentile_approx(fraud_score,0.95) over(partition by state) as percentail 
                     from frauddf)
            select policy_id,state,fraud_score from main where fraud_score == percentail order by state
         ''').show()

+---------+----------+-----------+
|policy_id|     state|fraud_score|
+---------+----------+-----------+
|        1|California|       0.92|
|       11|   Florida|       0.98|
|        4|  New York|       0.94|
|        7|     Texas|       0.98|
+---------+----------+-----------+



#### 108 233 Snaps Analysis M

In [171]:
activities_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/108m_activities.csv'))

In [172]:
age_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/108m_age.csv'))

In [294]:
activities_df.printSchema()

root
 |-- activity_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- activity_type: string (nullable = true)
 |-- time_spent: double (nullable = true)



In [295]:
age_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- age_bucket: string (nullable = true)



In [311]:
window_spec = Window.partitionBy('user_id')

send = activities_df.filter(col('activity_type') == lit('send'))\
.select('user_id',col('time_spent').alias('send_time')).withColumn('send_user',sum('send_time').over(window_spec))

open = activities_df.filter(col('activity_type') == lit('open'))\
.select('user_id',col('time_spent').alias('open_time')).withColumn('open_user',sum('open_time').over(window_spec))

age_df.join(send,'user_id','inner').join(open,'user_id','inner')\
.withColumn('total_user',expr("send_user + open_user"))\
.withColumn(('send_perc'),round(expr("send_user/total_user") * 100,2))\
.withColumn(('open_perc'),round(expr("open_user/total_user") * 100,2))\
.select('age_bucket','send_perc','open_perc').distinct().show()

+----------+---------+---------+
|age_bucket|send_perc|open_perc|
+----------+---------+---------+
|     31-35|    37.84|    62.16|
|     21-25|    54.31|    45.69|
|     26-30|    82.26|    17.74|
+----------+---------+---------+



In [327]:
activities_df.createOrReplaceTempView('activitiesdf')
age_df.createOrReplaceTempView('agedf')

spark.sql('''
 with send_user as (select user_id,sum(time_spent) over(partition by user_id) as send_user from activitiesdf where activity_type = "send"),
      open_user as (select user_id,sum(time_spent) over(partition by user_id) as open_user from activitiesdf where activity_type = "open"), 
      totaltime as (select agedf.user_id,age_bucket,send_user,open_user,(send_user + open_user) as total_user from agedf
                         join send_user on send_user.user_id = agedf.user_id
                         join open_user on open_user.user_id = agedf.user_id)
              select distinct age_bucket,round(((send_user/total_user)*100),2) as send_perc,
                    round(((open_user/total_user)*100),2) as open_perc from totaltime
         ''').show()

+----------+---------+---------+
|age_bucket|send_perc|open_perc|
+----------+---------+---------+
|     31-35|    37.84|    62.16|
|     21-25|    54.31|    45.69|
|     26-30|    82.26|    17.74|
+----------+---------+---------+



#### 109 234 Friends With No Mutual Friends M

In [173]:
friends_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/109m_friends.csv'))

In [110]:
friends_df.printSchema()

root
 |-- user_id1: integer (nullable = true)
 |-- user_id2: integer (nullable = true)



In [140]:
window_spec = Window.partitionBy('user_id1')

left = friends_df.withColumn('cnt',count('user_id1').over(window_spec)).filter(col('cnt').isin(['2','3']))\
.withColumn('arr',explode(array('user_id1','user_id2'))).select('arr').distinct().collect()

out = [left[i][0] for i in range(0,len(left))]

right = friends_df.withColumn('cnt',count('user_id1').over(window_spec)).filter(col('cnt').isin(['1']))\
.withColumn('arr',explode(array('user_id1','user_id2'))).select('arr').distinct()

right.filter(~col('arr').isin(out)).join(friends_df,right.arr == friends_df.user_id1,'inner')\
.select('user_id1','user_id2').show()

+--------+--------+
|user_id1|user_id2|
+--------+--------+
|       6|       7|
|       8|       9|
+--------+--------+



#### 110 236 Find Trending Hashtags M

In [174]:
tweets_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/110m_tweets.csv'))

In [331]:
tweets_df.printSchema() #

root
 |-- user_id: integer (nullable = true)
 |-- tweet_id: integer (nullable = true)
 |-- tweet: string (nullable = true)
 |-- tweet_date: string (nullable = true)



In [346]:
widow_spec = Window.partitionBy('regex')

tweets_df.withColumn('regex',regexp_extract('tweet',r"(#\w+)",1))\
.withColumn('hashtag_count',count('regex').over(widow_spec))\
.select(col('regex').alias('hashtag'),'hashtag_count').distinct().orderBy(col('hashtag_count').desc()).show()

+---------+-------------+
|  hashtag|hashtag_count|
+---------+-------------+
|#HappyDay|            3|
|#TechLife|            2|
|  #Nature|            1|
|#WorkLife|            1|
+---------+-------------+



In [387]:
tweets_df.createOrReplaceTempView('tweetsdf')

spark.sql('''
 with main as (select regexp_extract(tweet,"(#\HappyDay|\#TechLife|\#Nature|#WorkLife)", 1) as hashtag from tweetsdf)
               select hashtag,count(hashtag) as hashtag_count from main group by hashtag order by hashtag_count desc 
         ''').show(truncate=False)

+---------+-------------+
|hashtag  |hashtag_count|
+---------+-------------+
|#HappyDay|3            |
|#TechLife|2            |
|#Nature  |1            |
|#WorkLife|1            |
+---------+-------------+



#### 111 237  Find Bursty Behavior M

In [175]:
posts_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/111m_posts.csv'))

In [390]:
posts_df.printSchema()

root
 |-- post_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- post_date: string (nullable = true)



#### 112 238 Friday Purchase III  M

In [176]:
purchases_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/112m_purchases.csv'))

In [177]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/112m_users.csv'))

In [5]:
purchases_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- amount_spend: integer (nullable = true)



In [6]:
users_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- membership: string (nullable = true)



In [41]:
window_spec = Window.partitionBy('membership','weekofmonth')

one = purchases_df.join(users_df,'user_id','inner')\
.withColumn('weekday',dayofweek('purchase_date'))\
.withColumn('weekofyear',weekofyear('purchase_date'))\
.withColumn('dayofmonth',dayofmonth('purchase_date'))\
.withColumn('firstdayofmonth',expr("date_sub(purchase_date,dayofmonth(purchase_date) -1)"))\
.withColumn('firstweekofmonth',weekofyear('firstdayofmonth'))\
.withColumn('weekofmonth',expr("weekofyear - firstweekofmonth +1"))

premium = one.select('weekofmonth','membership','amount_spend').filter(col('membership') == lit('Premium')).distinct()
vip = one.select(col('weekofmonth').alias('vipweekofmonth'),col('membership').alias('vipmembership'),col('amount_spend').alias('vipamount_spend') )\
.filter(col('membership') == lit('VIP')).distinct()

main = premium.join(vip,premium.weekofmonth == vip.vipweekofmonth,'outer')\
.withColumn('weekofmonth',when(col('weekofmonth').isNull(),col('vipweekofmonth')).otherwise(col('weekofmonth') ))\
.withColumn('vipweekofmonth',when(col('vipweekofmonth').isNull(),col('weekofmonth')).otherwise(col('vipweekofmonth')))

x = main.select('weekofmonth',coalesce('membership',lit('Premium')).alias('membership'),
                            coalesce('amount_spend',lit('0')).alias('amount_spend')).distinct()

y = main.select(col('vipweekofmonth').alias('weekofmonth'),coalesce('vipmembership',lit('VIP')).alias('membership'),
                            coalesce('vipamount_spend',lit('0')).alias('amount_spend')).distinct()
left = x.union(y).orderBy('weekofmonth','membership').withColumn('amount_spend',sum('amount_spend').over(window_spec)).distinct()

z = one.select('weekofmonth').filter(col('weekofmonth') == 3).crossJoin(left.select('membership').distinct())\
.select('weekofmonth','membership',lit('0').alias('total_amount'))

left.union(z).orderBy('weekofmonth','membership').show()

+-----------+----------+------------+
|weekofmonth|membership|amount_spend|
+-----------+----------+------------+
|          1|   Premium|      1126.0|
|          1|       VIP|         0.0|
|          2|   Premium|         0.0|
|          2|       VIP|      7473.0|
|          3|   Premium|           0|
|          3|       VIP|           0|
|          4|   Premium|     17117.0|
|          4|       VIP|     14933.0|
+-----------+----------+------------+



#### 113 239 Find Longest Calls M

In [178]:
contacts_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/113m_contacts.csv'))

In [179]:
calls_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/113m_calls.csv'))

In [168]:
contacts_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)



In [45]:
calls_df.printSchema()

root
 |-- contact_id: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- duration: integer (nullable = true)



In [153]:
window_spec = Window.orderBy(col('duration').desc())

indoming = calls_df.filter(col('type') == lit('incoming'))\
.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') <= 3)

outgoing = calls_df.filter(col('type') == lit('outgoing'))\
.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') <= 3)

one = indoming.union(outgoing).join(contacts_df,col('id') == col('contact_id'),'inner')\
.select('first_name','type','duration')\
.withColumn('hours',floor(expr("duration/3600")))\
.withColumn('minutes',floor(expr("duration % 3600 / 60")))\
.withColumn('seconds',floor(expr("duration % 60")))\
.withColumn('duration_formatted',concat(lpad('hours',2,'0'),lit(':'),
                                        lpad('minutes',2,'0'),lit(':'),
                                        lpad('seconds',2,'0')))\
.select('first_name','type','duration_formatted').show()

+----------+--------+------------------+
|first_name|    type|duration_formatted|
+----------+--------+------------------+
|   Michael|incoming|          00:07:00|
|      Jane|incoming|          00:05:00|
|     Emily|incoming|          00:03:00|
|     Alice|outgoing|          00:06:00|
|     Emily|outgoing|          00:04:40|
|      Jane|outgoing|          00:04:00|
+----------+--------+------------------+



24/09/18 15:01:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/18 15:01:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [175]:
contacts_df.createOrReplaceTempView('contactsdf')
calls_df.createOrReplaceTempView('callsdf')

spark.sql('''
 with one as (select contact_id,type,duration,rank() over(order by duration) as rnk from callsdf where type = "incoming"),
      two as (select contact_id,type,duration,rank() over(order by duration) as rnk from callsdf where type = "outgoing"),
    third as ((select contact_id,type,duration from one where rnk >= 3) 
                    union (select contact_id,type,duration from two where rnk >=3)),
     four as (select first_name ,type,duration,floor(duration/3600) as hour,
                   floor(duration/60) as minutes,floor(duration%60) as seconds from contactsdf join third on id = contact_id)
             select first_name ,type,duration,concat(lpad(hour,2,'0'),":",lpad(minutes,2,'0'),":",lpad(seconds,2,'0')) as duration_formatted
                  from four
        ''').show()

24/09/18 15:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/18 15:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+--------+--------+------------------+
|first_name|    type|duration|duration_formatted|
+----------+--------+--------+------------------+
|     Alice|outgoing|     360|          00:06:00|
|   Michael|incoming|     420|          00:07:00|
|      Jane|outgoing|     240|          00:04:00|
|      Jane|incoming|     300|          00:05:00|
|     Emily|outgoing|     280|          00:04:40|
|     Emily|incoming|     180|          00:03:00|
+----------+--------+--------+------------------+



#### 114 240 Server Utilization Time M

In [180]:
servers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/114m_servers.csv'))

In [243]:
servers_df.printSchema()

root
 |-- server_id: integer (nullable = true)
 |-- status_time: string (nullable = true)
 |-- session_status: string (nullable = true)



In [419]:
window_spec = Window.partitionBy("server_id").orderBy("status_time")

servers_df.withColumn("next_status_time", lead("status_time").over(window_spec))\
.filter(col("session_status") == "start")\
.withColumn("duration_seconds", unix_timestamp("next_status_time") - unix_timestamp("status_time"))\
.agg((sum("duration_seconds") / 86400).cast('int').alias("total_uptime_days")).show()

+-----------------+
|total_uptime_days|
+-----------------+
|                1|
+-----------------+



In [418]:
servers_df.createOrReplaceTempView('serversdf')

spark.sql('''
 with one as (select server_id,status_time,lead(status_time,1) over(partition by server_id order by status_time) as nex_status__time,
                   session_status from serversdf),
      two as (select server_id, (unix_timestamp(nex_status__time) - unix_timestamp(status_time)) as duration_sec 
                   from one where session_status = "start")
              select cast(sum((duration_sec/86400)) as int) as total_uptime_days from two
         ''').show()

+-----------------+
|total_uptime_days|
+-----------------+
|                1|
+-----------------+



#### 115 241 Consecutive Available Seats II M

In [181]:
cinema_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/115m_cinema.csv'))

In [248]:
cinema_df.printSchema()

root
 |-- seat_id: integer (nullable = true)
 |-- free: integer (nullable = true)



In [297]:
window_spec = Window.orderBy(col('seat_id').asc())
window_rnk = Window.partitionBy('free','diff').orderBy(col('seat_id').asc(),col('diff').asc())
window_cnt = Window.partitionBy('diff')

cinema_df.withColumn('led',lead('free',1).over(window_spec))\
.withColumn('dif',expr("free - led "))\
.select('seat_id','free','led',coalesce('dif',lit('0')).alias('diff'))\
.withColumn('rnk',rank().over(window_rnk))\
.withColumn('cnt',count('diff').over(window_cnt))\
.withColumn('min',min('rnk').over(window_cnt))\
.withColumn('max',max('rnk').over(window_cnt))\
.withColumn('first_seat_id',when( ((col('diff') == 0) & (col('rnk') == col('min'))),col('seat_id')))\
.withColumn('last_seat_id',when( ((col('diff') == 0) & (col('rnk') == col('max'))),col('seat_id')))\
.withColumn('consecutive_seats_len',when((col('diff') == 0) & (col('rnk') == 1)  ,col('cnt')))\
.select( sum(coalesce('first_seat_id',lit(0))).alias('first_seat_id'),
         sum(coalesce('last_seat_id',lit(0))).alias('last_seat_id'),
         sum(coalesce('consecutive_seats_len',lit(0))).alias('consecutive_seats_len')).show()

+-------------+------------+---------------------+
|first_seat_id|last_seat_id|consecutive_seats_len|
+-------------+------------+---------------------+
|            3|           5|                    3|
+-------------+------------+---------------------+



24/09/18 20:53:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [337]:
cinema_df.createOrReplaceTempView('cinemadf')

spark.sql('''
 with one as (select seat_id,free,lead(free,1) over(order by seat_id asc) as ledfree,
                   coalesce((free - lead(free,1) over(order by seat_id asc)),0) as diff from cinemadf),
      two as (select seat_id,free,ledfree,diff,rank() over(partition by free,diff order by seat_id asc,diff asc) as rnk,
                   count(diff) over(partition by diff) as cnt from one),
    three as (select seat_id,free,ledfree,diff,rnk,cnt,
                     min(rnk) over(partition by diff) as mn,
                     max(rnk) over(partition by diff) as mx from two),
     four as (select seat_id,free,ledfree,diff,rnk,cnt,mn,mx,
                     case when (mn = rnk) and (diff = 0) then seat_id end as first_seat_id, 
                     case when (mx = rnk) and (diff = 0) then seat_id end as last_seat_id,
                     case when (rnk = 1)  and (diff = 0) then cnt end cntseat from three)
              select sum(coalesce(first_seat_id,0)) as first_seat_id,
                     sum(coalesce(last_seat_id,0)) as last_seat_id,
                     sum(coalesce(cntseat,0)) as consecutive_seats_len from four
         ''').show()

+-------------+------------+---------------------+
|first_seat_id|last_seat_id|consecutive_seats_len|
+-------------+------------+---------------------+
|            3|           5|                    3|
+-------------+------------+---------------------+



24/09/18 21:44:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 116 243 alculate Parking Fees and Duration M

In [182]:
parking_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/116m_parking.csv'))

In [341]:
parking_df.printSchema()

root
 |-- lot_id: integer (nullable = true)
 |-- car_id: integer (nullable = true)
 |-- entry_time: string (nullable = true)
 |-- exit_time: string (nullable = true)
 |-- fee_paid: double (nullable = true)



In [372]:
window_spec = Window.partitionBy('car_id','lot_id')
window_sum = Window.partitionBy('car_id')
window_rnk = Window.partitionBy('car_id').orderBy(col('car_id').desc(),col('lot_id').asc())

parking_df.withColumn('diff',expr("unix_timestamp(exit_time) - unix_timestamp(entry_time)")/3600)\
.withColumn('total_fee_paid',sum('fee_paid').over(window_sum))\
.withColumn('total_houurs',sum('diff').over(window_sum))\
.withColumn('avg_hourly_fee',round(expr("total_fee_paid/total_houurs"),2))\
.withColumn('most_time_lot',sum('diff').over(window_spec))\
.withColumn('rnk',rank().over(window_rnk)).filter(col('rnk') == 1)\
.select('car_id','total_fee_paid','avg_hourly_fee','lot_id').distinct().orderBy('car_id').show()

+------+--------------+--------------+------+
|car_id|total_fee_paid|avg_hourly_fee|lot_id|
+------+--------------+--------------+------+
|  1001|          18.0|           2.4|     1|
|  1002|           6.0|          1.33|     2|
+------+--------------+--------------+------+



In [390]:
parking_df.createOrReplaceTempView('parkingdf')

spark.sql('''
 with one as (select lot_id,car_id,entry_time,exit_time,((unix_timestamp(exit_time) - unix_timestamp(entry_time))/3600) as diff,
                   fee_paid,sum(fee_paid) over(partition by car_id) as total_fee_paid from parkingdf),
      two as (select lot_id,car_id,diff,fee_paid,total_fee_paid,sum(diff) over(partition by car_id) as total_hours,
                   sum(diff) over(partition by car_id,lot_id) as most_time_lot,
                   rank() over(partition by car_id order by car_id desc, lot_id asc) as rnk from one),
    three as (select lot_id,car_id,diff,fee_paid,total_fee_paid,total_hours,most_time_lot,
                    round(total_fee_paid/total_hours,2) as avg_hourly_fee,rnk  from two)
              select distinct car_id,total_fee_paid,avg_hourly_fee,lot_id from three where rnk = 1 order by car_id
         ''').show()

+------+--------------+--------------+------+
|car_id|total_fee_paid|avg_hourly_fee|lot_id|
+------+--------------+--------------+------+
|  1001|          18.0|           2.4|     1|
|  1002|           6.0|          1.33|     2|
+------+--------------+--------------+------+



#### 117 245 Find Top Scoring Students M

In [183]:
students_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/117m_students.csv'))

In [184]:
courses_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/117m_courses.csv'))

In [185]:
enrollments_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/117m_enrollments.csv'))

In [394]:
students_df.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- major: string (nullable = true)



In [395]:
courses_df.printSchema()

root
 |-- course_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- credits: integer (nullable = true)
 |-- major: string (nullable = true)



In [397]:
enrollments_df.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- course_id: integer (nullable = true)
 |-- semester: string (nullable = true)
 |-- grade: string (nullable = true)



In [403]:
window_spec = Window.partitionBy('student_id','grade')

enrollments_df.withColumn('cnt',count('grade').over(window_spec)).filter(col('cnt') == 2)\
.select('student_id').distinct().show()

+----------+
|student_id|
+----------+
|         1|
|         3|
+----------+



In [408]:
students_df.createOrReplaceTempView('studentsdf')
courses_df.createOrReplaceTempView('coursesdf')
enrollments_df.createOrReplaceTempView('enrollmentsdf')

spark.sql('''
 with one as (select student_id,count(grade) over(partition by student_id,grade)  cnt from enrollmentsdf)
              select distinct student_id from one where cnt = 2 order by student_id
         ''').show()

+----------+
|student_id|
+----------+
|         1|
|         3|
+----------+



#### 118 247 itwise User Permissions Analysis M

In [186]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/118m_user.csv'))

In [8]:
users_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- permissions: integer (nullable = true)



In [90]:
window_spec = Window.orderBy(col('user_id').asc())

users_df.withColumn('one',lead('permissions',1).over(window_spec))\
.withColumn('two',lead('permissions',2).over(window_spec))\
.withColumn('three',lead('permissions',3).over(window_spec)).filter(col('user_id') == 1)\
.withColumn('common_perms',expr("permissions & one & two & three"))\
.withColumn('any_perms',expr("permissions | one | two | three")).select('common_perms','any_perms').show()

+------------+---------+
|common_perms|any_perms|
+------------+---------+
|           0|       15|
+------------+---------+



24/09/19 19:16:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [95]:
users_df.createOrReplaceTempView('usersdf')

spark.sql('''
 with one as (select user_id,permissions,
                  lead(permissions,1) over(order by user_id asc) as one,
                  lead(permissions,2) over(order by user_id asc) as two,
                  lead(permissions,3) over(order by user_id asc) as three
                  from usersdf)
             select (permissions & one & two & three) as common_perms,(permissions | one | two | three) as any_perms 
                  from one where user_id = 1
         ''').show()

+------------+---------+
|common_perms|any_perms|
+------------+---------+
|           0|       15|
+------------+---------+



24/09/19 19:24:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 119 248 Odd and Even Transactions M

In [187]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/119m_transactions.csv'))

In [14]:
transactions_df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- amount: integer (nullable = true)
 |-- transaction_date: string (nullable = true)



In [20]:
window_spec = Window.partitionBy('transaction_date')

transactions_df.withColumn('even',when(col('amount')%2 == 0,col('amount')))\
.withColumn('odd',when(col('amount')%2 !=0,col('amount')))\
.withColumn('odd_sum',sum('odd').over(window_spec))\
.withColumn('even_sum',sum('even').over(window_spec))\
.select('transaction_date',coalesce('odd_sum',lit('0')).alias('odd_sum'),coalesce('even_sum',lit('0')).alias('even_sum'))\
.distinct().show()

+----------------+-------+--------+
|transaction_date|odd_sum|even_sum|
+----------------+-------+--------+
|      2024-07-01|     75|     350|
|      2024-07-02|      0|     350|
|      2024-07-03|      0|     120|
+----------------+-------+--------+



In [24]:
transactions_df.createOrReplaceTempView('transactionsdf')

spark.sql('''
 with one as (select amount,transaction_date,case when amount%2 == 0 then amount end as even,
                                         case when amount%2 != 0 then amount end as odd from transactionsdf),
      two as (select transaction_date,sum(even) over(partition by transaction_date) as even_sum,
                                      sum(odd) over(partition by transaction_date) as odd_sum from one)
              select distinct transaction_date,coalesce(odd_sum,'0') as odd_sum,even_sum from two
         ''').show()

+----------------+-------+--------+
|transaction_date|odd_sum|even_sum|
+----------------+-------+--------+
|      2024-07-01|     75|     350|
|      2024-07-02|      0|     350|
|      2024-07-03|      0|     120|
+----------------+-------+--------+



#### 120 249 Customer Purchasing Behavior Analysis M

In [188]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/120m_transactions.csv'))

In [189]:
products_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/120m_products.csv'))

In [30]:
transactions_df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- amount: double (nullable = true)



In [31]:
products_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)



In [68]:
window_spec = Window.partitionBy('customer_id')
window_cat = Window.partitionBy('customer_id','category')
window_rnk = Window.partitionBy('customer_id').orderBy(col('transaction_date').desc())

products_df = products_df.withColumnRenamed('product_id','productid')
df = products_df.join(transactions_df,products_df.productid == transactions_df.product_id,'inner')

df1 = df.withColumn('total_amount',sum('amount').over(window_spec))\
.withColumn('transaction_count',count('transaction_id').over(window_spec))\
.withColumn('unique_categories',count('category').over(window_spec))\
.withColumn('avg_transaction_amount',expr("total_amount/transaction_count"))\
.withColumn('top_cat',count('category').over(window_cat))\
.withColumn('top_category',rank().over(window_rnk))\
.withColumn('loyalty_score',expr("(transaction_count * 10) + (total_amount/100)"))\
.select('customer_id','total_amount','transaction_count','unique_categories','avg_transaction_amount',
        'category','loyalty_score','top_category').distinct().filter(col('top_category') == 1)\
.select('customer_id','total_amount','transaction_count','unique_categories','avg_transaction_amount',
        'category','loyalty_score').show()

+-----------+------------+-----------------+-----------------+----------------------+--------+-------------+
|customer_id|total_amount|transaction_count|unique_categories|avg_transaction_amount|category|loyalty_score|
+-----------+------------+-----------------+-----------------+----------------------+--------+-------------+
|        101|       450.0|                3|                3|                 150.0|       C|         34.5|
|        102|       300.0|                2|                2|                 150.0|       C|         23.0|
+-----------+------------+-----------------+-----------------+----------------------+--------+-------------+



In [80]:
transactions_df.createOrReplaceTempView('transactionsdf')
products_df.createOrReplaceTempView('productsdf')

spark.sql('''
 with one as (select customer_id,product_id,transaction_date,amount,category,price,
                   sum(amount) over(partition by customer_id) as total_amount,
                   count(transaction_id) over(partition by customer_id) as transaction_count,
                   count(category) over(partition by customer_id) as unique_categories
                   from transactionsdf join productsdf on transactionsdf.product_id = productsdf.productid),
      two as (select customer_id,product_id,transaction_date,amount,category,price,total_amount,
                   transaction_count,unique_categories,(total_amount/transaction_count) as avg_transaction_amount,
                   ((transaction_count * 10) + (total_amount/100)) as loyalty_score,
                   count(category) over(partition by 'customer_id','category') as top_cat
                   from one),
    three as (select customer_id,product_id,transaction_date,amount,category,price,total_amount,
                   transaction_count,unique_categories,avg_transaction_amount,loyalty_score,top_cat,
                   rank() over(partition by customer_id order by transaction_date desc) as rnk from two)
              select customer_id,total_amount,transaction_count,unique_categories,avg_transaction_amount,category,loyalty_score
                     from three where rnk = 1
        ''').show()

+-----------+------------+-----------------+-----------------+----------------------+--------+-------------+
|customer_id|total_amount|transaction_count|unique_categories|avg_transaction_amount|category|loyalty_score|
+-----------+------------+-----------------+-----------------+----------------------+--------+-------------+
|        101|       450.0|                3|                3|                 150.0|       C|         34.5|
|        102|       300.0|                2|                2|                 150.0|       C|         23.0|
+-----------+------------+-----------------+-----------------+----------------------+--------+-------------+

